In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import seaborn as sns
from IPython.display import display
import datetime
import numpy as np
from statsmodels import robust
from sklearn import preprocessing
from sklearn.preprocessing import Imputer

In [2]:
def null_ratio(df):
        null_count = df.isnull().sum()
        null_percent = 100 * df.isnull().sum()/len(df)
        null_table = pd.concat([null_count, null_percent], axis=1)
        null_table = null_table.rename(columns = {0 : 'Null Count', 1 : 'Null Percent'})
        return null_table.sort_values('Null Percent', ascending=0)

In [3]:
def return_all_rows(x):
    pd.set_option('display.max_rows', len(x))
    return x
    pd.reset_option('display.max_rows')
    
def return_all_columns(x):
    pd.set_option('display.max_columns', len(x))
    return x.head(5)
    pd.reset_option('display.max_columns')

In [4]:
def overview(df):
    print("Number of columns:", len(df.columns))
    print("Number of rows:", len(df.index))
    df.head(5)

In [5]:
def drop_dups(df):
    # list comprehension of the cols that end with '_y'
    y_drop = [x for x in df if x.endswith('_y')]
    df.drop(y_drop, axis=1, inplace=True)

In [6]:
def floatToString(inputValue):
    result = ('%.15f' % inputValue).rstrip('0').rstrip('.')
    return '0' if result == '-0' else result

In [7]:
def mad(arr):
    """ 
    Get Median Absolute Deviation and multiple by 1.486 to mimic standard deviation
        https://www.ibm.com/support/knowledgecenter/SSWLVY_1.0.0/com.ibm.spss.analyticcatalyst.help/analytic_catalyst/modified_z.html
    Median Absolute Deviation: a "Robust" version of standard deviation.
        Indices variabililty of the sample.
        https://en.wikipedia.org/wiki/Median_absolute_deviation
    """
    arr = np.ma.array(arr).compressed() # should be faster to not use masked arrays.
    med = np.nanmedian(arr)
    mad = np.nanmedian(np.abs(arr - med))
    # Multiply coefficient by 1.486 to mimic Standard Deviation (source: IBM)
    return 1.486 * mad

In [8]:
def meanad(arr):
    """ 
    Get Mean Absolute Deviation and multiple by 1.253314 to mimic standard deviation
        https://www.ibm.com/support/knowledgecenter/SSWLVY_1.0.0/com.ibm.spss.analyticcatalyst.help/analytic_catalyst/modified_z.html
    Mean Absolute Deviation: a "Robust" version of standard deviation.
        Indices variabililty of the sample.
        https://en.wikipedia.org/wiki/Mean_absolute_deviation
    """
    arr = np.ma.array(arr).compressed() # should be faster to not use masked arrays.
    med = np.nanmedian(arr)
    mad = np.nanmean(np.abs(arr - med))
    # Multiply coefficient by 1.253314 to mimic Standard Deviation (source: IBM)
    return 1.253314 * mad

In [9]:
def fill_na(column):
    try:
        median = np.nanmedian(column)
        column = column.fillna(median)
        return column
    except:
        return column

In [10]:
def clip_outliers(column):
    
    # Use try in case all null column
    try:
        floor = column.quantile(0.03)
        ceiling = column.quantile(0.97)
        column = column.clip(floor, ceiling)
        return column
    # If error, return as is
    except:
        return column

In [11]:
# df - Load
df = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_Financial_Ratios_Firm_Level.csv')

In [12]:
# Link Table - Load
link_table = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_CRSP_Compustat_Merged_Linking_Table.csv')

In [13]:
# Link Table - Clean - LINKENDDT
# Replace 'E' values with future date (current unique ID)
link_table['LINKENDDT'] = link_table['LINKENDDT'].str.replace('E','20200101').astype(int)

In [14]:
# Link Table - Clean - LPERMNO
# Convert PERMNO from int to object
link_table['LPERMNO'] = link_table['LPERMNO'].astype(object)

In [15]:
# Link Table - Clean - LINKDT & LINKENDDT
# Get start and end dates for link to determine identification during given time
link_table['LINKDT'] = pd.to_datetime(link_table['LINKDT'], format='%Y%m%d')
link_table['LINKENDDT'] = pd.to_datetime(link_table['LINKENDDT'], format='%Y%m%d')

In [16]:
# Merge - df & Link Table
# Merge financial ratio suite with link table to build more foreign keys
df = pd.merge(df, link_table, on='gvkey', how='left', suffixes=('', '_y'))

In [17]:
# df - Enrich - Time Features
# Convert to time
df['public_date'] = pd.to_datetime(df['public_date'])
# Create year
df['year'] = df['public_date'].dt.year
# Create month
df['month'] = df['public_date'].dt.month
# Create year-month
df["year-month"] = df['public_date'].apply(lambda x: x.strftime('%Y-%m'))

In [18]:
# df - Enrich - Time-based Unique Identifiers
# Create Unique Identifier for links from other databases for specific date-time
df["GVKEY-year-month"] = df["gvkey"].map(str) + "-" + df["year-month"]
df["CUSIP-year-month"] = df["cusip"].map(str) + "-" + df["year-month"]
df["TIC-year-month"] = df["tic"].map(str) + "-" + df["year-month"]
df["PERMNO-year-month"] = df["LPERMNO"].map(str) + "-" + df["year-month"]

In [19]:
# df - Subset - Correct Unique ID's per public data
# Filter dataframe to rows of unique identifiers during correct link range
df = df[(df.public_date >= df.LINKDT) & (df.public_date <= df.LINKENDDT)]

In [20]:
# CRSP - Load
# Load CRSP/Compustat Merged Database - Security Monthly from Wharton
CRSP_comp_merge = pd.read_csv('c:/users/sconner/desktop/W205_Final/sample_data/Sample_CRSP_Compustat_Merged_Security_Monthly.csv', low_memory=False)

In [21]:
# CRSP - Enrich - Time Features

# Convert to date-time
CRSP_comp_merge['datadate'] = pd.to_datetime(CRSP_comp_merge['datadate'])
# Create year
CRSP_comp_merge['year'] = CRSP_comp_merge['datadate'].dt.year
# Create month
CRSP_comp_merge['month'] = CRSP_comp_merge['datadate'].dt.month
# Create year-month
CRSP_comp_merge["year-month"] = CRSP_comp_merge['datadate'].apply(lambda x: x.strftime('%Y-%m'))

In [22]:
# CRSP - Enrich - Time-based Unique Identifiers
# CRSP - Unique ID in time (GVKEY + time-month)
CRSP_comp_merge["GVKEY-year-month"] = CRSP_comp_merge["GVKEY"].map(str) + "-" + CRSP_comp_merge["year-month"]

In [23]:
# CRSP - Sort - Date and GVKEY
CRSP_comp_merge = CRSP_comp_merge.sort_values(by=['GVKEY','datadate'], ascending=[True,True])

In [24]:
# CRSP - Enrich - Forward (PRCCM -- Price - Close - Monthly) 1 – 36 months
CRSP_comp_merge['forward_one_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-1)
CRSP_comp_merge['forward_two_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-2)
CRSP_comp_merge['forward_three_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-3)
CRSP_comp_merge['forward_four_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-4)
CRSP_comp_merge['forward_five_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-5)
CRSP_comp_merge['forward_six_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-6)
CRSP_comp_merge['forward_seven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-7)
CRSP_comp_merge['forward_eight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-8)
CRSP_comp_merge['forward_nine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-9)
CRSP_comp_merge['forward_ten_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-10)
CRSP_comp_merge['forward_eleven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-11)
CRSP_comp_merge['forward_twelve_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-12)
CRSP_comp_merge['forward_thirteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-13)
CRSP_comp_merge['forward_fourteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-14)
CRSP_comp_merge['forward_fifteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-15)
CRSP_comp_merge['forward_sixteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-16)
CRSP_comp_merge['forward_seventeen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-17)
CRSP_comp_merge['forward_eighteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-18)
CRSP_comp_merge['forward_nineteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-19)
CRSP_comp_merge['forward_twenty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-20)
CRSP_comp_merge['forward_twentyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-21)
CRSP_comp_merge['forward_twentytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-22)
CRSP_comp_merge['forward_twentythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-23)
CRSP_comp_merge['forward_twentyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-24)
CRSP_comp_merge['forward_twentyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-25)
CRSP_comp_merge['forward_twentysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-26)
CRSP_comp_merge['forward_twentyseven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-27)
CRSP_comp_merge['forward_twentyeight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-28)
CRSP_comp_merge['forward_twentynine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-29)
CRSP_comp_merge['forward_thirty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-30)
CRSP_comp_merge['forward_thirtyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-31)
CRSP_comp_merge['forward_thirtytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-32)
CRSP_comp_merge['forward_thirtythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-33)
CRSP_comp_merge['forward_thirtyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-34)
CRSP_comp_merge['forward_thirtyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-35)
CRSP_comp_merge['forward_thirtysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(-36)

In [25]:
# CRSP - Enrich - Forward (AJEXM -- Cumulative Adjustment Factor - Ex Date -Monthly) 1 – 36 months
CRSP_comp_merge['forward_one_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-1)
CRSP_comp_merge['forward_two_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-2)
CRSP_comp_merge['forward_three_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-3)
CRSP_comp_merge['forward_four_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-4)
CRSP_comp_merge['forward_five_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-5)
CRSP_comp_merge['forward_six_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-6)
CRSP_comp_merge['forward_seven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-7)
CRSP_comp_merge['forward_eight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-8)
CRSP_comp_merge['forward_nine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-9)
CRSP_comp_merge['forward_ten_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-10)
CRSP_comp_merge['forward_eleven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-11)
CRSP_comp_merge['forward_twelve_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-12)
CRSP_comp_merge['forward_thirteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-13)
CRSP_comp_merge['forward_fourteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-14)
CRSP_comp_merge['forward_fifteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-15)
CRSP_comp_merge['forward_sixteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-16)
CRSP_comp_merge['forward_seventeen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-17)
CRSP_comp_merge['forward_eighteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-18)
CRSP_comp_merge['forward_nineteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-19)
CRSP_comp_merge['forward_twenty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-20)
CRSP_comp_merge['forward_twentyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-21)
CRSP_comp_merge['forward_twentytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-22)
CRSP_comp_merge['forward_twentythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-23)
CRSP_comp_merge['forward_twentyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-24)
CRSP_comp_merge['forward_twentyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-25)
CRSP_comp_merge['forward_twentysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-26)
CRSP_comp_merge['forward_twentyseven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-27)
CRSP_comp_merge['forward_twentyeight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-28)
CRSP_comp_merge['forward_twentynine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-29)
CRSP_comp_merge['forward_thirty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-30)
CRSP_comp_merge['forward_thirtyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-31)
CRSP_comp_merge['forward_thirtytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-32)
CRSP_comp_merge['forward_thirtythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-33)
CRSP_comp_merge['forward_thirtyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-34)
CRSP_comp_merge['forward_thirtyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-35)
CRSP_comp_merge['forward_thirtysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(-36)

In [26]:
# CRSP - Enrich - Forward (TRFM -- Monthly Total Return Factor) 1 – 36 months
CRSP_comp_merge['forward_one_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-1)
CRSP_comp_merge['forward_two_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-2)
CRSP_comp_merge['forward_three_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-3)
CRSP_comp_merge['forward_four_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-4)
CRSP_comp_merge['forward_five_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-5)
CRSP_comp_merge['forward_six_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-6)
CRSP_comp_merge['forward_seven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-7)
CRSP_comp_merge['forward_eight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-8)
CRSP_comp_merge['forward_nine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-9)
CRSP_comp_merge['forward_ten_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-10)
CRSP_comp_merge['forward_eleven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-11)
CRSP_comp_merge['forward_twelve_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-12)
CRSP_comp_merge['forward_thirteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-13)
CRSP_comp_merge['forward_fourteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-14)
CRSP_comp_merge['forward_fifteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-15)
CRSP_comp_merge['forward_sixteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-16)
CRSP_comp_merge['forward_seventeen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-17)
CRSP_comp_merge['forward_eighteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-18)
CRSP_comp_merge['forward_nineteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-19)
CRSP_comp_merge['forward_twenty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-20)
CRSP_comp_merge['forward_twentyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-21)
CRSP_comp_merge['forward_twentytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-22)
CRSP_comp_merge['forward_twentythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-23)
CRSP_comp_merge['forward_twentyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-24)
CRSP_comp_merge['forward_twentyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-25)
CRSP_comp_merge['forward_twentysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-26)
CRSP_comp_merge['forward_twentyseven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-27)
CRSP_comp_merge['forward_twentyeight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-28)
CRSP_comp_merge['forward_twentynine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-29)
CRSP_comp_merge['forward_thirty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-30)
CRSP_comp_merge['forward_thirtyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-31)
CRSP_comp_merge['forward_thirtytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-32)
CRSP_comp_merge['forward_thirtythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-33)
CRSP_comp_merge['forward_thirtyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-34)
CRSP_comp_merge['forward_thirtyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-35)
CRSP_comp_merge['forward_thirtysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(-36)


In [27]:
# CRSP - Enrich - Forward Return over number of months (1 - 36)
CRSP_comp_merge['forward_one_month_return'] = ((((CRSP_comp_merge['forward_one_month_prccm']/CRSP_comp_merge['forward_one_month_ajexm'])*CRSP_comp_merge['forward_one_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_two_month_return'] = ((((CRSP_comp_merge['forward_two_month_prccm']/CRSP_comp_merge['forward_two_month_ajexm'])*CRSP_comp_merge['forward_two_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_three_month_return'] = ((((CRSP_comp_merge['forward_three_month_prccm']/CRSP_comp_merge['forward_three_month_ajexm'])*CRSP_comp_merge['forward_three_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_four_month_return'] = ((((CRSP_comp_merge['forward_four_month_prccm']/CRSP_comp_merge['forward_four_month_ajexm'])*CRSP_comp_merge['forward_four_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_five_month_return'] = ((((CRSP_comp_merge['forward_five_month_prccm']/CRSP_comp_merge['forward_five_month_ajexm'])*CRSP_comp_merge['forward_five_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_six_month_return'] = ((((CRSP_comp_merge['forward_six_month_prccm']/CRSP_comp_merge['forward_six_month_ajexm'])*CRSP_comp_merge['forward_six_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_seven_month_return'] = ((((CRSP_comp_merge['forward_seven_month_prccm']/CRSP_comp_merge['forward_seven_month_ajexm'])*CRSP_comp_merge['forward_seven_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_eight_month_return'] = ((((CRSP_comp_merge['forward_eight_month_prccm']/CRSP_comp_merge['forward_eight_month_ajexm'])*CRSP_comp_merge['forward_eight_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_nine_month_return'] = ((((CRSP_comp_merge['forward_nine_month_prccm']/CRSP_comp_merge['forward_nine_month_ajexm'])*CRSP_comp_merge['forward_nine_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_ten_month_return'] = ((((CRSP_comp_merge['forward_ten_month_prccm']/CRSP_comp_merge['forward_ten_month_ajexm'])*CRSP_comp_merge['forward_ten_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_eleven_month_return'] = ((((CRSP_comp_merge['forward_eleven_month_prccm']/CRSP_comp_merge['forward_eleven_month_ajexm'])*CRSP_comp_merge['forward_eleven_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twelve_month_return'] = ((((CRSP_comp_merge['forward_twelve_month_prccm']/CRSP_comp_merge['forward_twelve_month_ajexm'])*CRSP_comp_merge['forward_twelve_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirteen_month_return'] = ((((CRSP_comp_merge['forward_thirteen_month_prccm']/CRSP_comp_merge['forward_thirteen_month_ajexm'])*CRSP_comp_merge['forward_thirteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_fourteen_month_return'] = ((((CRSP_comp_merge['forward_fourteen_month_prccm']/CRSP_comp_merge['forward_fourteen_month_ajexm'])*CRSP_comp_merge['forward_fourteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_fifteen_month_return'] = ((((CRSP_comp_merge['forward_fifteen_month_prccm']/CRSP_comp_merge['forward_fifteen_month_ajexm'])*CRSP_comp_merge['forward_fifteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_sixteen_month_return'] = ((((CRSP_comp_merge['forward_sixteen_month_prccm']/CRSP_comp_merge['forward_sixteen_month_ajexm'])*CRSP_comp_merge['forward_sixteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_seventeen_month_return'] = ((((CRSP_comp_merge['forward_seventeen_month_prccm']/CRSP_comp_merge['forward_seventeen_month_ajexm'])*CRSP_comp_merge['forward_seventeen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_eighteen_month_return'] = ((((CRSP_comp_merge['forward_eighteen_month_prccm']/CRSP_comp_merge['forward_eighteen_month_ajexm'])*CRSP_comp_merge['forward_eighteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_nineteen_month_return'] = ((((CRSP_comp_merge['forward_nineteen_month_prccm']/CRSP_comp_merge['forward_nineteen_month_ajexm'])*CRSP_comp_merge['forward_nineteen_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twenty_month_return'] = ((((CRSP_comp_merge['forward_twenty_month_prccm']/CRSP_comp_merge['forward_twenty_month_ajexm'])*CRSP_comp_merge['forward_twenty_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyone_month_return'] = ((((CRSP_comp_merge['forward_twentyone_month_prccm']/CRSP_comp_merge['forward_twentyone_month_ajexm'])*CRSP_comp_merge['forward_twentyone_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentytwo_month_return'] = ((((CRSP_comp_merge['forward_twentytwo_month_prccm']/CRSP_comp_merge['forward_twentytwo_month_ajexm'])*CRSP_comp_merge['forward_twentytwo_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentythree_month_return'] = ((((CRSP_comp_merge['forward_twentythree_month_prccm']/CRSP_comp_merge['forward_twentythree_month_ajexm'])*CRSP_comp_merge['forward_twentythree_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyfour_month_return'] = ((((CRSP_comp_merge['forward_twentyfour_month_prccm']/CRSP_comp_merge['forward_twentyfour_month_ajexm'])*CRSP_comp_merge['forward_twentyfour_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyfive_month_return'] = ((((CRSP_comp_merge['forward_twentyfive_month_prccm']/CRSP_comp_merge['forward_twentyfive_month_ajexm'])*CRSP_comp_merge['forward_twentyfive_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentysix_month_return'] = ((((CRSP_comp_merge['forward_twentysix_month_prccm']/CRSP_comp_merge['forward_twentysix_month_ajexm'])*CRSP_comp_merge['forward_twentysix_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyseven_month_return'] = ((((CRSP_comp_merge['forward_twentyseven_month_prccm']/CRSP_comp_merge['forward_twentyseven_month_ajexm'])*CRSP_comp_merge['forward_twentyseven_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentyeight_month_return'] = ((((CRSP_comp_merge['forward_twentyeight_month_prccm']/CRSP_comp_merge['forward_twentyeight_month_ajexm'])*CRSP_comp_merge['forward_twentyeight_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_twentynine_month_return'] = ((((CRSP_comp_merge['forward_twentynine_month_prccm']/CRSP_comp_merge['forward_twentynine_month_ajexm'])*CRSP_comp_merge['forward_twentynine_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirty_month_return'] = ((((CRSP_comp_merge['forward_thirty_month_prccm']/CRSP_comp_merge['forward_thirty_month_ajexm'])*CRSP_comp_merge['forward_thirty_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtyone_month_return'] = ((((CRSP_comp_merge['forward_thirtyone_month_prccm']/CRSP_comp_merge['forward_thirtyone_month_ajexm'])*CRSP_comp_merge['forward_thirtyone_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtytwo_month_return'] = ((((CRSP_comp_merge['forward_thirtytwo_month_prccm']/CRSP_comp_merge['forward_thirtytwo_month_ajexm'])*CRSP_comp_merge['forward_thirtytwo_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtythree_month_return'] = ((((CRSP_comp_merge['forward_thirtythree_month_prccm']/CRSP_comp_merge['forward_thirtythree_month_ajexm'])*CRSP_comp_merge['forward_thirtythree_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtyfour_month_return'] = ((((CRSP_comp_merge['forward_thirtyfour_month_prccm']/CRSP_comp_merge['forward_thirtyfour_month_ajexm'])*CRSP_comp_merge['forward_thirtyfour_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtyfive_month_return'] = ((((CRSP_comp_merge['forward_thirtyfive_month_prccm']/CRSP_comp_merge['forward_thirtyfive_month_ajexm'])*CRSP_comp_merge['forward_thirtyfive_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100
CRSP_comp_merge['forward_thirtysix_month_return'] = ((((CRSP_comp_merge['forward_thirtysix_month_prccm']/CRSP_comp_merge['forward_thirtysix_month_ajexm'])*CRSP_comp_merge['forward_thirtysix_month_trfm'])/((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['trfm']))-1)*100

In [28]:
# CRSP - Enrich - Past (PRCCM -- Price - Close - Monthly) 1 – 36 months
CRSP_comp_merge['past_one_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(1)
CRSP_comp_merge['past_two_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(2)
CRSP_comp_merge['past_three_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(3)
CRSP_comp_merge['past_four_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(4)
CRSP_comp_merge['past_five_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(5)
CRSP_comp_merge['past_six_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(6)
CRSP_comp_merge['past_seven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(7)
CRSP_comp_merge['past_eight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(8)
CRSP_comp_merge['past_nine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(9)
CRSP_comp_merge['past_ten_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(10)
CRSP_comp_merge['past_eleven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(11)
CRSP_comp_merge['past_twelve_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(12)
CRSP_comp_merge['past_thirteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(13)
CRSP_comp_merge['past_fourteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(14)
CRSP_comp_merge['past_fifteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(15)
CRSP_comp_merge['past_sixteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(16)
CRSP_comp_merge['past_seventeen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(17)
CRSP_comp_merge['past_eighteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(18)
CRSP_comp_merge['past_nineteen_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(19)
CRSP_comp_merge['past_twenty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(20)
CRSP_comp_merge['past_twentyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(21)
CRSP_comp_merge['past_twentytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(22)
CRSP_comp_merge['past_twentythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(23)
CRSP_comp_merge['past_twentyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(24)
CRSP_comp_merge['past_twentyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(25)
CRSP_comp_merge['past_twentysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(26)
CRSP_comp_merge['past_twentyseven_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(27)
CRSP_comp_merge['past_twentyeight_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(28)
CRSP_comp_merge['past_twentynine_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(29)
CRSP_comp_merge['past_thirty_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(30)
CRSP_comp_merge['past_thirtyone_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(31)
CRSP_comp_merge['past_thirtytwo_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(32)
CRSP_comp_merge['past_thirtythree_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(33)
CRSP_comp_merge['past_thirtyfour_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(34)
CRSP_comp_merge['past_thirtyfive_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(35)
CRSP_comp_merge['past_thirtysix_month_prccm'] = CRSP_comp_merge.groupby('GVKEY')['prccm'].shift(36)

In [29]:
# CRSP - Enrich -  Past (AJEXM -- Cumulative Adjustment Factor - Ex Date -Monthly) 1 – 36 months
CRSP_comp_merge['past_one_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(1)
CRSP_comp_merge['past_two_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(2)
CRSP_comp_merge['past_three_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(3)
CRSP_comp_merge['past_four_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(4)
CRSP_comp_merge['past_five_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(5)
CRSP_comp_merge['past_six_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(6)
CRSP_comp_merge['past_seven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(7)
CRSP_comp_merge['past_eight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(8)
CRSP_comp_merge['past_nine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(9)
CRSP_comp_merge['past_ten_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(10)
CRSP_comp_merge['past_eleven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(11)
CRSP_comp_merge['past_twelve_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(12)
CRSP_comp_merge['past_thirteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(13)
CRSP_comp_merge['past_fourteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(14)
CRSP_comp_merge['past_fifteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(15)
CRSP_comp_merge['past_sixteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(16)
CRSP_comp_merge['past_seventeen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(17)
CRSP_comp_merge['past_eighteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(18)
CRSP_comp_merge['past_nineteen_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(19)
CRSP_comp_merge['past_twenty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(20)
CRSP_comp_merge['past_twentyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(21)
CRSP_comp_merge['past_twentytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(22)
CRSP_comp_merge['past_twentythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(23)
CRSP_comp_merge['past_twentyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(24)
CRSP_comp_merge['past_twentyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(25)
CRSP_comp_merge['past_twentysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(26)
CRSP_comp_merge['past_twentyseven_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(27)
CRSP_comp_merge['past_twentyeight_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(28)
CRSP_comp_merge['past_twentynine_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(29)
CRSP_comp_merge['past_thirty_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(30)
CRSP_comp_merge['past_thirtyone_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(31)
CRSP_comp_merge['past_thirtytwo_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(32)
CRSP_comp_merge['past_thirtythree_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(33)
CRSP_comp_merge['past_thirtyfour_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(34)
CRSP_comp_merge['past_thirtyfive_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(35)
CRSP_comp_merge['past_thirtysix_month_ajexm'] = CRSP_comp_merge.groupby('GVKEY')['ajexm'].shift(36)

In [30]:
# CRSP - Enrich -  Past (TRFM -- Monthly Total Return Factor) 1 – 36 months
CRSP_comp_merge['past_one_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(1)
CRSP_comp_merge['past_two_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(2)
CRSP_comp_merge['past_three_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(3)
CRSP_comp_merge['past_four_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(4)
CRSP_comp_merge['past_five_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(5)
CRSP_comp_merge['past_six_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(6)
CRSP_comp_merge['past_seven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(7)
CRSP_comp_merge['past_eight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(8)
CRSP_comp_merge['past_nine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(9)
CRSP_comp_merge['past_ten_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(10)
CRSP_comp_merge['past_eleven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(11)
CRSP_comp_merge['past_twelve_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(12)
CRSP_comp_merge['past_thirteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(13)
CRSP_comp_merge['past_fourteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(14)
CRSP_comp_merge['past_fifteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(15)
CRSP_comp_merge['past_sixteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(16)
CRSP_comp_merge['past_seventeen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(17)
CRSP_comp_merge['past_eighteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(18)
CRSP_comp_merge['past_nineteen_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(19)
CRSP_comp_merge['past_twenty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(20)
CRSP_comp_merge['past_twentyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(21)
CRSP_comp_merge['past_twentytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(22)
CRSP_comp_merge['past_twentythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(23)
CRSP_comp_merge['past_twentyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(24)
CRSP_comp_merge['past_twentyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(25)
CRSP_comp_merge['past_twentysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(26)
CRSP_comp_merge['past_twentyseven_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(27)
CRSP_comp_merge['past_twentyeight_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(28)
CRSP_comp_merge['past_twentynine_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(29)
CRSP_comp_merge['past_thirty_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(30)
CRSP_comp_merge['past_thirtyone_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(31)
CRSP_comp_merge['past_thirtytwo_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(32)
CRSP_comp_merge['past_thirtythree_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(33)
CRSP_comp_merge['past_thirtyfour_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(34)
CRSP_comp_merge['past_thirtyfive_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(35)
CRSP_comp_merge['past_thirtysix_month_trfm'] = CRSP_comp_merge.groupby('GVKEY')['trfm'].shift(36)

In [31]:
# CRSP - Enrich -  Past Return over number of months (1 - 36)
CRSP_comp_merge['past_one_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_one_month_trfm'])/((CRSP_comp_merge['past_one_month_prccm']/CRSP_comp_merge['past_one_month_ajexm'])*CRSP_comp_merge['past_one_month_trfm']))-1)*100
CRSP_comp_merge['past_two_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_two_month_trfm'])/((CRSP_comp_merge['past_two_month_prccm']/CRSP_comp_merge['past_two_month_ajexm'])*CRSP_comp_merge['past_two_month_trfm']))-1)*100
CRSP_comp_merge['past_three_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_three_month_trfm'])/((CRSP_comp_merge['past_three_month_prccm']/CRSP_comp_merge['past_three_month_ajexm'])*CRSP_comp_merge['past_three_month_trfm']))-1)*100
CRSP_comp_merge['past_four_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_four_month_trfm'])/((CRSP_comp_merge['past_four_month_prccm']/CRSP_comp_merge['past_four_month_ajexm'])*CRSP_comp_merge['past_four_month_trfm']))-1)*100
CRSP_comp_merge['past_five_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_five_month_trfm'])/((CRSP_comp_merge['past_five_month_prccm']/CRSP_comp_merge['past_five_month_ajexm'])*CRSP_comp_merge['past_five_month_trfm']))-1)*100
CRSP_comp_merge['past_six_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_six_month_trfm'])/((CRSP_comp_merge['past_six_month_prccm']/CRSP_comp_merge['past_six_month_ajexm'])*CRSP_comp_merge['past_six_month_trfm']))-1)*100
CRSP_comp_merge['past_seven_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_seven_month_trfm'])/((CRSP_comp_merge['past_seven_month_prccm']/CRSP_comp_merge['past_seven_month_ajexm'])*CRSP_comp_merge['past_seven_month_trfm']))-1)*100
CRSP_comp_merge['past_eight_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_eight_month_trfm'])/((CRSP_comp_merge['past_eight_month_prccm']/CRSP_comp_merge['past_eight_month_ajexm'])*CRSP_comp_merge['past_eight_month_trfm']))-1)*100
CRSP_comp_merge['past_nine_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_nine_month_trfm'])/((CRSP_comp_merge['past_nine_month_prccm']/CRSP_comp_merge['past_nine_month_ajexm'])*CRSP_comp_merge['past_nine_month_trfm']))-1)*100
CRSP_comp_merge['past_ten_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_ten_month_trfm'])/((CRSP_comp_merge['past_ten_month_prccm']/CRSP_comp_merge['past_ten_month_ajexm'])*CRSP_comp_merge['past_ten_month_trfm']))-1)*100
CRSP_comp_merge['past_eleven_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_eleven_month_trfm'])/((CRSP_comp_merge['past_eleven_month_prccm']/CRSP_comp_merge['past_eleven_month_ajexm'])*CRSP_comp_merge['past_eleven_month_trfm']))-1)*100
CRSP_comp_merge['past_twelve_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twelve_month_trfm'])/((CRSP_comp_merge['past_twelve_month_prccm']/CRSP_comp_merge['past_twelve_month_ajexm'])*CRSP_comp_merge['past_twelve_month_trfm']))-1)*100
CRSP_comp_merge['past_thirteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirteen_month_trfm'])/((CRSP_comp_merge['past_thirteen_month_prccm']/CRSP_comp_merge['past_thirteen_month_ajexm'])*CRSP_comp_merge['past_thirteen_month_trfm']))-1)*100
CRSP_comp_merge['past_fourteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_fourteen_month_trfm'])/((CRSP_comp_merge['past_fourteen_month_prccm']/CRSP_comp_merge['past_fourteen_month_ajexm'])*CRSP_comp_merge['past_fourteen_month_trfm']))-1)*100
CRSP_comp_merge['past_fifteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_fifteen_month_trfm'])/((CRSP_comp_merge['past_fifteen_month_prccm']/CRSP_comp_merge['past_fifteen_month_ajexm'])*CRSP_comp_merge['past_fifteen_month_trfm']))-1)*100
CRSP_comp_merge['past_sixteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_sixteen_month_trfm'])/((CRSP_comp_merge['past_sixteen_month_prccm']/CRSP_comp_merge['past_sixteen_month_ajexm'])*CRSP_comp_merge['past_sixteen_month_trfm']))-1)*100
CRSP_comp_merge['past_seventeen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_seventeen_month_trfm'])/((CRSP_comp_merge['past_seventeen_month_prccm']/CRSP_comp_merge['past_seventeen_month_ajexm'])*CRSP_comp_merge['past_seventeen_month_trfm']))-1)*100
CRSP_comp_merge['past_eighteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_eighteen_month_trfm'])/((CRSP_comp_merge['past_eighteen_month_prccm']/CRSP_comp_merge['past_eighteen_month_ajexm'])*CRSP_comp_merge['past_eighteen_month_trfm']))-1)*100
CRSP_comp_merge['past_nineteen_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_nineteen_month_trfm'])/((CRSP_comp_merge['past_nineteen_month_prccm']/CRSP_comp_merge['past_nineteen_month_ajexm'])*CRSP_comp_merge['past_nineteen_month_trfm']))-1)*100
CRSP_comp_merge['past_twenty_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twenty_month_trfm'])/((CRSP_comp_merge['past_twenty_month_prccm']/CRSP_comp_merge['past_twenty_month_ajexm'])*CRSP_comp_merge['past_twenty_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyone_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyone_month_trfm'])/((CRSP_comp_merge['past_twentyone_month_prccm']/CRSP_comp_merge['past_twentyone_month_ajexm'])*CRSP_comp_merge['past_twentyone_month_trfm']))-1)*100
CRSP_comp_merge['past_twentytwo_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentytwo_month_trfm'])/((CRSP_comp_merge['past_twentytwo_month_prccm']/CRSP_comp_merge['past_twentytwo_month_ajexm'])*CRSP_comp_merge['past_twentytwo_month_trfm']))-1)*100
CRSP_comp_merge['past_twentythree_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentythree_month_trfm'])/((CRSP_comp_merge['past_twentythree_month_prccm']/CRSP_comp_merge['past_twentythree_month_ajexm'])*CRSP_comp_merge['past_twentythree_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyfour_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyfour_month_trfm'])/((CRSP_comp_merge['past_twentyfour_month_prccm']/CRSP_comp_merge['past_twentyfour_month_ajexm'])*CRSP_comp_merge['past_twentyfour_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyfive_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyfive_month_trfm'])/((CRSP_comp_merge['past_twentyfive_month_prccm']/CRSP_comp_merge['past_twentyfive_month_ajexm'])*CRSP_comp_merge['past_twentyfive_month_trfm']))-1)*100
CRSP_comp_merge['past_twentysix_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentysix_month_trfm'])/((CRSP_comp_merge['past_twentysix_month_prccm']/CRSP_comp_merge['past_twentysix_month_ajexm'])*CRSP_comp_merge['past_twentysix_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyseven_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyseven_month_trfm'])/((CRSP_comp_merge['past_twentyseven_month_prccm']/CRSP_comp_merge['past_twentyseven_month_ajexm'])*CRSP_comp_merge['past_twentyseven_month_trfm']))-1)*100
CRSP_comp_merge['past_twentyeight_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentyeight_month_trfm'])/((CRSP_comp_merge['past_twentyeight_month_prccm']/CRSP_comp_merge['past_twentyeight_month_ajexm'])*CRSP_comp_merge['past_twentyeight_month_trfm']))-1)*100
CRSP_comp_merge['past_twentynine_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_twentynine_month_trfm'])/((CRSP_comp_merge['past_twentynine_month_prccm']/CRSP_comp_merge['past_twentynine_month_ajexm'])*CRSP_comp_merge['past_twentynine_month_trfm']))-1)*100
CRSP_comp_merge['past_thirty_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirty_month_trfm'])/((CRSP_comp_merge['past_thirty_month_prccm']/CRSP_comp_merge['past_thirty_month_ajexm'])*CRSP_comp_merge['past_thirty_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtyone_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtyone_month_trfm'])/((CRSP_comp_merge['past_thirtyone_month_prccm']/CRSP_comp_merge['past_thirtyone_month_ajexm'])*CRSP_comp_merge['past_thirtyone_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtytwo_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtytwo_month_trfm'])/((CRSP_comp_merge['past_thirtytwo_month_prccm']/CRSP_comp_merge['past_thirtytwo_month_ajexm'])*CRSP_comp_merge['past_thirtytwo_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtythree_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtythree_month_trfm'])/((CRSP_comp_merge['past_thirtythree_month_prccm']/CRSP_comp_merge['past_thirtythree_month_ajexm'])*CRSP_comp_merge['past_thirtythree_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtyfour_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtyfour_month_trfm'])/((CRSP_comp_merge['past_thirtyfour_month_prccm']/CRSP_comp_merge['past_thirtyfour_month_ajexm'])*CRSP_comp_merge['past_thirtyfour_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtyfive_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtyfive_month_trfm'])/((CRSP_comp_merge['past_thirtyfive_month_prccm']/CRSP_comp_merge['past_thirtyfive_month_ajexm'])*CRSP_comp_merge['past_thirtyfive_month_trfm']))-1)*100
CRSP_comp_merge['past_thirtysix_month_return'] = ((((CRSP_comp_merge['prccm']/CRSP_comp_merge['ajexm'])*CRSP_comp_merge['past_thirtysix_month_trfm'])/((CRSP_comp_merge['past_thirtysix_month_prccm']/CRSP_comp_merge['past_thirtysix_month_ajexm'])*CRSP_comp_merge['past_thirtysix_month_trfm']))-1)*100

In [32]:
# Merge - df & CRSP
df = df.merge(CRSP_comp_merge, on='GVKEY-year-month', how='left', suffixes=('', '_y'))

In [33]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [34]:
# Recommendations - Load
# Summary Statistics (Consensus Recommendations) from Wharton
recommendations = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_Recommendations_Summary_Statistics.csv', low_memory=False)

In [35]:
# Recommendations - Enrich - Time Features
recommendations['STATPERS'] = pd.to_datetime(recommendations['STATPERS'])
# Create year
recommendations['year'] = recommendations['STATPERS'].dt.year
# Create month
recommendations['month'] = recommendations['STATPERS'].dt.month
# Creat year-month
recommendations["year-month"] = recommendations['STATPERS'].apply(lambda x: x.strftime('%Y-%m'))

In [36]:
# Recommendations - Enrich - Analyst Change
recommendations["recup"] = recommendations["NUMUP"] / recommendations["NUMREC"] 
recommendations["recdown"] = recommendations["NUMDOWN"] / recommendations["NUMREC"]

In [37]:
# Recommendations - Enrich - Time-based Unique Identifiers
recommendations["TIC-year-month"] = recommendations["OFTIC"].map(str) + "-" + recommendations["year-month"]

In [38]:
# Merge - df & Recommendations
df = df.merge(recommendations, on='TIC-year-month', how='left', suffixes=('', '_y'))

In [39]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [40]:
# Beta Suite - Load
beta_suite = pd.read_csv('c:/users/sconner/Desktop/W205_Final/sample_data/Sample_Beta_Suite.csv', low_memory=False)

In [41]:
# Beta Suite - Enrich - Time Features
# Convert to data-time
beta_suite['DATE'] = pd.to_datetime(beta_suite['DATE'])
# Create year
beta_suite['year'] = beta_suite['DATE'].dt.year
# Create month
beta_suite['month'] = beta_suite['DATE'].dt.month
# Create year-month
beta_suite["year-month"] = beta_suite['DATE'].apply(lambda x: x.strftime('%Y-%m'))

In [42]:
# Beta Suite - Enrich - Time-based Unique Identifiers
beta_suite["PERMNO-year-month"] = beta_suite["PERMNO"].map(str) + "-" + beta_suite["year-month"]

In [43]:
# Merge - df & Beta Suite
df = df.merge(beta_suite, on='PERMNO-year-month', how='left', suffixes=('', '_y'))

In [44]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [45]:
# Sector - Enrich
# https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard

sector = pd.DataFrame()

sector['GSECTOR'] = [10.0, 
                       15.0, 
                       20.0, 
                       25.0, 
                       30.0, 
                       35.0, 
                       40.0, 
                       45.0, 
                       50.0, 
                       55.0, 
                       60.0]

sector['sector'] = ['Energy', 
                        'Materials', 
                        'Industrials', 
                        'Consumer Discretionary', 
                        'Consumer Staples', 
                        'Health Care', 
                        'Financials', 
                        'Information Technology', 
                        'Telecommunication Services', 
                        'Utilities', 
                        'Real Estate']

In [46]:
# Merge - df & Sector
df = df.merge(sector, on='GSECTOR', how='left', suffixes=('', '_y'))

In [47]:
# df - Clean - Remove duplicate features
drop_dups(df)

In [48]:
# df - Enrich - Time Features
df['january'] = np.where(df['month'] == 1, int(1), int(0))
df['february'] = np.where(df['month'] == 2, int(1), int(0))
df['march'] = np.where(df['month'] == 3, int(1), int(0))
df['april'] = np.where(df['month'] == 4, int(1), int(0))
df['may'] = np.where(df['month'] == 5, int(1), int(0))
df['june'] = np.where(df['month'] == 6, int(1), int(0))
df['july'] = np.where(df['month'] == 7, int(1), int(0))
df['august'] = np.where(df['month'] == 8, int(1), int(0))
df['september'] = np.where(df['month'] == 9, int(1), int(0))
df['october'] = np.where(df['month'] == 10, int(1), int(0))
df['november'] = np.where(df['month'] == 11, int(1), int(0))
df['december'] = np.where(df['month'] == 12, int(1), int(0))

In [49]:
# df - Subset - Relevant features with adequate data

df = df[[
        'GVKEY', 
        'sector',
        'year-month', 
        'forward_one_month_return', 
        'forward_two_month_return', 
        'forward_three_month_return', 
        'forward_four_month_return', 
        'forward_five_month_return', 
        'forward_six_month_return', 
        'forward_seven_month_return', 
        'forward_eight_month_return', 
        'forward_nine_month_return', 
        'forward_ten_month_return', 
        'forward_eleven_month_return', 
        'forward_twelve_month_return', 
        'forward_thirteen_month_return', 
        'forward_fourteen_month_return', 
        'forward_fifteen_month_return', 
        'forward_sixteen_month_return', 
        'forward_seventeen_month_return', 
        'forward_eighteen_month_return', 
        'forward_nineteen_month_return', 
        'forward_twenty_month_return', 
        'forward_twentyone_month_return', 
        'forward_twentytwo_month_return', 
        'forward_twentythree_month_return', 
        'forward_twentyfour_month_return', 
        'forward_twentyfive_month_return', 
        'forward_twentysix_month_return', 
        'forward_twentyseven_month_return', 
        'forward_twentyeight_month_return', 
        'forward_twentynine_month_return', 
        'forward_thirty_month_return', 
        'forward_thirtyone_month_return', 
        'forward_thirtytwo_month_return', 
        'forward_thirtythree_month_return', 
        'forward_thirtyfour_month_return', 
        'forward_thirtyfive_month_return', 
        'forward_thirtysix_month_return', 
        'past_one_month_return', 
        'past_two_month_return', 
        'past_three_month_return', 
        'past_four_month_return', 
        'past_five_month_return', 
        'past_six_month_return', 
        'past_seven_month_return', 
        'past_eight_month_return', 
        'past_nine_month_return', 
        'past_ten_month_return', 
        'past_eleven_month_return', 
        'past_twelve_month_return', 
        'past_thirteen_month_return', 
        'past_fourteen_month_return', 
        'past_fifteen_month_return', 
        'past_sixteen_month_return', 
        'past_seventeen_month_return', 
        'past_eighteen_month_return', 
        'past_nineteen_month_return', 
        'past_twenty_month_return', 
        'past_twentyone_month_return', 
        'past_twentytwo_month_return', 
        'past_twentythree_month_return', 
        'past_twentyfour_month_return', 
        'past_twentyfive_month_return', 
        'past_twentysix_month_return', 
        'past_twentyseven_month_return', 
        'past_twentyeight_month_return', 
        'past_twentynine_month_return', 
        'past_thirty_month_return', 
        'past_thirtyone_month_return', 
        'past_thirtytwo_month_return', 
        'past_thirtythree_month_return', 
        'past_thirtyfour_month_return', 
        'past_thirtyfive_month_return', 
        'past_thirtysix_month_return', 
        'accrual', 
        'adv_sale', 
        'aftret_eq', 
        'aftret_equity', 
        'aftret_invcapx', 
        'at_turn', 
        'bm', 
        'CAPEI', 
        'capital_ratio', 
        'cash_conversion', 
        'cash_debt', 
        'cash_lt', 
        'cash_ratio', 
        'cfm', 
        'curr_debt', 
        'curr_ratio', 
        'de_ratio', 
        'debt_assets', 
        'debt_at', 
        'debt_capital', 
        'debt_ebitda', 
        'debt_invcap', 
        'DIVYIELD', 
        'dltt_be', 
        'dpr', 
        'efftax', 
        'equity_invcap', 
        'evm', 
        'fcf_ocf', 
        'gpm', 
        'GProf', 
        'int_debt', 
        'int_totdebt', 
        'intcov', 
        'intcov_ratio', 
        'inv_turn', 
        'invt_act', 
        'lt_debt', 
        'lt_ppent', 
        'npm', 
        'ocf_lct', 
        'opmad', 
        'opmbd', 
        'pay_turn', 
        'pcf', 
        'pe_exi', 
        'pe_inc', 
        'pe_op_basic', 
        'pe_op_dil', 
        'PEG_1yrforward', 
        'PEG_ltgforward', 
        'PEG_trailing', 
        'pretret_earnat', 
        'pretret_noa', 
        'profit_lct', 
        'ps', 
        'ptb', 
        'ptpm', 
        'quick_ratio', 
        'rd_sale', 
        'rect_act', 
        'rect_turn', 
        'roa', 
        'roce', 
        'roe', 
        'sale_equity', 
        'sale_invcap', 
        'sale_nwc', 
        'short_debt', 
        'staff_sale', 
        'totdebt_invcap', 
        'dvpspm', 
        'dvpsxm', 
        'dvrate', 
        'spcsrc', 
        'alpha', 
        'b_hml', 
        'b_mkt', 
        'b_smb', 
        'b_umd', 
        'exret', 
        'ivol', 
        'n', 
        'R2', 
        'tvol', 
        'BUYPCT', 
        'HOLDPCT',
        'SELLPCT', 
        'MEANREC', 
        'MEDREC',
        'recup', 
        'recdown',
        'STDEV', 
        'january', 
        'february', 
        'march', 
        'april', 
        'may', 
        'june', 
        'july', 
        'august', 
        'september', 
        'october', 
        'november', 
        'december'
        ]]

In [50]:
# df - Clean - Dividends - DIVYIELD, dvpspm, dvpsxm, & dvrate
# Companies who do not report dividends are unlikely to have paid dividends, the majority of companies do not pay as well.
df['DIVYIELD'] = df['DIVYIELD'].replace('%','',regex=True).astype('float')/100
df['DIVYIELD'] = df['DIVYIELD'].fillna(0)
df['DIVYIELD'] = pd.to_numeric(df['DIVYIELD'])

df['dvpspm'] = df['dvpspm'].fillna(0)
df['dvpsxm'] = df['dvpsxm'].fillna(0)
df['dvrate'] = df['dvrate'].fillna(0)

In [51]:
# df - Clean - Ratings - SPCSRC
# Per S&P Quality Rankings information; 
# A+ - 98-100 percentile (Highest, 2%)
# A - 92-98 percentile (High, 6%) 
# A- - 86-92 percentile (Above Average, 6%) 
# B+ - 70-86 percentile (Average, 16%) 
# B - 51-70 percentile (Below Average, 19%)
# B- - 25-51 percentile (Lower, 26%)
# C - 1-25 percentile (Lowest, 24%)
# D - 0-1 percentile (In Reorganization, 1%) 

# Using the average of percentile ranges above

df['spcsrc'] = df['spcsrc'].map({
                                'A+': 99,
                                'A': 95,
                                'A-': 89,
                                'B+': 79.5,
                                'B': 60.5,
                                'B-': 38,
                                'C': 13,
                                'D': 0.5,
                                })

df['spcsrc'] = pd.to_numeric(df['spcsrc'])

In [52]:
# df - Clean - Beta Suite - exret, ivol, R2, tvol

df['exret'] = df['exret'].replace('%','',regex=True).astype('float')/100
df['exret'] = pd.to_numeric(df['exret'])

df['ivol'] = df['ivol'].replace('%','',regex=True).astype('float')/100
df['ivol'] = pd.to_numeric(df['ivol'])

df['R2'] = df['R2'].replace('%','',regex=True).astype('float')/100
df['R2'] = pd.to_numeric(df['R2'])

df['tvol'] = df['tvol'].replace('%','',regex=True).astype('float')/100
df['tvol'] = pd.to_numeric(df['tvol'])

In [53]:
# df - Clean - Replace 
for column in df:
    if (df[column].dtype == float):
        df[column] = fill_na(df[column])

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [54]:
# df - Clean - Clip Top/Bottom 3% and fill NaN's with median
for column in df:
    if (df[column].dtype == float):
        df[column] = clip_outliers(df[column])

In [55]:
return_all_rows(df.dtypes)

GVKEY                                 int64
sector                               object
year-month                           object
forward_one_month_return            float64
forward_two_month_return            float64
forward_three_month_return          float64
forward_four_month_return           float64
forward_five_month_return           float64
forward_six_month_return            float64
forward_seven_month_return          float64
forward_eight_month_return          float64
forward_nine_month_return           float64
forward_ten_month_return            float64
forward_eleven_month_return         float64
forward_twelve_month_return         float64
forward_thirteen_month_return       float64
forward_fourteen_month_return       float64
forward_fifteen_month_return        float64
forward_sixteen_month_return        float64
forward_seventeen_month_return      float64
forward_eighteen_month_return       float64
forward_nineteen_month_return       float64
forward_twenty_month_return     

In [56]:
df.describe()

,GVKEY,forward_one_month_return,forward_two_month_return,forward_three_month_return,forward_four_month_return,forward_five_month_return,forward_six_month_return,forward_seven_month_return,forward_eight_month_return,forward_nine_month_return,...,march,april,may,june,july,august,september,october,november,december
count,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,...,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000,47539.000000
mean,70699.487368,-0.449705,-1.184556,-1.941632,-3.535769,-4.848162,-5.817913,-6.240171,-5.988397,-4.708819,...,0.083321,0.083195,0.083279,0.083258,0.083069,0.083363,0.083658,0.083468,0.083342,0.083111
std,71125.349280,11.844652,14.520954,16.782231,16.933120,17.847433,16.768542,16.452543,14.377823,12.840528,...,0.276370,0.276179,0.276306,0.276275,0.275989,0.276433,0.276877,0.276592,0.276402,0.276052
min,1004.000000,-29.797553,-37.693105,-45.064231,-48.790612,-52.906516,-53.567884,-54.694885,-51.702970,-47.037707,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12635.000000,-5.865016,-7.424227,-7.954899,-8.168741,-7.525928,-4.649209,-5.235602,-5.071970,-4.166667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26069.000000,-0.322200,-0.579950,-0.952474,-2.309859,-3.758622,-4.649209,-5.235602,-5.071970,-4.166667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,147349.000000,4.800478,5.431461,4.969398,2.787829,-0.429927,-4.649209,-5.235602,-5.071970,-4.166667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,315318.000000,30.809446,34.452947,39.716081,36.600443,39.246820,34.289117,35.072091,29.644619,30.673249,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [57]:
# Market Median Variable - Enirch - All
forward_one_month_return_median = df.groupby(['year-month'])[['forward_one_month_return']].apply(np.nanmedian)
forward_two_month_return_median = df.groupby(['year-month'])[['forward_two_month_return']].apply(np.nanmedian)
forward_three_month_return_median = df.groupby(['year-month'])[['forward_three_month_return']].apply(np.nanmedian)
forward_four_month_return_median = df.groupby(['year-month'])[['forward_four_month_return']].apply(np.nanmedian)
forward_five_month_return_median = df.groupby(['year-month'])[['forward_five_month_return']].apply(np.nanmedian)
forward_six_month_return_median = df.groupby(['year-month'])[['forward_six_month_return']].apply(np.nanmedian)
forward_seven_month_return_median = df.groupby(['year-month'])[['forward_seven_month_return']].apply(np.nanmedian)
forward_eight_month_return_median = df.groupby(['year-month'])[['forward_eight_month_return']].apply(np.nanmedian)
forward_nine_month_return_median = df.groupby(['year-month'])[['forward_nine_month_return']].apply(np.nanmedian)
forward_ten_month_return_median = df.groupby(['year-month'])[['forward_ten_month_return']].apply(np.nanmedian)
forward_eleven_month_return_median = df.groupby(['year-month'])[['forward_eleven_month_return']].apply(np.nanmedian)
forward_twelve_month_return_median = df.groupby(['year-month'])[['forward_twelve_month_return']].apply(np.nanmedian)
forward_thirteen_month_return_median = df.groupby(['year-month'])[['forward_thirteen_month_return']].apply(np.nanmedian)
forward_fourteen_month_return_median = df.groupby(['year-month'])[['forward_fourteen_month_return']].apply(np.nanmedian)
forward_fifteen_month_return_median = df.groupby(['year-month'])[['forward_fifteen_month_return']].apply(np.nanmedian)
forward_sixteen_month_return_median = df.groupby(['year-month'])[['forward_sixteen_month_return']].apply(np.nanmedian)
forward_seventeen_month_return_median = df.groupby(['year-month'])[['forward_seventeen_month_return']].apply(np.nanmedian)
forward_eighteen_month_return_median = df.groupby(['year-month'])[['forward_eighteen_month_return']].apply(np.nanmedian)
forward_nineteen_month_return_median = df.groupby(['year-month'])[['forward_nineteen_month_return']].apply(np.nanmedian)
forward_twenty_month_return_median = df.groupby(['year-month'])[['forward_twenty_month_return']].apply(np.nanmedian)
forward_twentyone_month_return_median = df.groupby(['year-month'])[['forward_twentyone_month_return']].apply(np.nanmedian)
forward_twentytwo_month_return_median = df.groupby(['year-month'])[['forward_twentytwo_month_return']].apply(np.nanmedian)
forward_twentythree_month_return_median = df.groupby(['year-month'])[['forward_twentythree_month_return']].apply(np.nanmedian)
forward_twentyfour_month_return_median = df.groupby(['year-month'])[['forward_twentyfour_month_return']].apply(np.nanmedian)
forward_twentyfive_month_return_median = df.groupby(['year-month'])[['forward_twentyfive_month_return']].apply(np.nanmedian)
forward_twentysix_month_return_median = df.groupby(['year-month'])[['forward_twentysix_month_return']].apply(np.nanmedian)
forward_twentyseven_month_return_median = df.groupby(['year-month'])[['forward_twentyseven_month_return']].apply(np.nanmedian)
forward_twentyeight_month_return_median = df.groupby(['year-month'])[['forward_twentyeight_month_return']].apply(np.nanmedian)
forward_twentynine_month_return_median = df.groupby(['year-month'])[['forward_twentynine_month_return']].apply(np.nanmedian)
forward_thirty_month_return_median = df.groupby(['year-month'])[['forward_thirty_month_return']].apply(np.nanmedian)
forward_thirtyone_month_return_median = df.groupby(['year-month'])[['forward_thirtyone_month_return']].apply(np.nanmedian)
forward_thirtytwo_month_return_median = df.groupby(['year-month'])[['forward_thirtytwo_month_return']].apply(np.nanmedian)
forward_thirtythree_month_return_median = df.groupby(['year-month'])[['forward_thirtythree_month_return']].apply(np.nanmedian)
forward_thirtyfour_month_return_median = df.groupby(['year-month'])[['forward_thirtyfour_month_return']].apply(np.nanmedian)
forward_thirtyfive_month_return_median = df.groupby(['year-month'])[['forward_thirtyfive_month_return']].apply(np.nanmedian)
forward_thirtysix_month_return_median = df.groupby(['year-month'])[['forward_thirtysix_month_return']].apply(np.nanmedian)
past_one_month_return_median = df.groupby(['year-month'])[['past_one_month_return']].apply(np.nanmedian)
past_two_month_return_median = df.groupby(['year-month'])[['past_two_month_return']].apply(np.nanmedian)
past_three_month_return_median = df.groupby(['year-month'])[['past_three_month_return']].apply(np.nanmedian)
past_four_month_return_median = df.groupby(['year-month'])[['past_four_month_return']].apply(np.nanmedian)
past_five_month_return_median = df.groupby(['year-month'])[['past_five_month_return']].apply(np.nanmedian)
past_six_month_return_median = df.groupby(['year-month'])[['past_six_month_return']].apply(np.nanmedian)
past_seven_month_return_median = df.groupby(['year-month'])[['past_seven_month_return']].apply(np.nanmedian)
past_eight_month_return_median = df.groupby(['year-month'])[['past_eight_month_return']].apply(np.nanmedian)
past_nine_month_return_median = df.groupby(['year-month'])[['past_nine_month_return']].apply(np.nanmedian)
past_ten_month_return_median = df.groupby(['year-month'])[['past_ten_month_return']].apply(np.nanmedian)
past_eleven_month_return_median = df.groupby(['year-month'])[['past_eleven_month_return']].apply(np.nanmedian)
past_twelve_month_return_median = df.groupby(['year-month'])[['past_twelve_month_return']].apply(np.nanmedian)
past_thirteen_month_return_median = df.groupby(['year-month'])[['past_thirteen_month_return']].apply(np.nanmedian)
past_fourteen_month_return_median = df.groupby(['year-month'])[['past_fourteen_month_return']].apply(np.nanmedian)
past_fifteen_month_return_median = df.groupby(['year-month'])[['past_fifteen_month_return']].apply(np.nanmedian)
past_sixteen_month_return_median = df.groupby(['year-month'])[['past_sixteen_month_return']].apply(np.nanmedian)
past_seventeen_month_return_median = df.groupby(['year-month'])[['past_seventeen_month_return']].apply(np.nanmedian)
past_eighteen_month_return_median = df.groupby(['year-month'])[['past_eighteen_month_return']].apply(np.nanmedian)
past_nineteen_month_return_median = df.groupby(['year-month'])[['past_nineteen_month_return']].apply(np.nanmedian)
past_twenty_month_return_median = df.groupby(['year-month'])[['past_twenty_month_return']].apply(np.nanmedian)
past_twentyone_month_return_median = df.groupby(['year-month'])[['past_twentyone_month_return']].apply(np.nanmedian)
past_twentytwo_month_return_median = df.groupby(['year-month'])[['past_twentytwo_month_return']].apply(np.nanmedian)
past_twentythree_month_return_median = df.groupby(['year-month'])[['past_twentythree_month_return']].apply(np.nanmedian)
past_twentyfour_month_return_median = df.groupby(['year-month'])[['past_twentyfour_month_return']].apply(np.nanmedian)
past_twentyfive_month_return_median = df.groupby(['year-month'])[['past_twentyfive_month_return']].apply(np.nanmedian)
past_twentysix_month_return_median = df.groupby(['year-month'])[['past_twentysix_month_return']].apply(np.nanmedian)
past_twentyseven_month_return_median = df.groupby(['year-month'])[['past_twentyseven_month_return']].apply(np.nanmedian)
past_twentyeight_month_return_median = df.groupby(['year-month'])[['past_twentyeight_month_return']].apply(np.nanmedian)
past_twentynine_month_return_median = df.groupby(['year-month'])[['past_twentynine_month_return']].apply(np.nanmedian)
past_thirty_month_return_median = df.groupby(['year-month'])[['past_thirty_month_return']].apply(np.nanmedian)
past_thirtyone_month_return_median = df.groupby(['year-month'])[['past_thirtyone_month_return']].apply(np.nanmedian)
past_thirtytwo_month_return_median = df.groupby(['year-month'])[['past_thirtytwo_month_return']].apply(np.nanmedian)
past_thirtythree_month_return_median = df.groupby(['year-month'])[['past_thirtythree_month_return']].apply(np.nanmedian)
past_thirtyfour_month_return_median = df.groupby(['year-month'])[['past_thirtyfour_month_return']].apply(np.nanmedian)
past_thirtyfive_month_return_median = df.groupby(['year-month'])[['past_thirtyfive_month_return']].apply(np.nanmedian)
past_thirtysix_month_return_median = df.groupby(['year-month'])[['past_thirtysix_month_return']].apply(np.nanmedian)
accrual_median = df.groupby(['year-month'])[['accrual']].apply(np.nanmedian)
adv_sale_median = df.groupby(['year-month'])[['adv_sale']].apply(np.nanmedian)
aftret_eq_median = df.groupby(['year-month'])[['aftret_eq']].apply(np.nanmedian)
aftret_equity_median = df.groupby(['year-month'])[['aftret_equity']].apply(np.nanmedian)
aftret_invcapx_median = df.groupby(['year-month'])[['aftret_invcapx']].apply(np.nanmedian)
at_turn_median = df.groupby(['year-month'])[['at_turn']].apply(np.nanmedian)
bm_median = df.groupby(['year-month'])[['bm']].apply(np.nanmedian)
CAPEI_median = df.groupby(['year-month'])[['CAPEI']].apply(np.nanmedian)
capital_ratio_median = df.groupby(['year-month'])[['capital_ratio']].apply(np.nanmedian)
cash_conversion_median = df.groupby(['year-month'])[['cash_conversion']].apply(np.nanmedian)
cash_debt_median = df.groupby(['year-month'])[['cash_debt']].apply(np.nanmedian)
cash_lt_median = df.groupby(['year-month'])[['cash_lt']].apply(np.nanmedian)
cash_ratio_median = df.groupby(['year-month'])[['cash_ratio']].apply(np.nanmedian)
cfm_median = df.groupby(['year-month'])[['cfm']].apply(np.nanmedian)
curr_debt_median = df.groupby(['year-month'])[['curr_debt']].apply(np.nanmedian)
curr_ratio_median = df.groupby(['year-month'])[['curr_ratio']].apply(np.nanmedian)
de_ratio_median = df.groupby(['year-month'])[['de_ratio']].apply(np.nanmedian)
debt_assets_median = df.groupby(['year-month'])[['debt_assets']].apply(np.nanmedian)
debt_at_median = df.groupby(['year-month'])[['debt_at']].apply(np.nanmedian)
debt_capital_median = df.groupby(['year-month'])[['debt_capital']].apply(np.nanmedian)
debt_ebitda_median = df.groupby(['year-month'])[['debt_ebitda']].apply(np.nanmedian)
debt_invcap_median = df.groupby(['year-month'])[['debt_invcap']].apply(np.nanmedian)
DIVYIELD_median = df.groupby(['year-month'])[['DIVYIELD']].apply(np.nanmedian)
dltt_be_median = df.groupby(['year-month'])[['dltt_be']].apply(np.nanmedian)
dpr_median = df.groupby(['year-month'])[['dpr']].apply(np.nanmedian)
efftax_median = df.groupby(['year-month'])[['efftax']].apply(np.nanmedian)
equity_invcap_median = df.groupby(['year-month'])[['equity_invcap']].apply(np.nanmedian)
evm_median = df.groupby(['year-month'])[['evm']].apply(np.nanmedian)
fcf_ocf_median = df.groupby(['year-month'])[['fcf_ocf']].apply(np.nanmedian)
gpm_median = df.groupby(['year-month'])[['gpm']].apply(np.nanmedian)
GProf_median = df.groupby(['year-month'])[['GProf']].apply(np.nanmedian)
int_debt_median = df.groupby(['year-month'])[['int_debt']].apply(np.nanmedian)
int_totdebt_median = df.groupby(['year-month'])[['int_totdebt']].apply(np.nanmedian)
intcov_median = df.groupby(['year-month'])[['intcov']].apply(np.nanmedian)
intcov_ratio_median = df.groupby(['year-month'])[['intcov_ratio']].apply(np.nanmedian)
inv_turn_median = df.groupby(['year-month'])[['inv_turn']].apply(np.nanmedian)
invt_act_median = df.groupby(['year-month'])[['invt_act']].apply(np.nanmedian)
lt_debt_median = df.groupby(['year-month'])[['lt_debt']].apply(np.nanmedian)
lt_ppent_median = df.groupby(['year-month'])[['lt_ppent']].apply(np.nanmedian)
npm_median = df.groupby(['year-month'])[['npm']].apply(np.nanmedian)
ocf_lct_median = df.groupby(['year-month'])[['ocf_lct']].apply(np.nanmedian)
opmad_median = df.groupby(['year-month'])[['opmad']].apply(np.nanmedian)
opmbd_median = df.groupby(['year-month'])[['opmbd']].apply(np.nanmedian)
pay_turn_median = df.groupby(['year-month'])[['pay_turn']].apply(np.nanmedian)
pcf_median = df.groupby(['year-month'])[['pcf']].apply(np.nanmedian)
pe_exi_median = df.groupby(['year-month'])[['pe_exi']].apply(np.nanmedian)
pe_inc_median = df.groupby(['year-month'])[['pe_inc']].apply(np.nanmedian)
pe_op_basic_median = df.groupby(['year-month'])[['pe_op_basic']].apply(np.nanmedian)
pe_op_dil_median = df.groupby(['year-month'])[['pe_op_dil']].apply(np.nanmedian)
PEG_1yrforward_median = df.groupby(['year-month'])[['PEG_1yrforward']].apply(np.nanmedian)
PEG_ltgforward_median = df.groupby(['year-month'])[['PEG_ltgforward']].apply(np.nanmedian)
PEG_trailing_median = df.groupby(['year-month'])[['PEG_trailing']].apply(np.nanmedian)
pretret_earnat_median = df.groupby(['year-month'])[['pretret_earnat']].apply(np.nanmedian)
pretret_noa_median = df.groupby(['year-month'])[['pretret_noa']].apply(np.nanmedian)
profit_lct_median = df.groupby(['year-month'])[['profit_lct']].apply(np.nanmedian)
ps_median = df.groupby(['year-month'])[['ps']].apply(np.nanmedian)
ptb_median = df.groupby(['year-month'])[['ptb']].apply(np.nanmedian)
ptpm_median = df.groupby(['year-month'])[['ptpm']].apply(np.nanmedian)
quick_ratio_median = df.groupby(['year-month'])[['quick_ratio']].apply(np.nanmedian)
rd_sale_median = df.groupby(['year-month'])[['rd_sale']].apply(np.nanmedian)
rect_act_median = df.groupby(['year-month'])[['rect_act']].apply(np.nanmedian)
rect_turn_median = df.groupby(['year-month'])[['rect_turn']].apply(np.nanmedian)
roa_median = df.groupby(['year-month'])[['roa']].apply(np.nanmedian)
roce_median = df.groupby(['year-month'])[['roce']].apply(np.nanmedian)
roe_median = df.groupby(['year-month'])[['roe']].apply(np.nanmedian)
sale_equity_median = df.groupby(['year-month'])[['sale_equity']].apply(np.nanmedian)
sale_invcap_median = df.groupby(['year-month'])[['sale_invcap']].apply(np.nanmedian)
sale_nwc_median = df.groupby(['year-month'])[['sale_nwc']].apply(np.nanmedian)
short_debt_median = df.groupby(['year-month'])[['short_debt']].apply(np.nanmedian)
staff_sale_median = df.groupby(['year-month'])[['staff_sale']].apply(np.nanmedian)
totdebt_invcap_median = df.groupby(['year-month'])[['totdebt_invcap']].apply(np.nanmedian)
dvpspm_median = df.groupby(['year-month'])[['dvpspm']].apply(np.nanmedian)
dvpsxm_median = df.groupby(['year-month'])[['dvpsxm']].apply(np.nanmedian)
dvrate_median = df.groupby(['year-month'])[['dvrate']].apply(np.nanmedian)
spcsrc_median = df.groupby(['year-month'])[['spcsrc']].apply(np.nanmedian)
alpha_median = df.groupby(['year-month'])[['alpha']].apply(np.nanmedian)
b_hml_median = df.groupby(['year-month'])[['b_hml']].apply(np.nanmedian)
b_mkt_median = df.groupby(['year-month'])[['b_mkt']].apply(np.nanmedian)
b_smb_median = df.groupby(['year-month'])[['b_smb']].apply(np.nanmedian)
b_umd_median = df.groupby(['year-month'])[['b_umd']].apply(np.nanmedian)
exret_median = df.groupby(['year-month'])[['exret']].apply(np.nanmedian)
ivol_median = df.groupby(['year-month'])[['ivol']].apply(np.nanmedian)
n_median = df.groupby(['year-month'])[['n']].apply(np.nanmedian)
R2_median = df.groupby(['year-month'])[['R2']].apply(np.nanmedian)
tvol_median = df.groupby(['year-month'])[['tvol']].apply(np.nanmedian)
BUYPCT_median = df.groupby(['year-month'])[['BUYPCT']].apply(np.nanmedian)
HOLDPCT_median = df.groupby(['year-month'])[['HOLDPCT']].apply(np.nanmedian)
MEANREC_median = df.groupby(['year-month'])[['MEANREC']].apply(np.nanmedian)
MEDREC_median = df.groupby(['year-month'])[['MEDREC']].apply(np.nanmedian)
recup_median = df.groupby(['year-month'])[['recup']].apply(np.nanmedian)
recdown_median = df.groupby(['year-month'])[['recdown']].apply(np.nanmedian)
SELLPCT_median = df.groupby(['year-month'])[['SELLPCT']].apply(np.nanmedian)
STDEV_median = df.groupby(['year-month'])[['STDEV']].apply(np.nanmedian)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [58]:
# Market Median Datasets - Enrich - All
forward_one_month_return_median.name = 'forward_one_month_return_median'
forward_two_month_return_median.name = 'forward_two_month_return_median'
forward_three_month_return_median.name = 'forward_three_month_return_median'
forward_four_month_return_median.name = 'forward_four_month_return_median'
forward_five_month_return_median.name = 'forward_five_month_return_median'
forward_six_month_return_median.name = 'forward_six_month_return_median'
forward_seven_month_return_median.name = 'forward_seven_month_return_median'
forward_eight_month_return_median.name = 'forward_eight_month_return_median'
forward_nine_month_return_median.name = 'forward_nine_month_return_median'
forward_ten_month_return_median.name = 'forward_ten_month_return_median'
forward_eleven_month_return_median.name = 'forward_eleven_month_return_median'
forward_twelve_month_return_median.name = 'forward_twelve_month_return_median'
forward_thirteen_month_return_median.name = 'forward_thirteen_month_return_median'
forward_fourteen_month_return_median.name = 'forward_fourteen_month_return_median'
forward_fifteen_month_return_median.name = 'forward_fifteen_month_return_median'
forward_sixteen_month_return_median.name = 'forward_sixteen_month_return_median'
forward_seventeen_month_return_median.name = 'forward_seventeen_month_return_median'
forward_eighteen_month_return_median.name = 'forward_eighteen_month_return_median'
forward_nineteen_month_return_median.name = 'forward_nineteen_month_return_median'
forward_twenty_month_return_median.name = 'forward_twenty_month_return_median'
forward_twentyone_month_return_median.name = 'forward_twentyone_month_return_median'
forward_twentytwo_month_return_median.name = 'forward_twentytwo_month_return_median'
forward_twentythree_month_return_median.name = 'forward_twentythree_month_return_median'
forward_twentyfour_month_return_median.name = 'forward_twentyfour_month_return_median'
forward_twentyfive_month_return_median.name = 'forward_twentyfive_month_return_median'
forward_twentysix_month_return_median.name = 'forward_twentysix_month_return_median'
forward_twentyseven_month_return_median.name = 'forward_twentyseven_month_return_median'
forward_twentyeight_month_return_median.name = 'forward_twentyeight_month_return_median'
forward_twentynine_month_return_median.name = 'forward_twentynine_month_return_median'
forward_thirty_month_return_median.name = 'forward_thirty_month_return_median'
forward_thirtyone_month_return_median.name = 'forward_thirtyone_month_return_median'
forward_thirtytwo_month_return_median.name = 'forward_thirtytwo_month_return_median'
forward_thirtythree_month_return_median.name = 'forward_thirtythree_month_return_median'
forward_thirtyfour_month_return_median.name = 'forward_thirtyfour_month_return_median'
forward_thirtyfive_month_return_median.name = 'forward_thirtyfive_month_return_median'
forward_thirtysix_month_return_median.name = 'forward_thirtysix_month_return_median'
past_one_month_return_median.name = 'past_one_month_return_median'
past_two_month_return_median.name = 'past_two_month_return_median'
past_three_month_return_median.name = 'past_three_month_return_median'
past_four_month_return_median.name = 'past_four_month_return_median'
past_five_month_return_median.name = 'past_five_month_return_median'
past_six_month_return_median.name = 'past_six_month_return_median'
past_seven_month_return_median.name = 'past_seven_month_return_median'
past_eight_month_return_median.name = 'past_eight_month_return_median'
past_nine_month_return_median.name = 'past_nine_month_return_median'
past_ten_month_return_median.name = 'past_ten_month_return_median'
past_eleven_month_return_median.name = 'past_eleven_month_return_median'
past_twelve_month_return_median.name = 'past_twelve_month_return_median'
past_thirteen_month_return_median.name = 'past_thirteen_month_return_median'
past_fourteen_month_return_median.name = 'past_fourteen_month_return_median'
past_fifteen_month_return_median.name = 'past_fifteen_month_return_median'
past_sixteen_month_return_median.name = 'past_sixteen_month_return_median'
past_seventeen_month_return_median.name = 'past_seventeen_month_return_median'
past_eighteen_month_return_median.name = 'past_eighteen_month_return_median'
past_nineteen_month_return_median.name = 'past_nineteen_month_return_median'
past_twenty_month_return_median.name = 'past_twenty_month_return_median'
past_twentyone_month_return_median.name = 'past_twentyone_month_return_median'
past_twentytwo_month_return_median.name = 'past_twentytwo_month_return_median'
past_twentythree_month_return_median.name = 'past_twentythree_month_return_median'
past_twentyfour_month_return_median.name = 'past_twentyfour_month_return_median'
past_twentyfive_month_return_median.name = 'past_twentyfive_month_return_median'
past_twentysix_month_return_median.name = 'past_twentysix_month_return_median'
past_twentyseven_month_return_median.name = 'past_twentyseven_month_return_median'
past_twentyeight_month_return_median.name = 'past_twentyeight_month_return_median'
past_twentynine_month_return_median.name = 'past_twentynine_month_return_median'
past_thirty_month_return_median.name = 'past_thirty_month_return_median'
past_thirtyone_month_return_median.name = 'past_thirtyone_month_return_median'
past_thirtytwo_month_return_median.name = 'past_thirtytwo_month_return_median'
past_thirtythree_month_return_median.name = 'past_thirtythree_month_return_median'
past_thirtyfour_month_return_median.name = 'past_thirtyfour_month_return_median'
past_thirtyfive_month_return_median.name = 'past_thirtyfive_month_return_median'
past_thirtysix_month_return_median.name = 'past_thirtysix_month_return_median'
accrual_median.name = 'accrual_median'
adv_sale_median.name = 'adv_sale_median'
aftret_eq_median.name = 'aftret_eq_median'
aftret_equity_median.name = 'aftret_equity_median'
aftret_invcapx_median.name = 'aftret_invcapx_median'
at_turn_median.name = 'at_turn_median'
bm_median.name = 'bm_median'
CAPEI_median.name = 'CAPEI_median'
capital_ratio_median.name = 'capital_ratio_median'
cash_conversion_median.name = 'cash_conversion_median'
cash_debt_median.name = 'cash_debt_median'
cash_lt_median.name = 'cash_lt_median'
cash_ratio_median.name = 'cash_ratio_median'
cfm_median.name = 'cfm_median'
curr_debt_median.name = 'curr_debt_median'
curr_ratio_median.name = 'curr_ratio_median'
de_ratio_median.name = 'de_ratio_median'
debt_assets_median.name = 'debt_assets_median'
debt_at_median.name = 'debt_at_median'
debt_capital_median.name = 'debt_capital_median'
debt_ebitda_median.name = 'debt_ebitda_median'
debt_invcap_median.name = 'debt_invcap_median'
DIVYIELD_median.name = 'DIVYIELD_median'
dltt_be_median.name = 'dltt_be_median'
dpr_median.name = 'dpr_median'
efftax_median.name = 'efftax_median'
equity_invcap_median.name = 'equity_invcap_median'
evm_median.name = 'evm_median'
fcf_ocf_median.name = 'fcf_ocf_median'
gpm_median.name = 'gpm_median'
GProf_median.name = 'GProf_median'
int_debt_median.name = 'int_debt_median'
int_totdebt_median.name = 'int_totdebt_median'
intcov_median.name = 'intcov_median'
intcov_ratio_median.name = 'intcov_ratio_median'
inv_turn_median.name = 'inv_turn_median'
invt_act_median.name = 'invt_act_median'
lt_debt_median.name = 'lt_debt_median'
lt_ppent_median.name = 'lt_ppent_median'
npm_median.name = 'npm_median'
ocf_lct_median.name = 'ocf_lct_median'
opmad_median.name = 'opmad_median'
opmbd_median.name = 'opmbd_median'
pay_turn_median.name = 'pay_turn_median'
pcf_median.name = 'pcf_median'
pe_exi_median.name = 'pe_exi_median'
pe_inc_median.name = 'pe_inc_median'
pe_op_basic_median.name = 'pe_op_basic_median'
pe_op_dil_median.name = 'pe_op_dil_median'
PEG_1yrforward_median.name = 'PEG_1yrforward_median'
PEG_ltgforward_median.name = 'PEG_ltgforward_median'
PEG_trailing_median.name = 'PEG_trailing_median'
pretret_earnat_median.name = 'pretret_earnat_median'
pretret_noa_median.name = 'pretret_noa_median'
profit_lct_median.name = 'profit_lct_median'
ps_median.name = 'ps_median'
ptb_median.name = 'ptb_median'
ptpm_median.name = 'ptpm_median'
quick_ratio_median.name = 'quick_ratio_median'
rd_sale_median.name = 'rd_sale_median'
rect_act_median.name = 'rect_act_median'
rect_turn_median.name = 'rect_turn_median'
roa_median.name = 'roa_median'
roce_median.name = 'roce_median'
roe_median.name = 'roe_median'
sale_equity_median.name = 'sale_equity_median'
sale_invcap_median.name = 'sale_invcap_median'
sale_nwc_median.name = 'sale_nwc_median'
short_debt_median.name = 'short_debt_median'
staff_sale_median.name = 'staff_sale_median'
totdebt_invcap_median.name = 'totdebt_invcap_median'
dvpspm_median.name = 'dvpspm_median'
dvpsxm_median.name = 'dvpsxm_median'
dvrate_median.name = 'dvrate_median'
spcsrc_median.name = 'spcsrc_median'
alpha_median.name = 'alpha_median'
b_hml_median.name = 'b_hml_median'
b_mkt_median.name = 'b_mkt_median'
b_smb_median.name = 'b_smb_median'
b_umd_median.name = 'b_umd_median'
exret_median.name = 'exret_median'
ivol_median.name = 'ivol_median'
n_median.name = 'n_median'
R2_median.name = 'R2_median'
tvol_median.name = 'tvol_median'
BUYPCT_median.name = 'BUYPCT_median'
HOLDPCT_median.name = 'HOLDPCT_median'
MEANREC_median.name = 'MEANREC_median'
MEDREC_median.name = 'MEDREC_median'
recup_median.name = 'recup_median'
recdown_median.name = 'recdown_median'
SELLPCT_median.name = 'SELLPCT_median'
STDEV_median.name = 'STDEV_median'

In [59]:
# Merge - df & Market Median Datasets
df = df.join(forward_one_month_return_median, on=['year-month'])
df = df.join(forward_two_month_return_median, on=['year-month'])
df = df.join(forward_three_month_return_median, on=['year-month'])
df = df.join(forward_four_month_return_median, on=['year-month'])
df = df.join(forward_five_month_return_median, on=['year-month'])
df = df.join(forward_six_month_return_median, on=['year-month'])
df = df.join(forward_seven_month_return_median, on=['year-month'])
df = df.join(forward_eight_month_return_median, on=['year-month'])
df = df.join(forward_nine_month_return_median, on=['year-month'])
df = df.join(forward_ten_month_return_median, on=['year-month'])
df = df.join(forward_eleven_month_return_median, on=['year-month'])
df = df.join(forward_twelve_month_return_median, on=['year-month'])
df = df.join(forward_thirteen_month_return_median, on=['year-month'])
df = df.join(forward_fourteen_month_return_median, on=['year-month'])
df = df.join(forward_fifteen_month_return_median, on=['year-month'])
df = df.join(forward_sixteen_month_return_median, on=['year-month'])
df = df.join(forward_seventeen_month_return_median, on=['year-month'])
df = df.join(forward_eighteen_month_return_median, on=['year-month'])
df = df.join(forward_nineteen_month_return_median, on=['year-month'])
df = df.join(forward_twenty_month_return_median, on=['year-month'])
df = df.join(forward_twentyone_month_return_median, on=['year-month'])
df = df.join(forward_twentytwo_month_return_median, on=['year-month'])
df = df.join(forward_twentythree_month_return_median, on=['year-month'])
df = df.join(forward_twentyfour_month_return_median, on=['year-month'])
df = df.join(forward_twentyfive_month_return_median, on=['year-month'])
df = df.join(forward_twentysix_month_return_median, on=['year-month'])
df = df.join(forward_twentyseven_month_return_median, on=['year-month'])
df = df.join(forward_twentyeight_month_return_median, on=['year-month'])
df = df.join(forward_twentynine_month_return_median, on=['year-month'])
df = df.join(forward_thirty_month_return_median, on=['year-month'])
df = df.join(forward_thirtyone_month_return_median, on=['year-month'])
df = df.join(forward_thirtytwo_month_return_median, on=['year-month'])
df = df.join(forward_thirtythree_month_return_median, on=['year-month'])
df = df.join(forward_thirtyfour_month_return_median, on=['year-month'])
df = df.join(forward_thirtyfive_month_return_median, on=['year-month'])
df = df.join(forward_thirtysix_month_return_median, on=['year-month'])
df = df.join(past_one_month_return_median, on=['year-month'])
df = df.join(past_two_month_return_median, on=['year-month'])
df = df.join(past_three_month_return_median, on=['year-month'])
df = df.join(past_four_month_return_median, on=['year-month'])
df = df.join(past_five_month_return_median, on=['year-month'])
df = df.join(past_six_month_return_median, on=['year-month'])
df = df.join(past_seven_month_return_median, on=['year-month'])
df = df.join(past_eight_month_return_median, on=['year-month'])
df = df.join(past_nine_month_return_median, on=['year-month'])
df = df.join(past_ten_month_return_median, on=['year-month'])
df = df.join(past_eleven_month_return_median, on=['year-month'])
df = df.join(past_twelve_month_return_median, on=['year-month'])
df = df.join(past_thirteen_month_return_median, on=['year-month'])
df = df.join(past_fourteen_month_return_median, on=['year-month'])
df = df.join(past_fifteen_month_return_median, on=['year-month'])
df = df.join(past_sixteen_month_return_median, on=['year-month'])
df = df.join(past_seventeen_month_return_median, on=['year-month'])
df = df.join(past_eighteen_month_return_median, on=['year-month'])
df = df.join(past_nineteen_month_return_median, on=['year-month'])
df = df.join(past_twenty_month_return_median, on=['year-month'])
df = df.join(past_twentyone_month_return_median, on=['year-month'])
df = df.join(past_twentytwo_month_return_median, on=['year-month'])
df = df.join(past_twentythree_month_return_median, on=['year-month'])
df = df.join(past_twentyfour_month_return_median, on=['year-month'])
df = df.join(past_twentyfive_month_return_median, on=['year-month'])
df = df.join(past_twentysix_month_return_median, on=['year-month'])
df = df.join(past_twentyseven_month_return_median, on=['year-month'])
df = df.join(past_twentyeight_month_return_median, on=['year-month'])
df = df.join(past_twentynine_month_return_median, on=['year-month'])
df = df.join(past_thirty_month_return_median, on=['year-month'])
df = df.join(past_thirtyone_month_return_median, on=['year-month'])
df = df.join(past_thirtytwo_month_return_median, on=['year-month'])
df = df.join(past_thirtythree_month_return_median, on=['year-month'])
df = df.join(past_thirtyfour_month_return_median, on=['year-month'])
df = df.join(past_thirtyfive_month_return_median, on=['year-month'])
df = df.join(past_thirtysix_month_return_median, on=['year-month'])
df = df.join(accrual_median, on=['year-month'])
df = df.join(adv_sale_median, on=['year-month'])
df = df.join(aftret_eq_median, on=['year-month'])
df = df.join(aftret_equity_median, on=['year-month'])
df = df.join(aftret_invcapx_median, on=['year-month'])
df = df.join(at_turn_median, on=['year-month'])
df = df.join(bm_median, on=['year-month'])
df = df.join(CAPEI_median, on=['year-month'])
df = df.join(capital_ratio_median, on=['year-month'])
df = df.join(cash_conversion_median, on=['year-month'])
df = df.join(cash_debt_median, on=['year-month'])
df = df.join(cash_lt_median, on=['year-month'])
df = df.join(cash_ratio_median, on=['year-month'])
df = df.join(cfm_median, on=['year-month'])
df = df.join(curr_debt_median, on=['year-month'])
df = df.join(curr_ratio_median, on=['year-month'])
df = df.join(de_ratio_median, on=['year-month'])
df = df.join(debt_assets_median, on=['year-month'])
df = df.join(debt_at_median, on=['year-month'])
df = df.join(debt_capital_median, on=['year-month'])
df = df.join(debt_ebitda_median, on=['year-month'])
df = df.join(debt_invcap_median, on=['year-month'])
df = df.join(DIVYIELD_median, on=['year-month'])
df = df.join(dltt_be_median, on=['year-month'])
df = df.join(dpr_median, on=['year-month'])
df = df.join(efftax_median, on=['year-month'])
df = df.join(equity_invcap_median, on=['year-month'])
df = df.join(evm_median, on=['year-month'])
df = df.join(fcf_ocf_median, on=['year-month'])
df = df.join(gpm_median, on=['year-month'])
df = df.join(GProf_median, on=['year-month'])
df = df.join(int_debt_median, on=['year-month'])
df = df.join(int_totdebt_median, on=['year-month'])
df = df.join(intcov_median, on=['year-month'])
df = df.join(intcov_ratio_median, on=['year-month'])
df = df.join(inv_turn_median, on=['year-month'])
df = df.join(invt_act_median, on=['year-month'])
df = df.join(lt_debt_median, on=['year-month'])
df = df.join(lt_ppent_median, on=['year-month'])
df = df.join(npm_median, on=['year-month'])
df = df.join(ocf_lct_median, on=['year-month'])
df = df.join(opmad_median, on=['year-month'])
df = df.join(opmbd_median, on=['year-month'])
df = df.join(pay_turn_median, on=['year-month'])
df = df.join(pcf_median, on=['year-month'])
df = df.join(pe_exi_median, on=['year-month'])
df = df.join(pe_inc_median, on=['year-month'])
df = df.join(pe_op_basic_median, on=['year-month'])
df = df.join(pe_op_dil_median, on=['year-month'])
df = df.join(PEG_1yrforward_median, on=['year-month'])
df = df.join(PEG_ltgforward_median, on=['year-month'])
df = df.join(PEG_trailing_median, on=['year-month'])
df = df.join(pretret_earnat_median, on=['year-month'])
df = df.join(pretret_noa_median, on=['year-month'])
df = df.join(profit_lct_median, on=['year-month'])
df = df.join(ps_median, on=['year-month'])
df = df.join(ptb_median, on=['year-month'])
df = df.join(ptpm_median, on=['year-month'])
df = df.join(quick_ratio_median, on=['year-month'])
df = df.join(rd_sale_median, on=['year-month'])
df = df.join(rect_act_median, on=['year-month'])
df = df.join(rect_turn_median, on=['year-month'])
df = df.join(roa_median, on=['year-month'])
df = df.join(roce_median, on=['year-month'])
df = df.join(roe_median, on=['year-month'])
df = df.join(sale_equity_median, on=['year-month'])
df = df.join(sale_invcap_median, on=['year-month'])
df = df.join(sale_nwc_median, on=['year-month'])
df = df.join(short_debt_median, on=['year-month'])
df = df.join(staff_sale_median, on=['year-month'])
df = df.join(totdebt_invcap_median, on=['year-month'])
df = df.join(dvpspm_median, on=['year-month'])
df = df.join(dvpsxm_median, on=['year-month'])
df = df.join(dvrate_median, on=['year-month'])
df = df.join(spcsrc_median, on=['year-month'])
df = df.join(alpha_median, on=['year-month'])
df = df.join(b_hml_median, on=['year-month'])
df = df.join(b_mkt_median, on=['year-month'])
df = df.join(b_smb_median, on=['year-month'])
df = df.join(b_umd_median, on=['year-month'])
df = df.join(exret_median, on=['year-month'])
df = df.join(ivol_median, on=['year-month'])
df = df.join(n_median, on=['year-month'])
df = df.join(R2_median, on=['year-month'])
df = df.join(tvol_median, on=['year-month'])
df = df.join(BUYPCT_median, on=['year-month'])
df = df.join(HOLDPCT_median, on=['year-month'])
df = df.join(MEANREC_median, on=['year-month'])
df = df.join(MEDREC_median, on=['year-month'])
df = df.join(recup_median, on=['year-month'])
df = df.join(recdown_median, on=['year-month'])
df = df.join(SELLPCT_median, on=['year-month'])
df = df.join(STDEV_median, on=['year-month'])

In [60]:
# Sector Median Variable - Enirch - All
forward_one_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_one_month_return']].apply(np.nanmedian)
forward_two_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_two_month_return']].apply(np.nanmedian)
forward_three_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_three_month_return']].apply(np.nanmedian)
forward_four_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_four_month_return']].apply(np.nanmedian)
forward_five_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_five_month_return']].apply(np.nanmedian)
forward_six_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_six_month_return']].apply(np.nanmedian)
forward_seven_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_seven_month_return']].apply(np.nanmedian)
forward_eight_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_eight_month_return']].apply(np.nanmedian)
forward_nine_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_nine_month_return']].apply(np.nanmedian)
forward_ten_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_ten_month_return']].apply(np.nanmedian)
forward_eleven_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_eleven_month_return']].apply(np.nanmedian)
forward_twelve_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twelve_month_return']].apply(np.nanmedian)
forward_thirteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirteen_month_return']].apply(np.nanmedian)
forward_fourteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_fourteen_month_return']].apply(np.nanmedian)
forward_fifteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_fifteen_month_return']].apply(np.nanmedian)
forward_sixteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_sixteen_month_return']].apply(np.nanmedian)
forward_seventeen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_seventeen_month_return']].apply(np.nanmedian)
forward_eighteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_eighteen_month_return']].apply(np.nanmedian)
forward_nineteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_nineteen_month_return']].apply(np.nanmedian)
forward_twenty_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twenty_month_return']].apply(np.nanmedian)
forward_twentyone_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentyone_month_return']].apply(np.nanmedian)
forward_twentytwo_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentytwo_month_return']].apply(np.nanmedian)
forward_twentythree_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentythree_month_return']].apply(np.nanmedian)
forward_twentyfour_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentyfour_month_return']].apply(np.nanmedian)
forward_twentyfive_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentyfive_month_return']].apply(np.nanmedian)
forward_twentysix_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentysix_month_return']].apply(np.nanmedian)
forward_twentyseven_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentyseven_month_return']].apply(np.nanmedian)
forward_twentyeight_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentyeight_month_return']].apply(np.nanmedian)
forward_twentynine_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_twentynine_month_return']].apply(np.nanmedian)
forward_thirty_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirty_month_return']].apply(np.nanmedian)
forward_thirtyone_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirtyone_month_return']].apply(np.nanmedian)
forward_thirtytwo_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirtytwo_month_return']].apply(np.nanmedian)
forward_thirtythree_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirtythree_month_return']].apply(np.nanmedian)
forward_thirtyfour_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirtyfour_month_return']].apply(np.nanmedian)
forward_thirtyfive_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirtyfive_month_return']].apply(np.nanmedian)
forward_thirtysix_month_return_sector_median = df.groupby(['year-month', 'sector'])[['forward_thirtysix_month_return']].apply(np.nanmedian)
past_one_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_one_month_return']].apply(np.nanmedian)
past_two_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_two_month_return']].apply(np.nanmedian)
past_three_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_three_month_return']].apply(np.nanmedian)
past_four_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_four_month_return']].apply(np.nanmedian)
past_five_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_five_month_return']].apply(np.nanmedian)
past_six_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_six_month_return']].apply(np.nanmedian)
past_seven_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_seven_month_return']].apply(np.nanmedian)
past_eight_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_eight_month_return']].apply(np.nanmedian)
past_nine_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_nine_month_return']].apply(np.nanmedian)
past_ten_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_ten_month_return']].apply(np.nanmedian)
past_eleven_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_eleven_month_return']].apply(np.nanmedian)
past_twelve_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twelve_month_return']].apply(np.nanmedian)
past_thirteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirteen_month_return']].apply(np.nanmedian)
past_fourteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_fourteen_month_return']].apply(np.nanmedian)
past_fifteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_fifteen_month_return']].apply(np.nanmedian)
past_sixteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_sixteen_month_return']].apply(np.nanmedian)
past_seventeen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_seventeen_month_return']].apply(np.nanmedian)
past_eighteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_eighteen_month_return']].apply(np.nanmedian)
past_nineteen_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_nineteen_month_return']].apply(np.nanmedian)
past_twenty_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twenty_month_return']].apply(np.nanmedian)
past_twentyone_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentyone_month_return']].apply(np.nanmedian)
past_twentytwo_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentytwo_month_return']].apply(np.nanmedian)
past_twentythree_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentythree_month_return']].apply(np.nanmedian)
past_twentyfour_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentyfour_month_return']].apply(np.nanmedian)
past_twentyfive_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentyfive_month_return']].apply(np.nanmedian)
past_twentysix_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentysix_month_return']].apply(np.nanmedian)
past_twentyseven_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentyseven_month_return']].apply(np.nanmedian)
past_twentyeight_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentyeight_month_return']].apply(np.nanmedian)
past_twentynine_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_twentynine_month_return']].apply(np.nanmedian)
past_thirty_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirty_month_return']].apply(np.nanmedian)
past_thirtyone_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirtyone_month_return']].apply(np.nanmedian)
past_thirtytwo_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirtytwo_month_return']].apply(np.nanmedian)
past_thirtythree_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirtythree_month_return']].apply(np.nanmedian)
past_thirtyfour_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirtyfour_month_return']].apply(np.nanmedian)
past_thirtyfive_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirtyfive_month_return']].apply(np.nanmedian)
past_thirtysix_month_return_sector_median = df.groupby(['year-month', 'sector'])[['past_thirtysix_month_return']].apply(np.nanmedian)
accrual_sector_median = df.groupby(['year-month', 'sector'])[['accrual']].apply(np.nanmedian)
adv_sale_sector_median = df.groupby(['year-month', 'sector'])[['adv_sale']].apply(np.nanmedian)
aftret_eq_sector_median = df.groupby(['year-month', 'sector'])[['aftret_eq']].apply(np.nanmedian)
aftret_equity_sector_median = df.groupby(['year-month', 'sector'])[['aftret_equity']].apply(np.nanmedian)
aftret_invcapx_sector_median = df.groupby(['year-month', 'sector'])[['aftret_invcapx']].apply(np.nanmedian)
at_turn_sector_median = df.groupby(['year-month', 'sector'])[['at_turn']].apply(np.nanmedian)
bm_sector_median = df.groupby(['year-month', 'sector'])[['bm']].apply(np.nanmedian)
CAPEI_sector_median = df.groupby(['year-month', 'sector'])[['CAPEI']].apply(np.nanmedian)
capital_ratio_sector_median = df.groupby(['year-month', 'sector'])[['capital_ratio']].apply(np.nanmedian)
cash_conversion_sector_median = df.groupby(['year-month', 'sector'])[['cash_conversion']].apply(np.nanmedian)
cash_debt_sector_median = df.groupby(['year-month', 'sector'])[['cash_debt']].apply(np.nanmedian)
cash_lt_sector_median = df.groupby(['year-month', 'sector'])[['cash_lt']].apply(np.nanmedian)
cash_ratio_sector_median = df.groupby(['year-month', 'sector'])[['cash_ratio']].apply(np.nanmedian)
cfm_sector_median = df.groupby(['year-month', 'sector'])[['cfm']].apply(np.nanmedian)
curr_debt_sector_median = df.groupby(['year-month', 'sector'])[['curr_debt']].apply(np.nanmedian)
curr_ratio_sector_median = df.groupby(['year-month', 'sector'])[['curr_ratio']].apply(np.nanmedian)
de_ratio_sector_median = df.groupby(['year-month', 'sector'])[['de_ratio']].apply(np.nanmedian)
debt_assets_sector_median = df.groupby(['year-month', 'sector'])[['debt_assets']].apply(np.nanmedian)
debt_at_sector_median = df.groupby(['year-month', 'sector'])[['debt_at']].apply(np.nanmedian)
debt_capital_sector_median = df.groupby(['year-month', 'sector'])[['debt_capital']].apply(np.nanmedian)
debt_ebitda_sector_median = df.groupby(['year-month', 'sector'])[['debt_ebitda']].apply(np.nanmedian)
debt_invcap_sector_median = df.groupby(['year-month', 'sector'])[['debt_invcap']].apply(np.nanmedian)
DIVYIELD_sector_median = df.groupby(['year-month', 'sector'])[['DIVYIELD']].apply(np.nanmedian)
dltt_be_sector_median = df.groupby(['year-month', 'sector'])[['dltt_be']].apply(np.nanmedian)
dpr_sector_median = df.groupby(['year-month', 'sector'])[['dpr']].apply(np.nanmedian)
efftax_sector_median = df.groupby(['year-month', 'sector'])[['efftax']].apply(np.nanmedian)
equity_invcap_sector_median = df.groupby(['year-month', 'sector'])[['equity_invcap']].apply(np.nanmedian)
evm_sector_median = df.groupby(['year-month', 'sector'])[['evm']].apply(np.nanmedian)
fcf_ocf_sector_median = df.groupby(['year-month', 'sector'])[['fcf_ocf']].apply(np.nanmedian)
gpm_sector_median = df.groupby(['year-month', 'sector'])[['gpm']].apply(np.nanmedian)
GProf_sector_median = df.groupby(['year-month', 'sector'])[['GProf']].apply(np.nanmedian)
int_debt_sector_median = df.groupby(['year-month', 'sector'])[['int_debt']].apply(np.nanmedian)
int_totdebt_sector_median = df.groupby(['year-month', 'sector'])[['int_totdebt']].apply(np.nanmedian)
intcov_sector_median = df.groupby(['year-month', 'sector'])[['intcov']].apply(np.nanmedian)
intcov_ratio_sector_median = df.groupby(['year-month', 'sector'])[['intcov_ratio']].apply(np.nanmedian)
inv_turn_sector_median = df.groupby(['year-month', 'sector'])[['inv_turn']].apply(np.nanmedian)
invt_act_sector_median = df.groupby(['year-month', 'sector'])[['invt_act']].apply(np.nanmedian)
lt_debt_sector_median = df.groupby(['year-month', 'sector'])[['lt_debt']].apply(np.nanmedian)
lt_ppent_sector_median = df.groupby(['year-month', 'sector'])[['lt_ppent']].apply(np.nanmedian)
npm_sector_median = df.groupby(['year-month', 'sector'])[['npm']].apply(np.nanmedian)
ocf_lct_sector_median = df.groupby(['year-month', 'sector'])[['ocf_lct']].apply(np.nanmedian)
opmad_sector_median = df.groupby(['year-month', 'sector'])[['opmad']].apply(np.nanmedian)
opmbd_sector_median = df.groupby(['year-month', 'sector'])[['opmbd']].apply(np.nanmedian)
pay_turn_sector_median = df.groupby(['year-month', 'sector'])[['pay_turn']].apply(np.nanmedian)
pcf_sector_median = df.groupby(['year-month', 'sector'])[['pcf']].apply(np.nanmedian)
pe_exi_sector_median = df.groupby(['year-month', 'sector'])[['pe_exi']].apply(np.nanmedian)
pe_inc_sector_median = df.groupby(['year-month', 'sector'])[['pe_inc']].apply(np.nanmedian)
pe_op_basic_sector_median = df.groupby(['year-month', 'sector'])[['pe_op_basic']].apply(np.nanmedian)
pe_op_dil_sector_median = df.groupby(['year-month', 'sector'])[['pe_op_dil']].apply(np.nanmedian)
PEG_1yrforward_sector_median = df.groupby(['year-month', 'sector'])[['PEG_1yrforward']].apply(np.nanmedian)
PEG_ltgforward_sector_median = df.groupby(['year-month', 'sector'])[['PEG_ltgforward']].apply(np.nanmedian)
PEG_trailing_sector_median = df.groupby(['year-month', 'sector'])[['PEG_trailing']].apply(np.nanmedian)
pretret_earnat_sector_median = df.groupby(['year-month', 'sector'])[['pretret_earnat']].apply(np.nanmedian)
pretret_noa_sector_median = df.groupby(['year-month', 'sector'])[['pretret_noa']].apply(np.nanmedian)
profit_lct_sector_median = df.groupby(['year-month', 'sector'])[['profit_lct']].apply(np.nanmedian)
ps_sector_median = df.groupby(['year-month', 'sector'])[['ps']].apply(np.nanmedian)
ptb_sector_median = df.groupby(['year-month', 'sector'])[['ptb']].apply(np.nanmedian)
ptpm_sector_median = df.groupby(['year-month', 'sector'])[['ptpm']].apply(np.nanmedian)
quick_ratio_sector_median = df.groupby(['year-month', 'sector'])[['quick_ratio']].apply(np.nanmedian)
rd_sale_sector_median = df.groupby(['year-month', 'sector'])[['rd_sale']].apply(np.nanmedian)
rect_act_sector_median = df.groupby(['year-month', 'sector'])[['rect_act']].apply(np.nanmedian)
rect_turn_sector_median = df.groupby(['year-month', 'sector'])[['rect_turn']].apply(np.nanmedian)
roa_sector_median = df.groupby(['year-month', 'sector'])[['roa']].apply(np.nanmedian)
roce_sector_median = df.groupby(['year-month', 'sector'])[['roce']].apply(np.nanmedian)
roe_sector_median = df.groupby(['year-month', 'sector'])[['roe']].apply(np.nanmedian)
sale_equity_sector_median = df.groupby(['year-month', 'sector'])[['sale_equity']].apply(np.nanmedian)
sale_invcap_sector_median = df.groupby(['year-month', 'sector'])[['sale_invcap']].apply(np.nanmedian)
sale_nwc_sector_median = df.groupby(['year-month', 'sector'])[['sale_nwc']].apply(np.nanmedian)
short_debt_sector_median = df.groupby(['year-month', 'sector'])[['short_debt']].apply(np.nanmedian)
staff_sale_sector_median = df.groupby(['year-month', 'sector'])[['staff_sale']].apply(np.nanmedian)
totdebt_invcap_sector_median = df.groupby(['year-month', 'sector'])[['totdebt_invcap']].apply(np.nanmedian)
dvpspm_sector_median = df.groupby(['year-month', 'sector'])[['dvpspm']].apply(np.nanmedian)
dvpsxm_sector_median = df.groupby(['year-month', 'sector'])[['dvpsxm']].apply(np.nanmedian)
dvrate_sector_median = df.groupby(['year-month', 'sector'])[['dvrate']].apply(np.nanmedian)
spcsrc_sector_median = df.groupby(['year-month', 'sector'])[['spcsrc']].apply(np.nanmedian)
alpha_sector_median = df.groupby(['year-month', 'sector'])[['alpha']].apply(np.nanmedian)
b_hml_sector_median = df.groupby(['year-month', 'sector'])[['b_hml']].apply(np.nanmedian)
b_mkt_sector_median = df.groupby(['year-month', 'sector'])[['b_mkt']].apply(np.nanmedian)
b_smb_sector_median = df.groupby(['year-month', 'sector'])[['b_smb']].apply(np.nanmedian)
b_umd_sector_median = df.groupby(['year-month', 'sector'])[['b_umd']].apply(np.nanmedian)
exret_sector_median = df.groupby(['year-month', 'sector'])[['exret']].apply(np.nanmedian)
ivol_sector_median = df.groupby(['year-month', 'sector'])[['ivol']].apply(np.nanmedian)
n_sector_median = df.groupby(['year-month', 'sector'])[['n']].apply(np.nanmedian)
R2_sector_median = df.groupby(['year-month', 'sector'])[['R2']].apply(np.nanmedian)
tvol_sector_median = df.groupby(['year-month', 'sector'])[['tvol']].apply(np.nanmedian)
BUYPCT_sector_median = df.groupby(['year-month', 'sector'])[['BUYPCT']].apply(np.nanmedian)
HOLDPCT_sector_median = df.groupby(['year-month', 'sector'])[['HOLDPCT']].apply(np.nanmedian)
MEANREC_sector_median = df.groupby(['year-month', 'sector'])[['MEANREC']].apply(np.nanmedian)
MEDREC_sector_median = df.groupby(['year-month', 'sector'])[['MEDREC']].apply(np.nanmedian)
recup_sector_median = df.groupby(['year-month', 'sector'])[['recup']].apply(np.nanmedian)
recdown_sector_median = df.groupby(['year-month', 'sector'])[['recdown']].apply(np.nanmedian)
SELLPCT_sector_median = df.groupby(['year-month', 'sector'])[['SELLPCT']].apply(np.nanmedian)
STDEV_sector_median = df.groupby(['year-month', 'sector'])[['STDEV']].apply(np.nanmedian)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [61]:
# Sector Median Datasets - Enrich - All
forward_one_month_return_sector_median.name = 'forward_one_month_return_sector_median'
forward_two_month_return_sector_median.name = 'forward_two_month_return_sector_median'
forward_three_month_return_sector_median.name = 'forward_three_month_return_sector_median'
forward_four_month_return_sector_median.name = 'forward_four_month_return_sector_median'
forward_five_month_return_sector_median.name = 'forward_five_month_return_sector_median'
forward_six_month_return_sector_median.name = 'forward_six_month_return_sector_median'
forward_seven_month_return_sector_median.name = 'forward_seven_month_return_sector_median'
forward_eight_month_return_sector_median.name = 'forward_eight_month_return_sector_median'
forward_nine_month_return_sector_median.name = 'forward_nine_month_return_sector_median'
forward_ten_month_return_sector_median.name = 'forward_ten_month_return_sector_median'
forward_eleven_month_return_sector_median.name = 'forward_eleven_month_return_sector_median'
forward_twelve_month_return_sector_median.name = 'forward_twelve_month_return_sector_median'
forward_thirteen_month_return_sector_median.name = 'forward_thirteen_month_return_sector_median'
forward_fourteen_month_return_sector_median.name = 'forward_fourteen_month_return_sector_median'
forward_fifteen_month_return_sector_median.name = 'forward_fifteen_month_return_sector_median'
forward_sixteen_month_return_sector_median.name = 'forward_sixteen_month_return_sector_median'
forward_seventeen_month_return_sector_median.name = 'forward_seventeen_month_return_sector_median'
forward_eighteen_month_return_sector_median.name = 'forward_eighteen_month_return_sector_median'
forward_nineteen_month_return_sector_median.name = 'forward_nineteen_month_return_sector_median'
forward_twenty_month_return_sector_median.name = 'forward_twenty_month_return_sector_median'
forward_twentyone_month_return_sector_median.name = 'forward_twentyone_month_return_sector_median'
forward_twentytwo_month_return_sector_median.name = 'forward_twentytwo_month_return_sector_median'
forward_twentythree_month_return_sector_median.name = 'forward_twentythree_month_return_sector_median'
forward_twentyfour_month_return_sector_median.name = 'forward_twentyfour_month_return_sector_median'
forward_twentyfive_month_return_sector_median.name = 'forward_twentyfive_month_return_sector_median'
forward_twentysix_month_return_sector_median.name = 'forward_twentysix_month_return_sector_median'
forward_twentyseven_month_return_sector_median.name = 'forward_twentyseven_month_return_sector_median'
forward_twentyeight_month_return_sector_median.name = 'forward_twentyeight_month_return_sector_median'
forward_twentynine_month_return_sector_median.name = 'forward_twentynine_month_return_sector_median'
forward_thirty_month_return_sector_median.name = 'forward_thirty_month_return_sector_median'
forward_thirtyone_month_return_sector_median.name = 'forward_thirtyone_month_return_sector_median'
forward_thirtytwo_month_return_sector_median.name = 'forward_thirtytwo_month_return_sector_median'
forward_thirtythree_month_return_sector_median.name = 'forward_thirtythree_month_return_sector_median'
forward_thirtyfour_month_return_sector_median.name = 'forward_thirtyfour_month_return_sector_median'
forward_thirtyfive_month_return_sector_median.name = 'forward_thirtyfive_month_return_sector_median'
forward_thirtysix_month_return_sector_median.name = 'forward_thirtysix_month_return_sector_median'
past_one_month_return_sector_median.name = 'past_one_month_return_sector_median'
past_two_month_return_sector_median.name = 'past_two_month_return_sector_median'
past_three_month_return_sector_median.name = 'past_three_month_return_sector_median'
past_four_month_return_sector_median.name = 'past_four_month_return_sector_median'
past_five_month_return_sector_median.name = 'past_five_month_return_sector_median'
past_six_month_return_sector_median.name = 'past_six_month_return_sector_median'
past_seven_month_return_sector_median.name = 'past_seven_month_return_sector_median'
past_eight_month_return_sector_median.name = 'past_eight_month_return_sector_median'
past_nine_month_return_sector_median.name = 'past_nine_month_return_sector_median'
past_ten_month_return_sector_median.name = 'past_ten_month_return_sector_median'
past_eleven_month_return_sector_median.name = 'past_eleven_month_return_sector_median'
past_twelve_month_return_sector_median.name = 'past_twelve_month_return_sector_median'
past_thirteen_month_return_sector_median.name = 'past_thirteen_month_return_sector_median'
past_fourteen_month_return_sector_median.name = 'past_fourteen_month_return_sector_median'
past_fifteen_month_return_sector_median.name = 'past_fifteen_month_return_sector_median'
past_sixteen_month_return_sector_median.name = 'past_sixteen_month_return_sector_median'
past_seventeen_month_return_sector_median.name = 'past_seventeen_month_return_sector_median'
past_eighteen_month_return_sector_median.name = 'past_eighteen_month_return_sector_median'
past_nineteen_month_return_sector_median.name = 'past_nineteen_month_return_sector_median'
past_twenty_month_return_sector_median.name = 'past_twenty_month_return_sector_median'
past_twentyone_month_return_sector_median.name = 'past_twentyone_month_return_sector_median'
past_twentytwo_month_return_sector_median.name = 'past_twentytwo_month_return_sector_median'
past_twentythree_month_return_sector_median.name = 'past_twentythree_month_return_sector_median'
past_twentyfour_month_return_sector_median.name = 'past_twentyfour_month_return_sector_median'
past_twentyfive_month_return_sector_median.name = 'past_twentyfive_month_return_sector_median'
past_twentysix_month_return_sector_median.name = 'past_twentysix_month_return_sector_median'
past_twentyseven_month_return_sector_median.name = 'past_twentyseven_month_return_sector_median'
past_twentyeight_month_return_sector_median.name = 'past_twentyeight_month_return_sector_median'
past_twentynine_month_return_sector_median.name = 'past_twentynine_month_return_sector_median'
past_thirty_month_return_sector_median.name = 'past_thirty_month_return_sector_median'
past_thirtyone_month_return_sector_median.name = 'past_thirtyone_month_return_sector_median'
past_thirtytwo_month_return_sector_median.name = 'past_thirtytwo_month_return_sector_median'
past_thirtythree_month_return_sector_median.name = 'past_thirtythree_month_return_sector_median'
past_thirtyfour_month_return_sector_median.name = 'past_thirtyfour_month_return_sector_median'
past_thirtyfive_month_return_sector_median.name = 'past_thirtyfive_month_return_sector_median'
past_thirtysix_month_return_sector_median.name = 'past_thirtysix_month_return_sector_median'
accrual_sector_median.name = 'accrual_sector_median'
adv_sale_sector_median.name = 'adv_sale_sector_median'
aftret_eq_sector_median.name = 'aftret_eq_sector_median'
aftret_equity_sector_median.name = 'aftret_equity_sector_median'
aftret_invcapx_sector_median.name = 'aftret_invcapx_sector_median'
at_turn_sector_median.name = 'at_turn_sector_median'
bm_sector_median.name = 'bm_sector_median'
CAPEI_sector_median.name = 'CAPEI_sector_median'
capital_ratio_sector_median.name = 'capital_ratio_sector_median'
cash_conversion_sector_median.name = 'cash_conversion_sector_median'
cash_debt_sector_median.name = 'cash_debt_sector_median'
cash_lt_sector_median.name = 'cash_lt_sector_median'
cash_ratio_sector_median.name = 'cash_ratio_sector_median'
cfm_sector_median.name = 'cfm_sector_median'
curr_debt_sector_median.name = 'curr_debt_sector_median'
curr_ratio_sector_median.name = 'curr_ratio_sector_median'
de_ratio_sector_median.name = 'de_ratio_sector_median'
debt_assets_sector_median.name = 'debt_assets_sector_median'
debt_at_sector_median.name = 'debt_at_sector_median'
debt_capital_sector_median.name = 'debt_capital_sector_median'
debt_ebitda_sector_median.name = 'debt_ebitda_sector_median'
debt_invcap_sector_median.name = 'debt_invcap_sector_median'
DIVYIELD_sector_median.name = 'DIVYIELD_sector_median'
dltt_be_sector_median.name = 'dltt_be_sector_median'
dpr_sector_median.name = 'dpr_sector_median'
efftax_sector_median.name = 'efftax_sector_median'
equity_invcap_sector_median.name = 'equity_invcap_sector_median'
evm_sector_median.name = 'evm_sector_median'
fcf_ocf_sector_median.name = 'fcf_ocf_sector_median'
gpm_sector_median.name = 'gpm_sector_median'
GProf_sector_median.name = 'GProf_sector_median'
int_debt_sector_median.name = 'int_debt_sector_median'
int_totdebt_sector_median.name = 'int_totdebt_sector_median'
intcov_sector_median.name = 'intcov_sector_median'
intcov_ratio_sector_median.name = 'intcov_ratio_sector_median'
inv_turn_sector_median.name = 'inv_turn_sector_median'
invt_act_sector_median.name = 'invt_act_sector_median'
lt_debt_sector_median.name = 'lt_debt_sector_median'
lt_ppent_sector_median.name = 'lt_ppent_sector_median'
npm_sector_median.name = 'npm_sector_median'
ocf_lct_sector_median.name = 'ocf_lct_sector_median'
opmad_sector_median.name = 'opmad_sector_median'
opmbd_sector_median.name = 'opmbd_sector_median'
pay_turn_sector_median.name = 'pay_turn_sector_median'
pcf_sector_median.name = 'pcf_sector_median'
pe_exi_sector_median.name = 'pe_exi_sector_median'
pe_inc_sector_median.name = 'pe_inc_sector_median'
pe_op_basic_sector_median.name = 'pe_op_basic_sector_median'
pe_op_dil_sector_median.name = 'pe_op_dil_sector_median'
PEG_1yrforward_sector_median.name = 'PEG_1yrforward_sector_median'
PEG_ltgforward_sector_median.name = 'PEG_ltgforward_sector_median'
PEG_trailing_sector_median.name = 'PEG_trailing_sector_median'
pretret_earnat_sector_median.name = 'pretret_earnat_sector_median'
pretret_noa_sector_median.name = 'pretret_noa_sector_median'
profit_lct_sector_median.name = 'profit_lct_sector_median'
ps_sector_median.name = 'ps_sector_median'
ptb_sector_median.name = 'ptb_sector_median'
ptpm_sector_median.name = 'ptpm_sector_median'
quick_ratio_sector_median.name = 'quick_ratio_sector_median'
rd_sale_sector_median.name = 'rd_sale_sector_median'
rect_act_sector_median.name = 'rect_act_sector_median'
rect_turn_sector_median.name = 'rect_turn_sector_median'
roa_sector_median.name = 'roa_sector_median'
roce_sector_median.name = 'roce_sector_median'
roe_sector_median.name = 'roe_sector_median'
sale_equity_sector_median.name = 'sale_equity_sector_median'
sale_invcap_sector_median.name = 'sale_invcap_sector_median'
sale_nwc_sector_median.name = 'sale_nwc_sector_median'
short_debt_sector_median.name = 'short_debt_sector_median'
staff_sale_sector_median.name = 'staff_sale_sector_median'
totdebt_invcap_sector_median.name = 'totdebt_invcap_sector_median'
dvpspm_sector_median.name = 'dvpspm_sector_median'
dvpsxm_sector_median.name = 'dvpsxm_sector_median'
dvrate_sector_median.name = 'dvrate_sector_median'
spcsrc_sector_median.name = 'spcsrc_sector_median'
alpha_sector_median.name = 'alpha_sector_median'
b_hml_sector_median.name = 'b_hml_sector_median'
b_mkt_sector_median.name = 'b_mkt_sector_median'
b_smb_sector_median.name = 'b_smb_sector_median'
b_umd_sector_median.name = 'b_umd_sector_median'
exret_sector_median.name = 'exret_sector_median'
ivol_sector_median.name = 'ivol_sector_median'
n_sector_median.name = 'n_sector_median'
R2_sector_median.name = 'R2_sector_median'
tvol_sector_median.name = 'tvol_sector_median'
BUYPCT_sector_median.name = 'BUYPCT_sector_median'
HOLDPCT_sector_median.name = 'HOLDPCT_sector_median'
MEANREC_sector_median.name = 'MEANREC_sector_median'
MEDREC_sector_median.name = 'MEDREC_sector_median'
recup_sector_median.name = 'recup_sector_median'
recdown_sector_median.name = 'recdown_sector_median'
SELLPCT_sector_median.name = 'SELLPCT_sector_median'
STDEV_sector_median.name = 'STDEV_sector_median'

In [62]:
# Merge - df & Sector Median Datasets
df = df.join(forward_one_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_two_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_three_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_four_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_five_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_six_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_seven_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_eight_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_nine_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_ten_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_eleven_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twelve_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_fourteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_fifteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_sixteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_seventeen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_eighteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_nineteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twenty_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentyone_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentytwo_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentythree_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentyfour_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentyfive_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentysix_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentyseven_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentyeight_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_twentynine_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirty_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirtyone_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirtytwo_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirtythree_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirtyfour_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirtyfive_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(forward_thirtysix_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_one_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_two_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_three_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_four_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_five_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_six_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_seven_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_eight_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_nine_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_ten_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_eleven_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twelve_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_fourteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_fifteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_sixteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_seventeen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_eighteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_nineteen_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twenty_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentyone_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentytwo_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentythree_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentyfour_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentyfive_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentysix_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentyseven_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentyeight_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_twentynine_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirty_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirtyone_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirtytwo_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirtythree_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirtyfour_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirtyfive_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(past_thirtysix_month_return_sector_median, on=['year-month', 'sector'])
df = df.join(accrual_sector_median, on=['year-month', 'sector'])
df = df.join(adv_sale_sector_median, on=['year-month', 'sector'])
df = df.join(aftret_eq_sector_median, on=['year-month', 'sector'])
df = df.join(aftret_equity_sector_median, on=['year-month', 'sector'])
df = df.join(aftret_invcapx_sector_median, on=['year-month', 'sector'])
df = df.join(at_turn_sector_median, on=['year-month', 'sector'])
df = df.join(bm_sector_median, on=['year-month', 'sector'])
df = df.join(CAPEI_sector_median, on=['year-month', 'sector'])
df = df.join(capital_ratio_sector_median, on=['year-month', 'sector'])
df = df.join(cash_conversion_sector_median, on=['year-month', 'sector'])
df = df.join(cash_debt_sector_median, on=['year-month', 'sector'])
df = df.join(cash_lt_sector_median, on=['year-month', 'sector'])
df = df.join(cash_ratio_sector_median, on=['year-month', 'sector'])
df = df.join(cfm_sector_median, on=['year-month', 'sector'])
df = df.join(curr_debt_sector_median, on=['year-month', 'sector'])
df = df.join(curr_ratio_sector_median, on=['year-month', 'sector'])
df = df.join(de_ratio_sector_median, on=['year-month', 'sector'])
df = df.join(debt_assets_sector_median, on=['year-month', 'sector'])
df = df.join(debt_at_sector_median, on=['year-month', 'sector'])
df = df.join(debt_capital_sector_median, on=['year-month', 'sector'])
df = df.join(debt_ebitda_sector_median, on=['year-month', 'sector'])
df = df.join(debt_invcap_sector_median, on=['year-month', 'sector'])
df = df.join(DIVYIELD_sector_median, on=['year-month', 'sector'])
df = df.join(dltt_be_sector_median, on=['year-month', 'sector'])
df = df.join(dpr_sector_median, on=['year-month', 'sector'])
df = df.join(efftax_sector_median, on=['year-month', 'sector'])
df = df.join(equity_invcap_sector_median, on=['year-month', 'sector'])
df = df.join(evm_sector_median, on=['year-month', 'sector'])
df = df.join(fcf_ocf_sector_median, on=['year-month', 'sector'])
df = df.join(gpm_sector_median, on=['year-month', 'sector'])
df = df.join(GProf_sector_median, on=['year-month', 'sector'])
df = df.join(int_debt_sector_median, on=['year-month', 'sector'])
df = df.join(int_totdebt_sector_median, on=['year-month', 'sector'])
df = df.join(intcov_sector_median, on=['year-month', 'sector'])
df = df.join(intcov_ratio_sector_median, on=['year-month', 'sector'])
df = df.join(inv_turn_sector_median, on=['year-month', 'sector'])
df = df.join(invt_act_sector_median, on=['year-month', 'sector'])
df = df.join(lt_debt_sector_median, on=['year-month', 'sector'])
df = df.join(lt_ppent_sector_median, on=['year-month', 'sector'])
df = df.join(npm_sector_median, on=['year-month', 'sector'])
df = df.join(ocf_lct_sector_median, on=['year-month', 'sector'])
df = df.join(opmad_sector_median, on=['year-month', 'sector'])
df = df.join(opmbd_sector_median, on=['year-month', 'sector'])
df = df.join(pay_turn_sector_median, on=['year-month', 'sector'])
df = df.join(pcf_sector_median, on=['year-month', 'sector'])
df = df.join(pe_exi_sector_median, on=['year-month', 'sector'])
df = df.join(pe_inc_sector_median, on=['year-month', 'sector'])
df = df.join(pe_op_basic_sector_median, on=['year-month', 'sector'])
df = df.join(pe_op_dil_sector_median, on=['year-month', 'sector'])
df = df.join(PEG_1yrforward_sector_median, on=['year-month', 'sector'])
df = df.join(PEG_ltgforward_sector_median, on=['year-month', 'sector'])
df = df.join(PEG_trailing_sector_median, on=['year-month', 'sector'])
df = df.join(pretret_earnat_sector_median, on=['year-month', 'sector'])
df = df.join(pretret_noa_sector_median, on=['year-month', 'sector'])
df = df.join(profit_lct_sector_median, on=['year-month', 'sector'])
df = df.join(ps_sector_median, on=['year-month', 'sector'])
df = df.join(ptb_sector_median, on=['year-month', 'sector'])
df = df.join(ptpm_sector_median, on=['year-month', 'sector'])
df = df.join(quick_ratio_sector_median, on=['year-month', 'sector'])
df = df.join(rd_sale_sector_median, on=['year-month', 'sector'])
df = df.join(rect_act_sector_median, on=['year-month', 'sector'])
df = df.join(rect_turn_sector_median, on=['year-month', 'sector'])
df = df.join(roa_sector_median, on=['year-month', 'sector'])
df = df.join(roce_sector_median, on=['year-month', 'sector'])
df = df.join(roe_sector_median, on=['year-month', 'sector'])
df = df.join(sale_equity_sector_median, on=['year-month', 'sector'])
df = df.join(sale_invcap_sector_median, on=['year-month', 'sector'])
df = df.join(sale_nwc_sector_median, on=['year-month', 'sector'])
df = df.join(short_debt_sector_median, on=['year-month', 'sector'])
df = df.join(staff_sale_sector_median, on=['year-month', 'sector'])
df = df.join(totdebt_invcap_sector_median, on=['year-month', 'sector'])
df = df.join(dvpspm_sector_median, on=['year-month', 'sector'])
df = df.join(dvpsxm_sector_median, on=['year-month', 'sector'])
df = df.join(dvrate_sector_median, on=['year-month', 'sector'])
df = df.join(spcsrc_sector_median, on=['year-month', 'sector'])
df = df.join(alpha_sector_median, on=['year-month', 'sector'])
df = df.join(b_hml_sector_median, on=['year-month', 'sector'])
df = df.join(b_mkt_sector_median, on=['year-month', 'sector'])
df = df.join(b_smb_sector_median, on=['year-month', 'sector'])
df = df.join(b_umd_sector_median, on=['year-month', 'sector'])
df = df.join(exret_sector_median, on=['year-month', 'sector'])
df = df.join(ivol_sector_median, on=['year-month', 'sector'])
df = df.join(n_sector_median, on=['year-month', 'sector'])
df = df.join(R2_sector_median, on=['year-month', 'sector'])
df = df.join(tvol_sector_median, on=['year-month', 'sector'])
df = df.join(BUYPCT_sector_median, on=['year-month', 'sector'])
df = df.join(HOLDPCT_sector_median, on=['year-month', 'sector'])
df = df.join(MEANREC_sector_median, on=['year-month', 'sector'])
df = df.join(MEDREC_sector_median, on=['year-month', 'sector'])
df = df.join(recup_sector_median, on=['year-month', 'sector'])
df = df.join(recdown_sector_median, on=['year-month', 'sector'])
df = df.join(SELLPCT_sector_median, on=['year-month', 'sector'])
df = df.join(STDEV_sector_median, on=['year-month', 'sector'])

In [63]:
# Market MAD Variable - Enirch - All
# Z-score denominator for securities by month, use mean absolute deviation in place of median absolute deviation is MAD = 0
# Adjustments to replicate standard deviation value included in fuction (meanad and mad)
if df.groupby(['year-month'])[['forward_one_month_return']].apply(mad).any() == 0:
    forward_one_month_return_mad = df.groupby(['year-month'])[['forward_one_month_return']].apply(meanad)
else:
    forward_one_month_return_mad = df.groupby(['year-month'])[['forward_one_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_two_month_return']].apply(mad).any() == 0:
    forward_two_month_return_mad = df.groupby(['year-month'])[['forward_two_month_return']].apply(meanad)
else:
    forward_two_month_return_mad = df.groupby(['year-month'])[['forward_two_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_three_month_return']].apply(mad).any() == 0:
    forward_three_month_return_mad = df.groupby(['year-month'])[['forward_three_month_return']].apply(meanad)
else:
    forward_three_month_return_mad = df.groupby(['year-month'])[['forward_three_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_four_month_return']].apply(mad).any() == 0:
    forward_four_month_return_mad = df.groupby(['year-month'])[['forward_four_month_return']].apply(meanad)
else:
    forward_four_month_return_mad = df.groupby(['year-month'])[['forward_four_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_five_month_return']].apply(mad).any() == 0:
    forward_five_month_return_mad = df.groupby(['year-month'])[['forward_five_month_return']].apply(meanad)
else:
    forward_five_month_return_mad = df.groupby(['year-month'])[['forward_five_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_six_month_return']].apply(mad).any() == 0:
    forward_six_month_return_mad = df.groupby(['year-month'])[['forward_six_month_return']].apply(meanad)
else:
    forward_six_month_return_mad = df.groupby(['year-month'])[['forward_six_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_seven_month_return']].apply(mad).any() == 0:
    forward_seven_month_return_mad = df.groupby(['year-month'])[['forward_seven_month_return']].apply(meanad)
else:
    forward_seven_month_return_mad = df.groupby(['year-month'])[['forward_seven_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_eight_month_return']].apply(mad).any() == 0:
    forward_eight_month_return_mad = df.groupby(['year-month'])[['forward_eight_month_return']].apply(meanad)
else:
    forward_eight_month_return_mad = df.groupby(['year-month'])[['forward_eight_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_nine_month_return']].apply(mad).any() == 0:
    forward_nine_month_return_mad = df.groupby(['year-month'])[['forward_nine_month_return']].apply(meanad)
else:
    forward_nine_month_return_mad = df.groupby(['year-month'])[['forward_nine_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_ten_month_return']].apply(mad).any() == 0:
    forward_ten_month_return_mad = df.groupby(['year-month'])[['forward_ten_month_return']].apply(meanad)
else:
    forward_ten_month_return_mad = df.groupby(['year-month'])[['forward_ten_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_eleven_month_return']].apply(mad).any() == 0:
    forward_eleven_month_return_mad = df.groupby(['year-month'])[['forward_eleven_month_return']].apply(meanad)
else:
    forward_eleven_month_return_mad = df.groupby(['year-month'])[['forward_eleven_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twelve_month_return']].apply(mad).any() == 0:
    forward_twelve_month_return_mad = df.groupby(['year-month'])[['forward_twelve_month_return']].apply(meanad)
else:
    forward_twelve_month_return_mad = df.groupby(['year-month'])[['forward_twelve_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirteen_month_return']].apply(mad).any() == 0:
    forward_thirteen_month_return_mad = df.groupby(['year-month'])[['forward_thirteen_month_return']].apply(meanad)
else:
    forward_thirteen_month_return_mad = df.groupby(['year-month'])[['forward_thirteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_fourteen_month_return']].apply(mad).any() == 0:
    forward_fourteen_month_return_mad = df.groupby(['year-month'])[['forward_fourteen_month_return']].apply(meanad)
else:
    forward_fourteen_month_return_mad = df.groupby(['year-month'])[['forward_fourteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_fifteen_month_return']].apply(mad).any() == 0:
    forward_fifteen_month_return_mad = df.groupby(['year-month'])[['forward_fifteen_month_return']].apply(meanad)
else:
    forward_fifteen_month_return_mad = df.groupby(['year-month'])[['forward_fifteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_sixteen_month_return']].apply(mad).any() == 0:
    forward_sixteen_month_return_mad = df.groupby(['year-month'])[['forward_sixteen_month_return']].apply(meanad)
else:
    forward_sixteen_month_return_mad = df.groupby(['year-month'])[['forward_sixteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_seventeen_month_return']].apply(mad).any() == 0:
    forward_seventeen_month_return_mad = df.groupby(['year-month'])[['forward_seventeen_month_return']].apply(meanad)
else:
    forward_seventeen_month_return_mad = df.groupby(['year-month'])[['forward_seventeen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_eighteen_month_return']].apply(mad).any() == 0:
    forward_eighteen_month_return_mad = df.groupby(['year-month'])[['forward_eighteen_month_return']].apply(meanad)
else:
    forward_eighteen_month_return_mad = df.groupby(['year-month'])[['forward_eighteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_nineteen_month_return']].apply(mad).any() == 0:
    forward_nineteen_month_return_mad = df.groupby(['year-month'])[['forward_nineteen_month_return']].apply(meanad)
else:
    forward_nineteen_month_return_mad = df.groupby(['year-month'])[['forward_nineteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twenty_month_return']].apply(mad).any() == 0:
    forward_twenty_month_return_mad = df.groupby(['year-month'])[['forward_twenty_month_return']].apply(meanad)
else:
    forward_twenty_month_return_mad = df.groupby(['year-month'])[['forward_twenty_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentyone_month_return']].apply(mad).any() == 0:
    forward_twentyone_month_return_mad = df.groupby(['year-month'])[['forward_twentyone_month_return']].apply(meanad)
else:
    forward_twentyone_month_return_mad = df.groupby(['year-month'])[['forward_twentyone_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentytwo_month_return']].apply(mad).any() == 0:
    forward_twentytwo_month_return_mad = df.groupby(['year-month'])[['forward_twentytwo_month_return']].apply(meanad)
else:
    forward_twentytwo_month_return_mad = df.groupby(['year-month'])[['forward_twentytwo_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentythree_month_return']].apply(mad).any() == 0:
    forward_twentythree_month_return_mad = df.groupby(['year-month'])[['forward_twentythree_month_return']].apply(meanad)
else:
    forward_twentythree_month_return_mad = df.groupby(['year-month'])[['forward_twentythree_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentyfour_month_return']].apply(mad).any() == 0:
    forward_twentyfour_month_return_mad = df.groupby(['year-month'])[['forward_twentyfour_month_return']].apply(meanad)
else:
    forward_twentyfour_month_return_mad = df.groupby(['year-month'])[['forward_twentyfour_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentyfive_month_return']].apply(mad).any() == 0:
    forward_twentyfive_month_return_mad = df.groupby(['year-month'])[['forward_twentyfive_month_return']].apply(meanad)
else:
    forward_twentyfive_month_return_mad = df.groupby(['year-month'])[['forward_twentyfive_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentysix_month_return']].apply(mad).any() == 0:
    forward_twentysix_month_return_mad = df.groupby(['year-month'])[['forward_twentysix_month_return']].apply(meanad)
else:
    forward_twentysix_month_return_mad = df.groupby(['year-month'])[['forward_twentysix_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentyseven_month_return']].apply(mad).any() == 0:
    forward_twentyseven_month_return_mad = df.groupby(['year-month'])[['forward_twentyseven_month_return']].apply(meanad)
else:
    forward_twentyseven_month_return_mad = df.groupby(['year-month'])[['forward_twentyseven_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentyeight_month_return']].apply(mad).any() == 0:
    forward_twentyeight_month_return_mad = df.groupby(['year-month'])[['forward_twentyeight_month_return']].apply(meanad)
else:
    forward_twentyeight_month_return_mad = df.groupby(['year-month'])[['forward_twentyeight_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_twentynine_month_return']].apply(mad).any() == 0:
    forward_twentynine_month_return_mad = df.groupby(['year-month'])[['forward_twentynine_month_return']].apply(meanad)
else:
    forward_twentynine_month_return_mad = df.groupby(['year-month'])[['forward_twentynine_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirty_month_return']].apply(mad).any() == 0:
    forward_thirty_month_return_mad = df.groupby(['year-month'])[['forward_thirty_month_return']].apply(meanad)
else:
    forward_thirty_month_return_mad = df.groupby(['year-month'])[['forward_thirty_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirtyone_month_return']].apply(mad).any() == 0:
    forward_thirtyone_month_return_mad = df.groupby(['year-month'])[['forward_thirtyone_month_return']].apply(meanad)
else:
    forward_thirtyone_month_return_mad = df.groupby(['year-month'])[['forward_thirtyone_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirtytwo_month_return']].apply(mad).any() == 0:
    forward_thirtytwo_month_return_mad = df.groupby(['year-month'])[['forward_thirtytwo_month_return']].apply(meanad)
else:
    forward_thirtytwo_month_return_mad = df.groupby(['year-month'])[['forward_thirtytwo_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirtythree_month_return']].apply(mad).any() == 0:
    forward_thirtythree_month_return_mad = df.groupby(['year-month'])[['forward_thirtythree_month_return']].apply(meanad)
else:
    forward_thirtythree_month_return_mad = df.groupby(['year-month'])[['forward_thirtythree_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirtyfour_month_return']].apply(mad).any() == 0:
    forward_thirtyfour_month_return_mad = df.groupby(['year-month'])[['forward_thirtyfour_month_return']].apply(meanad)
else:
    forward_thirtyfour_month_return_mad = df.groupby(['year-month'])[['forward_thirtyfour_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirtyfive_month_return']].apply(mad).any() == 0:
    forward_thirtyfive_month_return_mad = df.groupby(['year-month'])[['forward_thirtyfive_month_return']].apply(meanad)
else:
    forward_thirtyfive_month_return_mad = df.groupby(['year-month'])[['forward_thirtyfive_month_return']].apply(mad)
if df.groupby(['year-month'])[['forward_thirtysix_month_return']].apply(mad).any() == 0:
    forward_thirtysix_month_return_mad = df.groupby(['year-month'])[['forward_thirtysix_month_return']].apply(meanad)
else:
    forward_thirtysix_month_return_mad = df.groupby(['year-month'])[['forward_thirtysix_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_one_month_return']].apply(mad).any() == 0:
    past_one_month_return_mad = df.groupby(['year-month'])[['past_one_month_return']].apply(meanad)
else:
    past_one_month_return_mad = df.groupby(['year-month'])[['past_one_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_two_month_return']].apply(mad).any() == 0:
    past_two_month_return_mad = df.groupby(['year-month'])[['past_two_month_return']].apply(meanad)
else:
    past_two_month_return_mad = df.groupby(['year-month'])[['past_two_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_three_month_return']].apply(mad).any() == 0:
    past_three_month_return_mad = df.groupby(['year-month'])[['past_three_month_return']].apply(meanad)
else:
    past_three_month_return_mad = df.groupby(['year-month'])[['past_three_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_four_month_return']].apply(mad).any() == 0:
    past_four_month_return_mad = df.groupby(['year-month'])[['past_four_month_return']].apply(meanad)
else:
    past_four_month_return_mad = df.groupby(['year-month'])[['past_four_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_five_month_return']].apply(mad).any() == 0:
    past_five_month_return_mad = df.groupby(['year-month'])[['past_five_month_return']].apply(meanad)
else:
    past_five_month_return_mad = df.groupby(['year-month'])[['past_five_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_six_month_return']].apply(mad).any() == 0:
    past_six_month_return_mad = df.groupby(['year-month'])[['past_six_month_return']].apply(meanad)
else:
    past_six_month_return_mad = df.groupby(['year-month'])[['past_six_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_seven_month_return']].apply(mad).any() == 0:
    past_seven_month_return_mad = df.groupby(['year-month'])[['past_seven_month_return']].apply(meanad)
else:
    past_seven_month_return_mad = df.groupby(['year-month'])[['past_seven_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_eight_month_return']].apply(mad).any() == 0:
    past_eight_month_return_mad = df.groupby(['year-month'])[['past_eight_month_return']].apply(meanad)
else:
    past_eight_month_return_mad = df.groupby(['year-month'])[['past_eight_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_nine_month_return']].apply(mad).any() == 0:
    past_nine_month_return_mad = df.groupby(['year-month'])[['past_nine_month_return']].apply(meanad)
else:
    past_nine_month_return_mad = df.groupby(['year-month'])[['past_nine_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_ten_month_return']].apply(mad).any() == 0:
    past_ten_month_return_mad = df.groupby(['year-month'])[['past_ten_month_return']].apply(meanad)
else:
    past_ten_month_return_mad = df.groupby(['year-month'])[['past_ten_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_eleven_month_return']].apply(mad).any() == 0:
    past_eleven_month_return_mad = df.groupby(['year-month'])[['past_eleven_month_return']].apply(meanad)
else:
    past_eleven_month_return_mad = df.groupby(['year-month'])[['past_eleven_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twelve_month_return']].apply(mad).any() == 0:
    past_twelve_month_return_mad = df.groupby(['year-month'])[['past_twelve_month_return']].apply(meanad)
else:
    past_twelve_month_return_mad = df.groupby(['year-month'])[['past_twelve_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirteen_month_return']].apply(mad).any() == 0:
    past_thirteen_month_return_mad = df.groupby(['year-month'])[['past_thirteen_month_return']].apply(meanad)
else:
    past_thirteen_month_return_mad = df.groupby(['year-month'])[['past_thirteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_fourteen_month_return']].apply(mad).any() == 0:
    past_fourteen_month_return_mad = df.groupby(['year-month'])[['past_fourteen_month_return']].apply(meanad)
else:
    past_fourteen_month_return_mad = df.groupby(['year-month'])[['past_fourteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_fifteen_month_return']].apply(mad).any() == 0:
    past_fifteen_month_return_mad = df.groupby(['year-month'])[['past_fifteen_month_return']].apply(meanad)
else:
    past_fifteen_month_return_mad = df.groupby(['year-month'])[['past_fifteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_sixteen_month_return']].apply(mad).any() == 0:
    past_sixteen_month_return_mad = df.groupby(['year-month'])[['past_sixteen_month_return']].apply(meanad)
else:
    past_sixteen_month_return_mad = df.groupby(['year-month'])[['past_sixteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_seventeen_month_return']].apply(mad).any() == 0:
    past_seventeen_month_return_mad = df.groupby(['year-month'])[['past_seventeen_month_return']].apply(meanad)
else:
    past_seventeen_month_return_mad = df.groupby(['year-month'])[['past_seventeen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_eighteen_month_return']].apply(mad).any() == 0:
    past_eighteen_month_return_mad = df.groupby(['year-month'])[['past_eighteen_month_return']].apply(meanad)
else:
    past_eighteen_month_return_mad = df.groupby(['year-month'])[['past_eighteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_nineteen_month_return']].apply(mad).any() == 0:
    past_nineteen_month_return_mad = df.groupby(['year-month'])[['past_nineteen_month_return']].apply(meanad)
else:
    past_nineteen_month_return_mad = df.groupby(['year-month'])[['past_nineteen_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twenty_month_return']].apply(mad).any() == 0:
    past_twenty_month_return_mad = df.groupby(['year-month'])[['past_twenty_month_return']].apply(meanad)
else:
    past_twenty_month_return_mad = df.groupby(['year-month'])[['past_twenty_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentyone_month_return']].apply(mad).any() == 0:
    past_twentyone_month_return_mad = df.groupby(['year-month'])[['past_twentyone_month_return']].apply(meanad)
else:
    past_twentyone_month_return_mad = df.groupby(['year-month'])[['past_twentyone_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentytwo_month_return']].apply(mad).any() == 0:
    past_twentytwo_month_return_mad = df.groupby(['year-month'])[['past_twentytwo_month_return']].apply(meanad)
else:
    past_twentytwo_month_return_mad = df.groupby(['year-month'])[['past_twentytwo_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentythree_month_return']].apply(mad).any() == 0:
    past_twentythree_month_return_mad = df.groupby(['year-month'])[['past_twentythree_month_return']].apply(meanad)
else:
    past_twentythree_month_return_mad = df.groupby(['year-month'])[['past_twentythree_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentyfour_month_return']].apply(mad).any() == 0:
    past_twentyfour_month_return_mad = df.groupby(['year-month'])[['past_twentyfour_month_return']].apply(meanad)
else:
    past_twentyfour_month_return_mad = df.groupby(['year-month'])[['past_twentyfour_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentyfive_month_return']].apply(mad).any() == 0:
    past_twentyfive_month_return_mad = df.groupby(['year-month'])[['past_twentyfive_month_return']].apply(meanad)
else:
    past_twentyfive_month_return_mad = df.groupby(['year-month'])[['past_twentyfive_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentysix_month_return']].apply(mad).any() == 0:
    past_twentysix_month_return_mad = df.groupby(['year-month'])[['past_twentysix_month_return']].apply(meanad)
else:
    past_twentysix_month_return_mad = df.groupby(['year-month'])[['past_twentysix_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentyseven_month_return']].apply(mad).any() == 0:
    past_twentyseven_month_return_mad = df.groupby(['year-month'])[['past_twentyseven_month_return']].apply(meanad)
else:
    past_twentyseven_month_return_mad = df.groupby(['year-month'])[['past_twentyseven_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentyeight_month_return']].apply(mad).any() == 0:
    past_twentyeight_month_return_mad = df.groupby(['year-month'])[['past_twentyeight_month_return']].apply(meanad)
else:
    past_twentyeight_month_return_mad = df.groupby(['year-month'])[['past_twentyeight_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_twentynine_month_return']].apply(mad).any() == 0:
    past_twentynine_month_return_mad = df.groupby(['year-month'])[['past_twentynine_month_return']].apply(meanad)
else:
    past_twentynine_month_return_mad = df.groupby(['year-month'])[['past_twentynine_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirty_month_return']].apply(mad).any() == 0:
    past_thirty_month_return_mad = df.groupby(['year-month'])[['past_thirty_month_return']].apply(meanad)
else:
    past_thirty_month_return_mad = df.groupby(['year-month'])[['past_thirty_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirtyone_month_return']].apply(mad).any() == 0:
    past_thirtyone_month_return_mad = df.groupby(['year-month'])[['past_thirtyone_month_return']].apply(meanad)
else:
    past_thirtyone_month_return_mad = df.groupby(['year-month'])[['past_thirtyone_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirtytwo_month_return']].apply(mad).any() == 0:
    past_thirtytwo_month_return_mad = df.groupby(['year-month'])[['past_thirtytwo_month_return']].apply(meanad)
else:
    past_thirtytwo_month_return_mad = df.groupby(['year-month'])[['past_thirtytwo_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirtythree_month_return']].apply(mad).any() == 0:
    past_thirtythree_month_return_mad = df.groupby(['year-month'])[['past_thirtythree_month_return']].apply(meanad)
else:
    past_thirtythree_month_return_mad = df.groupby(['year-month'])[['past_thirtythree_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirtyfour_month_return']].apply(mad).any() == 0:
    past_thirtyfour_month_return_mad = df.groupby(['year-month'])[['past_thirtyfour_month_return']].apply(meanad)
else:
    past_thirtyfour_month_return_mad = df.groupby(['year-month'])[['past_thirtyfour_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirtyfive_month_return']].apply(mad).any() == 0:
    past_thirtyfive_month_return_mad = df.groupby(['year-month'])[['past_thirtyfive_month_return']].apply(meanad)
else:
    past_thirtyfive_month_return_mad = df.groupby(['year-month'])[['past_thirtyfive_month_return']].apply(mad)
if df.groupby(['year-month'])[['past_thirtysix_month_return']].apply(mad).any() == 0:
    past_thirtysix_month_return_mad = df.groupby(['year-month'])[['past_thirtysix_month_return']].apply(meanad)
else:
    past_thirtysix_month_return_mad = df.groupby(['year-month'])[['past_thirtysix_month_return']].apply(mad)
if df.groupby(['year-month'])[['accrual']].apply(mad).any() == 0:
    accrual_mad = df.groupby(['year-month'])[['accrual']].apply(meanad)
else:
    accrual_mad = df.groupby(['year-month'])[['accrual']].apply(mad)
if df.groupby(['year-month'])[['adv_sale']].apply(mad).any() == 0:
    adv_sale_mad = df.groupby(['year-month'])[['adv_sale']].apply(meanad)
else:
    adv_sale_mad = df.groupby(['year-month'])[['adv_sale']].apply(mad)
if df.groupby(['year-month'])[['aftret_eq']].apply(mad).any() == 0:
    aftret_eq_mad = df.groupby(['year-month'])[['aftret_eq']].apply(meanad)
else:
    aftret_eq_mad = df.groupby(['year-month'])[['aftret_eq']].apply(mad)
if df.groupby(['year-month'])[['aftret_equity']].apply(mad).any() == 0:
    aftret_equity_mad = df.groupby(['year-month'])[['aftret_equity']].apply(meanad)
else:
    aftret_equity_mad = df.groupby(['year-month'])[['aftret_equity']].apply(mad)
if df.groupby(['year-month'])[['aftret_invcapx']].apply(mad).any() == 0:
    aftret_invcapx_mad = df.groupby(['year-month'])[['aftret_invcapx']].apply(meanad)
else:
    aftret_invcapx_mad = df.groupby(['year-month'])[['aftret_invcapx']].apply(mad)
if df.groupby(['year-month'])[['at_turn']].apply(mad).any() == 0:
    at_turn_mad = df.groupby(['year-month'])[['at_turn']].apply(meanad)
else:
    at_turn_mad = df.groupby(['year-month'])[['at_turn']].apply(mad)
if df.groupby(['year-month'])[['bm']].apply(mad).any() == 0:
    bm_mad = df.groupby(['year-month'])[['bm']].apply(meanad)
else:
    bm_mad = df.groupby(['year-month'])[['bm']].apply(mad)
if df.groupby(['year-month'])[['CAPEI']].apply(mad).any() == 0:
    CAPEI_mad = df.groupby(['year-month'])[['CAPEI']].apply(meanad)
else:
    CAPEI_mad = df.groupby(['year-month'])[['CAPEI']].apply(mad)
if df.groupby(['year-month'])[['capital_ratio']].apply(mad).any() == 0:
    capital_ratio_mad = df.groupby(['year-month'])[['capital_ratio']].apply(meanad)
else:
    capital_ratio_mad = df.groupby(['year-month'])[['capital_ratio']].apply(mad)
if df.groupby(['year-month'])[['cash_conversion']].apply(mad).any() == 0:
    cash_conversion_mad = df.groupby(['year-month'])[['cash_conversion']].apply(meanad)
else:
    cash_conversion_mad = df.groupby(['year-month'])[['cash_conversion']].apply(mad)
if df.groupby(['year-month'])[['cash_debt']].apply(mad).any() == 0:
    cash_debt_mad = df.groupby(['year-month'])[['cash_debt']].apply(meanad)
else:
    cash_debt_mad = df.groupby(['year-month'])[['cash_debt']].apply(mad)
if df.groupby(['year-month'])[['cash_lt']].apply(mad).any() == 0:
    cash_lt_mad = df.groupby(['year-month'])[['cash_lt']].apply(meanad)
else:
    cash_lt_mad = df.groupby(['year-month'])[['cash_lt']].apply(mad)
if df.groupby(['year-month'])[['cash_ratio']].apply(mad).any() == 0:
    cash_ratio_mad = df.groupby(['year-month'])[['cash_ratio']].apply(meanad)
else:
    cash_ratio_mad = df.groupby(['year-month'])[['cash_ratio']].apply(mad)
if df.groupby(['year-month'])[['cfm']].apply(mad).any() == 0:
    cfm_mad = df.groupby(['year-month'])[['cfm']].apply(meanad)
else:
    cfm_mad = df.groupby(['year-month'])[['cfm']].apply(mad)
if df.groupby(['year-month'])[['curr_debt']].apply(mad).any() == 0:
    curr_debt_mad = df.groupby(['year-month'])[['curr_debt']].apply(meanad)
else:
    curr_debt_mad = df.groupby(['year-month'])[['curr_debt']].apply(mad)
if df.groupby(['year-month'])[['curr_ratio']].apply(mad).any() == 0:
    curr_ratio_mad = df.groupby(['year-month'])[['curr_ratio']].apply(meanad)
else:
    curr_ratio_mad = df.groupby(['year-month'])[['curr_ratio']].apply(mad)
if df.groupby(['year-month'])[['de_ratio']].apply(mad).any() == 0:
    de_ratio_mad = df.groupby(['year-month'])[['de_ratio']].apply(meanad)
else:
    de_ratio_mad = df.groupby(['year-month'])[['de_ratio']].apply(mad)
if df.groupby(['year-month'])[['debt_assets']].apply(mad).any() == 0:
    debt_assets_mad = df.groupby(['year-month'])[['debt_assets']].apply(meanad)
else:
    debt_assets_mad = df.groupby(['year-month'])[['debt_assets']].apply(mad)
if df.groupby(['year-month'])[['debt_at']].apply(mad).any() == 0:
    debt_at_mad = df.groupby(['year-month'])[['debt_at']].apply(meanad)
else:
    debt_at_mad = df.groupby(['year-month'])[['debt_at']].apply(mad)
if df.groupby(['year-month'])[['debt_capital']].apply(mad).any() == 0:
    debt_capital_mad = df.groupby(['year-month'])[['debt_capital']].apply(meanad)
else:
    debt_capital_mad = df.groupby(['year-month'])[['debt_capital']].apply(mad)
if df.groupby(['year-month'])[['debt_ebitda']].apply(mad).any() == 0:
    debt_ebitda_mad = df.groupby(['year-month'])[['debt_ebitda']].apply(meanad)
else:
    debt_ebitda_mad = df.groupby(['year-month'])[['debt_ebitda']].apply(mad)
if df.groupby(['year-month'])[['debt_invcap']].apply(mad).any() == 0:
    debt_invcap_mad = df.groupby(['year-month'])[['debt_invcap']].apply(meanad)
else:
    debt_invcap_mad = df.groupby(['year-month'])[['debt_invcap']].apply(mad)
if df.groupby(['year-month'])[['DIVYIELD']].apply(mad).any() == 0:
    DIVYIELD_mad = df.groupby(['year-month'])[['DIVYIELD']].apply(meanad)
else:
    DIVYIELD_mad = df.groupby(['year-month'])[['DIVYIELD']].apply(mad)
if df.groupby(['year-month'])[['dltt_be']].apply(mad).any() == 0:
    dltt_be_mad = df.groupby(['year-month'])[['dltt_be']].apply(meanad)
else:
    dltt_be_mad = df.groupby(['year-month'])[['dltt_be']].apply(mad)
if df.groupby(['year-month'])[['dpr']].apply(mad).any() == 0:
    dpr_mad = df.groupby(['year-month'])[['dpr']].apply(meanad)
else:
    dpr_mad = df.groupby(['year-month'])[['dpr']].apply(mad)
if df.groupby(['year-month'])[['efftax']].apply(mad).any() == 0:
    efftax_mad = df.groupby(['year-month'])[['efftax']].apply(meanad)
else:
    efftax_mad = df.groupby(['year-month'])[['efftax']].apply(mad)
if df.groupby(['year-month'])[['equity_invcap']].apply(mad).any() == 0:
    equity_invcap_mad = df.groupby(['year-month'])[['equity_invcap']].apply(meanad)
else:
    equity_invcap_mad = df.groupby(['year-month'])[['equity_invcap']].apply(mad)
if df.groupby(['year-month'])[['evm']].apply(mad).any() == 0:
    evm_mad = df.groupby(['year-month'])[['evm']].apply(meanad)
else:
    evm_mad = df.groupby(['year-month'])[['evm']].apply(mad)
if df.groupby(['year-month'])[['fcf_ocf']].apply(mad).any() == 0:
    fcf_ocf_mad = df.groupby(['year-month'])[['fcf_ocf']].apply(meanad)
else:
    fcf_ocf_mad = df.groupby(['year-month'])[['fcf_ocf']].apply(mad)
if df.groupby(['year-month'])[['gpm']].apply(mad).any() == 0:
    gpm_mad = df.groupby(['year-month'])[['gpm']].apply(meanad)
else:
    gpm_mad = df.groupby(['year-month'])[['gpm']].apply(mad)
if df.groupby(['year-month'])[['GProf']].apply(mad).any() == 0:
    GProf_mad = df.groupby(['year-month'])[['GProf']].apply(meanad)
else:
    GProf_mad = df.groupby(['year-month'])[['GProf']].apply(mad)
if df.groupby(['year-month'])[['int_debt']].apply(mad).any() == 0:
    int_debt_mad = df.groupby(['year-month'])[['int_debt']].apply(meanad)
else:
    int_debt_mad = df.groupby(['year-month'])[['int_debt']].apply(mad)
if df.groupby(['year-month'])[['int_totdebt']].apply(mad).any() == 0:
    int_totdebt_mad = df.groupby(['year-month'])[['int_totdebt']].apply(meanad)
else:
    int_totdebt_mad = df.groupby(['year-month'])[['int_totdebt']].apply(mad)
if df.groupby(['year-month'])[['intcov']].apply(mad).any() == 0:
    intcov_mad = df.groupby(['year-month'])[['intcov']].apply(meanad)
else:
    intcov_mad = df.groupby(['year-month'])[['intcov']].apply(mad)
if df.groupby(['year-month'])[['intcov_ratio']].apply(mad).any() == 0:
    intcov_ratio_mad = df.groupby(['year-month'])[['intcov_ratio']].apply(meanad)
else:
    intcov_ratio_mad = df.groupby(['year-month'])[['intcov_ratio']].apply(mad)
if df.groupby(['year-month'])[['inv_turn']].apply(mad).any() == 0:
    inv_turn_mad = df.groupby(['year-month'])[['inv_turn']].apply(meanad)
else:
    inv_turn_mad = df.groupby(['year-month'])[['inv_turn']].apply(mad)
if df.groupby(['year-month'])[['invt_act']].apply(mad).any() == 0:
    invt_act_mad = df.groupby(['year-month'])[['invt_act']].apply(meanad)
else:
    invt_act_mad = df.groupby(['year-month'])[['invt_act']].apply(mad)
if df.groupby(['year-month'])[['lt_debt']].apply(mad).any() == 0:
    lt_debt_mad = df.groupby(['year-month'])[['lt_debt']].apply(meanad)
else:
    lt_debt_mad = df.groupby(['year-month'])[['lt_debt']].apply(mad)
if df.groupby(['year-month'])[['lt_ppent']].apply(mad).any() == 0:
    lt_ppent_mad = df.groupby(['year-month'])[['lt_ppent']].apply(meanad)
else:
    lt_ppent_mad = df.groupby(['year-month'])[['lt_ppent']].apply(mad)
if df.groupby(['year-month'])[['npm']].apply(mad).any() == 0:
    npm_mad = df.groupby(['year-month'])[['npm']].apply(meanad)
else:
    npm_mad = df.groupby(['year-month'])[['npm']].apply(mad)
if df.groupby(['year-month'])[['ocf_lct']].apply(mad).any() == 0:
    ocf_lct_mad = df.groupby(['year-month'])[['ocf_lct']].apply(meanad)
else:
    ocf_lct_mad = df.groupby(['year-month'])[['ocf_lct']].apply(mad)
if df.groupby(['year-month'])[['opmad']].apply(mad).any() == 0:
    opmad_mad = df.groupby(['year-month'])[['opmad']].apply(meanad)
else:
    opmad_mad = df.groupby(['year-month'])[['opmad']].apply(mad)
if df.groupby(['year-month'])[['opmbd']].apply(mad).any() == 0:
    opmbd_mad = df.groupby(['year-month'])[['opmbd']].apply(meanad)
else:
    opmbd_mad = df.groupby(['year-month'])[['opmbd']].apply(mad)
if df.groupby(['year-month'])[['pay_turn']].apply(mad).any() == 0:
    pay_turn_mad = df.groupby(['year-month'])[['pay_turn']].apply(meanad)
else:
    pay_turn_mad = df.groupby(['year-month'])[['pay_turn']].apply(mad)
if df.groupby(['year-month'])[['pcf']].apply(mad).any() == 0:
    pcf_mad = df.groupby(['year-month'])[['pcf']].apply(meanad)
else:
    pcf_mad = df.groupby(['year-month'])[['pcf']].apply(mad)
if df.groupby(['year-month'])[['pe_exi']].apply(mad).any() == 0:
    pe_exi_mad = df.groupby(['year-month'])[['pe_exi']].apply(meanad)
else:
    pe_exi_mad = df.groupby(['year-month'])[['pe_exi']].apply(mad)
if df.groupby(['year-month'])[['pe_inc']].apply(mad).any() == 0:
    pe_inc_mad = df.groupby(['year-month'])[['pe_inc']].apply(meanad)
else:
    pe_inc_mad = df.groupby(['year-month'])[['pe_inc']].apply(mad)
if df.groupby(['year-month'])[['pe_op_basic']].apply(mad).any() == 0:
    pe_op_basic_mad = df.groupby(['year-month'])[['pe_op_basic']].apply(meanad)
else:
    pe_op_basic_mad = df.groupby(['year-month'])[['pe_op_basic']].apply(mad)
if df.groupby(['year-month'])[['pe_op_dil']].apply(mad).any() == 0:
    pe_op_dil_mad = df.groupby(['year-month'])[['pe_op_dil']].apply(meanad)
else:
    pe_op_dil_mad = df.groupby(['year-month'])[['pe_op_dil']].apply(mad)
if df.groupby(['year-month'])[['PEG_1yrforward']].apply(mad).any() == 0:
    PEG_1yrforward_mad = df.groupby(['year-month'])[['PEG_1yrforward']].apply(meanad)
else:
    PEG_1yrforward_mad = df.groupby(['year-month'])[['PEG_1yrforward']].apply(mad)
if df.groupby(['year-month'])[['PEG_ltgforward']].apply(mad).any() == 0:
    PEG_ltgforward_mad = df.groupby(['year-month'])[['PEG_ltgforward']].apply(meanad)
else:
    PEG_ltgforward_mad = df.groupby(['year-month'])[['PEG_ltgforward']].apply(mad)
if df.groupby(['year-month'])[['PEG_trailing']].apply(mad).any() == 0:
    PEG_trailing_mad = df.groupby(['year-month'])[['PEG_trailing']].apply(meanad)
else:
    PEG_trailing_mad = df.groupby(['year-month'])[['PEG_trailing']].apply(mad)
if df.groupby(['year-month'])[['pretret_earnat']].apply(mad).any() == 0:
    pretret_earnat_mad = df.groupby(['year-month'])[['pretret_earnat']].apply(meanad)
else:
    pretret_earnat_mad = df.groupby(['year-month'])[['pretret_earnat']].apply(mad)
if df.groupby(['year-month'])[['pretret_noa']].apply(mad).any() == 0:
    pretret_noa_mad = df.groupby(['year-month'])[['pretret_noa']].apply(meanad)
else:
    pretret_noa_mad = df.groupby(['year-month'])[['pretret_noa']].apply(mad)
if df.groupby(['year-month'])[['profit_lct']].apply(mad).any() == 0:
    profit_lct_mad = df.groupby(['year-month'])[['profit_lct']].apply(meanad)
else:
    profit_lct_mad = df.groupby(['year-month'])[['profit_lct']].apply(mad)
if df.groupby(['year-month'])[['ps']].apply(mad).any() == 0:
    ps_mad = df.groupby(['year-month'])[['ps']].apply(meanad)
else:
    ps_mad = df.groupby(['year-month'])[['ps']].apply(mad)
if df.groupby(['year-month'])[['ptb']].apply(mad).any() == 0:
    ptb_mad = df.groupby(['year-month'])[['ptb']].apply(meanad)
else:
    ptb_mad = df.groupby(['year-month'])[['ptb']].apply(mad)
if df.groupby(['year-month'])[['ptpm']].apply(mad).any() == 0:
    ptpm_mad = df.groupby(['year-month'])[['ptpm']].apply(meanad)
else:
    ptpm_mad = df.groupby(['year-month'])[['ptpm']].apply(mad)
if df.groupby(['year-month'])[['quick_ratio']].apply(mad).any() == 0:
    quick_ratio_mad = df.groupby(['year-month'])[['quick_ratio']].apply(meanad)
else:
    quick_ratio_mad = df.groupby(['year-month'])[['quick_ratio']].apply(mad)
if df.groupby(['year-month'])[['rd_sale']].apply(mad).any() == 0:
    rd_sale_mad = df.groupby(['year-month'])[['rd_sale']].apply(meanad)
else:
    rd_sale_mad = df.groupby(['year-month'])[['rd_sale']].apply(mad)
if df.groupby(['year-month'])[['rect_act']].apply(mad).any() == 0:
    rect_act_mad = df.groupby(['year-month'])[['rect_act']].apply(meanad)
else:
    rect_act_mad = df.groupby(['year-month'])[['rect_act']].apply(mad)
if df.groupby(['year-month'])[['rect_turn']].apply(mad).any() == 0:
    rect_turn_mad = df.groupby(['year-month'])[['rect_turn']].apply(meanad)
else:
    rect_turn_mad = df.groupby(['year-month'])[['rect_turn']].apply(mad)
if df.groupby(['year-month'])[['roa']].apply(mad).any() == 0:
    roa_mad = df.groupby(['year-month'])[['roa']].apply(meanad)
else:
    roa_mad = df.groupby(['year-month'])[['roa']].apply(mad)
if df.groupby(['year-month'])[['roce']].apply(mad).any() == 0:
    roce_mad = df.groupby(['year-month'])[['roce']].apply(meanad)
else:
    roce_mad = df.groupby(['year-month'])[['roce']].apply(mad)
if df.groupby(['year-month'])[['roe']].apply(mad).any() == 0:
    roe_mad = df.groupby(['year-month'])[['roe']].apply(meanad)
else:
    roe_mad = df.groupby(['year-month'])[['roe']].apply(mad)
if df.groupby(['year-month'])[['sale_equity']].apply(mad).any() == 0:
    sale_equity_mad = df.groupby(['year-month'])[['sale_equity']].apply(meanad)
else:
    sale_equity_mad = df.groupby(['year-month'])[['sale_equity']].apply(mad)
if df.groupby(['year-month'])[['sale_invcap']].apply(mad).any() == 0:
    sale_invcap_mad = df.groupby(['year-month'])[['sale_invcap']].apply(meanad)
else:
    sale_invcap_mad = df.groupby(['year-month'])[['sale_invcap']].apply(mad)
if df.groupby(['year-month'])[['sale_nwc']].apply(mad).any() == 0:
    sale_nwc_mad = df.groupby(['year-month'])[['sale_nwc']].apply(meanad)
else:
    sale_nwc_mad = df.groupby(['year-month'])[['sale_nwc']].apply(mad)
if df.groupby(['year-month'])[['short_debt']].apply(mad).any() == 0:
    short_debt_mad = df.groupby(['year-month'])[['short_debt']].apply(meanad)
else:
    short_debt_mad = df.groupby(['year-month'])[['short_debt']].apply(mad)
if df.groupby(['year-month'])[['staff_sale']].apply(mad).any() == 0:
    staff_sale_mad = df.groupby(['year-month'])[['staff_sale']].apply(meanad)
else:
    staff_sale_mad = df.groupby(['year-month'])[['staff_sale']].apply(mad)
if df.groupby(['year-month'])[['totdebt_invcap']].apply(mad).any() == 0:
    totdebt_invcap_mad = df.groupby(['year-month'])[['totdebt_invcap']].apply(meanad)
else:
    totdebt_invcap_mad = df.groupby(['year-month'])[['totdebt_invcap']].apply(mad)
if df.groupby(['year-month'])[['dvpspm']].apply(mad).any() == 0:
    dvpspm_mad = df.groupby(['year-month'])[['dvpspm']].apply(meanad)
else:
    dvpspm_mad = df.groupby(['year-month'])[['dvpspm']].apply(mad)
if df.groupby(['year-month'])[['dvpsxm']].apply(mad).any() == 0:
    dvpsxm_mad = df.groupby(['year-month'])[['dvpsxm']].apply(meanad)
else:
    dvpsxm_mad = df.groupby(['year-month'])[['dvpsxm']].apply(mad)
if df.groupby(['year-month'])[['dvrate']].apply(mad).any() == 0:
    dvrate_mad = df.groupby(['year-month'])[['dvrate']].apply(meanad)
else:
    dvrate_mad = df.groupby(['year-month'])[['dvrate']].apply(mad)
if df.groupby(['year-month'])[['spcsrc']].apply(mad).any() == 0:
    spcsrc_mad = df.groupby(['year-month'])[['spcsrc']].apply(meanad)
else:
    spcsrc_mad = df.groupby(['year-month'])[['spcsrc']].apply(mad)
if df.groupby(['year-month'])[['alpha']].apply(mad).any() == 0:
    alpha_mad = df.groupby(['year-month'])[['alpha']].apply(meanad)
else:
    alpha_mad = df.groupby(['year-month'])[['alpha']].apply(mad)
if df.groupby(['year-month'])[['b_hml']].apply(mad).any() == 0:
    b_hml_mad = df.groupby(['year-month'])[['b_hml']].apply(meanad)
else:
    b_hml_mad = df.groupby(['year-month'])[['b_hml']].apply(mad)
if df.groupby(['year-month'])[['b_mkt']].apply(mad).any() == 0:
    b_mkt_mad = df.groupby(['year-month'])[['b_mkt']].apply(meanad)
else:
    b_mkt_mad = df.groupby(['year-month'])[['b_mkt']].apply(mad)
if df.groupby(['year-month'])[['b_smb']].apply(mad).any() == 0:
    b_smb_mad = df.groupby(['year-month'])[['b_smb']].apply(meanad)
else:
    b_smb_mad = df.groupby(['year-month'])[['b_smb']].apply(mad)
if df.groupby(['year-month'])[['b_umd']].apply(mad).any() == 0:
    b_umd_mad = df.groupby(['year-month'])[['b_umd']].apply(meanad)
else:
    b_umd_mad = df.groupby(['year-month'])[['b_umd']].apply(mad)
if df.groupby(['year-month'])[['exret']].apply(mad).any() == 0:
    exret_mad = df.groupby(['year-month'])[['exret']].apply(meanad)
else:
    exret_mad = df.groupby(['year-month'])[['exret']].apply(mad)
if df.groupby(['year-month'])[['ivol']].apply(mad).any() == 0:
    ivol_mad = df.groupby(['year-month'])[['ivol']].apply(meanad)
else:
    ivol_mad = df.groupby(['year-month'])[['ivol']].apply(mad)
if df.groupby(['year-month'])[['n']].apply(mad).any() == 0:
    n_mad = df.groupby(['year-month'])[['n']].apply(meanad)
else:
    n_mad = df.groupby(['year-month'])[['n']].apply(mad)
if df.groupby(['year-month'])[['R2']].apply(mad).any() == 0:
    R2_mad = df.groupby(['year-month'])[['R2']].apply(meanad)
else:
    R2_mad = df.groupby(['year-month'])[['R2']].apply(mad)
if df.groupby(['year-month'])[['tvol']].apply(mad).any() == 0:
    tvol_mad = df.groupby(['year-month'])[['tvol']].apply(meanad)
else:
    tvol_mad = df.groupby(['year-month'])[['tvol']].apply(mad)
if df.groupby(['year-month'])[['BUYPCT']].apply(mad).any() == 0:
    BUYPCT_mad = df.groupby(['year-month'])[['BUYPCT']].apply(meanad)
else:
    BUYPCT_mad = df.groupby(['year-month'])[['BUYPCT']].apply(mad)
if df.groupby(['year-month'])[['HOLDPCT']].apply(mad).any() == 0:
    HOLDPCT_mad = df.groupby(['year-month'])[['HOLDPCT']].apply(meanad)
else:
    HOLDPCT_mad = df.groupby(['year-month'])[['HOLDPCT']].apply(mad)
if df.groupby(['year-month'])[['MEANREC']].apply(mad).any() == 0:
    MEANREC_mad = df.groupby(['year-month'])[['MEANREC']].apply(meanad)
else:
    MEANREC_mad = df.groupby(['year-month'])[['MEANREC']].apply(mad)
if df.groupby(['year-month'])[['MEDREC']].apply(mad).any() == 0:
    MEDREC_mad = df.groupby(['year-month'])[['MEDREC']].apply(meanad)
else:
    MEDREC_mad = df.groupby(['year-month'])[['MEDREC']].apply(mad)
if df.groupby(['year-month'])[['recup']].apply(mad).any() == 0:
    recup_mad = df.groupby(['year-month'])[['recup']].apply(meanad)
else:
    recup_mad = df.groupby(['year-month'])[['recup']].apply(mad)
if df.groupby(['year-month'])[['recdown']].apply(mad).any() == 0:
    recdown_mad = df.groupby(['year-month'])[['recdown']].apply(meanad)
else:
    recdown_mad = df.groupby(['year-month'])[['recdown']].apply(mad)
if df.groupby(['year-month'])[['SELLPCT']].apply(mad).any() == 0:
    SELLPCT_mad = df.groupby(['year-month'])[['SELLPCT']].apply(meanad)
else:
    SELLPCT_mad = df.groupby(['year-month'])[['SELLPCT']].apply(mad)
if df.groupby(['year-month'])[['STDEV']].apply(mad).any() == 0:
    STDEV_mad = df.groupby(['year-month'])[['STDEV']].apply(meanad)
else:
    STDEV_mad = df.groupby(['year-month'])[['STDEV']].apply(mad)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)
C:\Users\

In [64]:
# Market MAD Datasets - Enrich - All
forward_one_month_return_mad.name = 'forward_one_month_return_mad'
forward_two_month_return_mad.name = 'forward_two_month_return_mad'
forward_three_month_return_mad.name = 'forward_three_month_return_mad'
forward_four_month_return_mad.name = 'forward_four_month_return_mad'
forward_five_month_return_mad.name = 'forward_five_month_return_mad'
forward_six_month_return_mad.name = 'forward_six_month_return_mad'
forward_seven_month_return_mad.name = 'forward_seven_month_return_mad'
forward_eight_month_return_mad.name = 'forward_eight_month_return_mad'
forward_nine_month_return_mad.name = 'forward_nine_month_return_mad'
forward_ten_month_return_mad.name = 'forward_ten_month_return_mad'
forward_eleven_month_return_mad.name = 'forward_eleven_month_return_mad'
forward_twelve_month_return_mad.name = 'forward_twelve_month_return_mad'
forward_thirteen_month_return_mad.name = 'forward_thirteen_month_return_mad'
forward_fourteen_month_return_mad.name = 'forward_fourteen_month_return_mad'
forward_fifteen_month_return_mad.name = 'forward_fifteen_month_return_mad'
forward_sixteen_month_return_mad.name = 'forward_sixteen_month_return_mad'
forward_seventeen_month_return_mad.name = 'forward_seventeen_month_return_mad'
forward_eighteen_month_return_mad.name = 'forward_eighteen_month_return_mad'
forward_nineteen_month_return_mad.name = 'forward_nineteen_month_return_mad'
forward_twenty_month_return_mad.name = 'forward_twenty_month_return_mad'
forward_twentyone_month_return_mad.name = 'forward_twentyone_month_return_mad'
forward_twentytwo_month_return_mad.name = 'forward_twentytwo_month_return_mad'
forward_twentythree_month_return_mad.name = 'forward_twentythree_month_return_mad'
forward_twentyfour_month_return_mad.name = 'forward_twentyfour_month_return_mad'
forward_twentyfive_month_return_mad.name = 'forward_twentyfive_month_return_mad'
forward_twentysix_month_return_mad.name = 'forward_twentysix_month_return_mad'
forward_twentyseven_month_return_mad.name = 'forward_twentyseven_month_return_mad'
forward_twentyeight_month_return_mad.name = 'forward_twentyeight_month_return_mad'
forward_twentynine_month_return_mad.name = 'forward_twentynine_month_return_mad'
forward_thirty_month_return_mad.name = 'forward_thirty_month_return_mad'
forward_thirtyone_month_return_mad.name = 'forward_thirtyone_month_return_mad'
forward_thirtytwo_month_return_mad.name = 'forward_thirtytwo_month_return_mad'
forward_thirtythree_month_return_mad.name = 'forward_thirtythree_month_return_mad'
forward_thirtyfour_month_return_mad.name = 'forward_thirtyfour_month_return_mad'
forward_thirtyfive_month_return_mad.name = 'forward_thirtyfive_month_return_mad'
forward_thirtysix_month_return_mad.name = 'forward_thirtysix_month_return_mad'
past_one_month_return_mad.name = 'past_one_month_return_mad'
past_two_month_return_mad.name = 'past_two_month_return_mad'
past_three_month_return_mad.name = 'past_three_month_return_mad'
past_four_month_return_mad.name = 'past_four_month_return_mad'
past_five_month_return_mad.name = 'past_five_month_return_mad'
past_six_month_return_mad.name = 'past_six_month_return_mad'
past_seven_month_return_mad.name = 'past_seven_month_return_mad'
past_eight_month_return_mad.name = 'past_eight_month_return_mad'
past_nine_month_return_mad.name = 'past_nine_month_return_mad'
past_ten_month_return_mad.name = 'past_ten_month_return_mad'
past_eleven_month_return_mad.name = 'past_eleven_month_return_mad'
past_twelve_month_return_mad.name = 'past_twelve_month_return_mad'
past_thirteen_month_return_mad.name = 'past_thirteen_month_return_mad'
past_fourteen_month_return_mad.name = 'past_fourteen_month_return_mad'
past_fifteen_month_return_mad.name = 'past_fifteen_month_return_mad'
past_sixteen_month_return_mad.name = 'past_sixteen_month_return_mad'
past_seventeen_month_return_mad.name = 'past_seventeen_month_return_mad'
past_eighteen_month_return_mad.name = 'past_eighteen_month_return_mad'
past_nineteen_month_return_mad.name = 'past_nineteen_month_return_mad'
past_twenty_month_return_mad.name = 'past_twenty_month_return_mad'
past_twentyone_month_return_mad.name = 'past_twentyone_month_return_mad'
past_twentytwo_month_return_mad.name = 'past_twentytwo_month_return_mad'
past_twentythree_month_return_mad.name = 'past_twentythree_month_return_mad'
past_twentyfour_month_return_mad.name = 'past_twentyfour_month_return_mad'
past_twentyfive_month_return_mad.name = 'past_twentyfive_month_return_mad'
past_twentysix_month_return_mad.name = 'past_twentysix_month_return_mad'
past_twentyseven_month_return_mad.name = 'past_twentyseven_month_return_mad'
past_twentyeight_month_return_mad.name = 'past_twentyeight_month_return_mad'
past_twentynine_month_return_mad.name = 'past_twentynine_month_return_mad'
past_thirty_month_return_mad.name = 'past_thirty_month_return_mad'
past_thirtyone_month_return_mad.name = 'past_thirtyone_month_return_mad'
past_thirtytwo_month_return_mad.name = 'past_thirtytwo_month_return_mad'
past_thirtythree_month_return_mad.name = 'past_thirtythree_month_return_mad'
past_thirtyfour_month_return_mad.name = 'past_thirtyfour_month_return_mad'
past_thirtyfive_month_return_mad.name = 'past_thirtyfive_month_return_mad'
past_thirtysix_month_return_mad.name = 'past_thirtysix_month_return_mad'
accrual_mad.name = 'accrual_mad'
adv_sale_mad.name = 'adv_sale_mad'
aftret_eq_mad.name = 'aftret_eq_mad'
aftret_equity_mad.name = 'aftret_equity_mad'
aftret_invcapx_mad.name = 'aftret_invcapx_mad'
at_turn_mad.name = 'at_turn_mad'
bm_mad.name = 'bm_mad'
CAPEI_mad.name = 'CAPEI_mad'
capital_ratio_mad.name = 'capital_ratio_mad'
cash_conversion_mad.name = 'cash_conversion_mad'
cash_debt_mad.name = 'cash_debt_mad'
cash_lt_mad.name = 'cash_lt_mad'
cash_ratio_mad.name = 'cash_ratio_mad'
cfm_mad.name = 'cfm_mad'
curr_debt_mad.name = 'curr_debt_mad'
curr_ratio_mad.name = 'curr_ratio_mad'
de_ratio_mad.name = 'de_ratio_mad'
debt_assets_mad.name = 'debt_assets_mad'
debt_at_mad.name = 'debt_at_mad'
debt_capital_mad.name = 'debt_capital_mad'
debt_ebitda_mad.name = 'debt_ebitda_mad'
debt_invcap_mad.name = 'debt_invcap_mad'
DIVYIELD_mad.name = 'DIVYIELD_mad'
dltt_be_mad.name = 'dltt_be_mad'
dpr_mad.name = 'dpr_mad'
efftax_mad.name = 'efftax_mad'
equity_invcap_mad.name = 'equity_invcap_mad'
evm_mad.name = 'evm_mad'
fcf_ocf_mad.name = 'fcf_ocf_mad'
gpm_mad.name = 'gpm_mad'
GProf_mad.name = 'GProf_mad'
int_debt_mad.name = 'int_debt_mad'
int_totdebt_mad.name = 'int_totdebt_mad'
intcov_mad.name = 'intcov_mad'
intcov_ratio_mad.name = 'intcov_ratio_mad'
inv_turn_mad.name = 'inv_turn_mad'
invt_act_mad.name = 'invt_act_mad'
lt_debt_mad.name = 'lt_debt_mad'
lt_ppent_mad.name = 'lt_ppent_mad'
npm_mad.name = 'npm_mad'
ocf_lct_mad.name = 'ocf_lct_mad'
opmad_mad.name = 'opmad_mad'
opmbd_mad.name = 'opmbd_mad'
pay_turn_mad.name = 'pay_turn_mad'
pcf_mad.name = 'pcf_mad'
pe_exi_mad.name = 'pe_exi_mad'
pe_inc_mad.name = 'pe_inc_mad'
pe_op_basic_mad.name = 'pe_op_basic_mad'
pe_op_dil_mad.name = 'pe_op_dil_mad'
PEG_1yrforward_mad.name = 'PEG_1yrforward_mad'
PEG_ltgforward_mad.name = 'PEG_ltgforward_mad'
PEG_trailing_mad.name = 'PEG_trailing_mad'
pretret_earnat_mad.name = 'pretret_earnat_mad'
pretret_noa_mad.name = 'pretret_noa_mad'
profit_lct_mad.name = 'profit_lct_mad'
ps_mad.name = 'ps_mad'
ptb_mad.name = 'ptb_mad'
ptpm_mad.name = 'ptpm_mad'
quick_ratio_mad.name = 'quick_ratio_mad'
rd_sale_mad.name = 'rd_sale_mad'
rect_act_mad.name = 'rect_act_mad'
rect_turn_mad.name = 'rect_turn_mad'
roa_mad.name = 'roa_mad'
roce_mad.name = 'roce_mad'
roe_mad.name = 'roe_mad'
sale_equity_mad.name = 'sale_equity_mad'
sale_invcap_mad.name = 'sale_invcap_mad'
sale_nwc_mad.name = 'sale_nwc_mad'
short_debt_mad.name = 'short_debt_mad'
staff_sale_mad.name = 'staff_sale_mad'
totdebt_invcap_mad.name = 'totdebt_invcap_mad'
dvpspm_mad.name = 'dvpspm_mad'
dvpsxm_mad.name = 'dvpsxm_mad'
dvrate_mad.name = 'dvrate_mad'
spcsrc_mad.name = 'spcsrc_mad'
alpha_mad.name = 'alpha_mad'
b_hml_mad.name = 'b_hml_mad'
b_mkt_mad.name = 'b_mkt_mad'
b_smb_mad.name = 'b_smb_mad'
b_umd_mad.name = 'b_umd_mad'
exret_mad.name = 'exret_mad'
ivol_mad.name = 'ivol_mad'
n_mad.name = 'n_mad'
R2_mad.name = 'R2_mad'
tvol_mad.name = 'tvol_mad'
BUYPCT_mad.name = 'BUYPCT_mad'
HOLDPCT_mad.name = 'HOLDPCT_mad'
MEANREC_mad.name = 'MEANREC_mad'
MEDREC_mad.name = 'MEDREC_mad'
recup_mad.name = 'recup_mad'
recdown_mad.name = 'recdown_mad'
SELLPCT_mad.name = 'SELLPCT_mad'
STDEV_mad.name = 'STDEV_mad'

In [65]:
# Join - df & Market Median Datasets
df = df.join(forward_one_month_return_mad, on=['year-month'])
df = df.join(forward_two_month_return_mad, on=['year-month'])
df = df.join(forward_three_month_return_mad, on=['year-month'])
df = df.join(forward_four_month_return_mad, on=['year-month'])
df = df.join(forward_five_month_return_mad, on=['year-month'])
df = df.join(forward_six_month_return_mad, on=['year-month'])
df = df.join(forward_seven_month_return_mad, on=['year-month'])
df = df.join(forward_eight_month_return_mad, on=['year-month'])
df = df.join(forward_nine_month_return_mad, on=['year-month'])
df = df.join(forward_ten_month_return_mad, on=['year-month'])
df = df.join(forward_eleven_month_return_mad, on=['year-month'])
df = df.join(forward_twelve_month_return_mad, on=['year-month'])
df = df.join(forward_thirteen_month_return_mad, on=['year-month'])
df = df.join(forward_fourteen_month_return_mad, on=['year-month'])
df = df.join(forward_fifteen_month_return_mad, on=['year-month'])
df = df.join(forward_sixteen_month_return_mad, on=['year-month'])
df = df.join(forward_seventeen_month_return_mad, on=['year-month'])
df = df.join(forward_eighteen_month_return_mad, on=['year-month'])
df = df.join(forward_nineteen_month_return_mad, on=['year-month'])
df = df.join(forward_twenty_month_return_mad, on=['year-month'])
df = df.join(forward_twentyone_month_return_mad, on=['year-month'])
df = df.join(forward_twentytwo_month_return_mad, on=['year-month'])
df = df.join(forward_twentythree_month_return_mad, on=['year-month'])
df = df.join(forward_twentyfour_month_return_mad, on=['year-month'])
df = df.join(forward_twentyfive_month_return_mad, on=['year-month'])
df = df.join(forward_twentysix_month_return_mad, on=['year-month'])
df = df.join(forward_twentyseven_month_return_mad, on=['year-month'])
df = df.join(forward_twentyeight_month_return_mad, on=['year-month'])
df = df.join(forward_twentynine_month_return_mad, on=['year-month'])
df = df.join(forward_thirty_month_return_mad, on=['year-month'])
df = df.join(forward_thirtyone_month_return_mad, on=['year-month'])
df = df.join(forward_thirtytwo_month_return_mad, on=['year-month'])
df = df.join(forward_thirtythree_month_return_mad, on=['year-month'])
df = df.join(forward_thirtyfour_month_return_mad, on=['year-month'])
df = df.join(forward_thirtyfive_month_return_mad, on=['year-month'])
df = df.join(forward_thirtysix_month_return_mad, on=['year-month'])
df = df.join(past_one_month_return_mad, on=['year-month'])
df = df.join(past_two_month_return_mad, on=['year-month'])
df = df.join(past_three_month_return_mad, on=['year-month'])
df = df.join(past_four_month_return_mad, on=['year-month'])
df = df.join(past_five_month_return_mad, on=['year-month'])
df = df.join(past_six_month_return_mad, on=['year-month'])
df = df.join(past_seven_month_return_mad, on=['year-month'])
df = df.join(past_eight_month_return_mad, on=['year-month'])
df = df.join(past_nine_month_return_mad, on=['year-month'])
df = df.join(past_ten_month_return_mad, on=['year-month'])
df = df.join(past_eleven_month_return_mad, on=['year-month'])
df = df.join(past_twelve_month_return_mad, on=['year-month'])
df = df.join(past_thirteen_month_return_mad, on=['year-month'])
df = df.join(past_fourteen_month_return_mad, on=['year-month'])
df = df.join(past_fifteen_month_return_mad, on=['year-month'])
df = df.join(past_sixteen_month_return_mad, on=['year-month'])
df = df.join(past_seventeen_month_return_mad, on=['year-month'])
df = df.join(past_eighteen_month_return_mad, on=['year-month'])
df = df.join(past_nineteen_month_return_mad, on=['year-month'])
df = df.join(past_twenty_month_return_mad, on=['year-month'])
df = df.join(past_twentyone_month_return_mad, on=['year-month'])
df = df.join(past_twentytwo_month_return_mad, on=['year-month'])
df = df.join(past_twentythree_month_return_mad, on=['year-month'])
df = df.join(past_twentyfour_month_return_mad, on=['year-month'])
df = df.join(past_twentyfive_month_return_mad, on=['year-month'])
df = df.join(past_twentysix_month_return_mad, on=['year-month'])
df = df.join(past_twentyseven_month_return_mad, on=['year-month'])
df = df.join(past_twentyeight_month_return_mad, on=['year-month'])
df = df.join(past_twentynine_month_return_mad, on=['year-month'])
df = df.join(past_thirty_month_return_mad, on=['year-month'])
df = df.join(past_thirtyone_month_return_mad, on=['year-month'])
df = df.join(past_thirtytwo_month_return_mad, on=['year-month'])
df = df.join(past_thirtythree_month_return_mad, on=['year-month'])
df = df.join(past_thirtyfour_month_return_mad, on=['year-month'])
df = df.join(past_thirtyfive_month_return_mad, on=['year-month'])
df = df.join(past_thirtysix_month_return_mad, on=['year-month'])
df = df.join(accrual_mad, on=['year-month'])
df = df.join(adv_sale_mad, on=['year-month'])
df = df.join(aftret_eq_mad, on=['year-month'])
df = df.join(aftret_equity_mad, on=['year-month'])
df = df.join(aftret_invcapx_mad, on=['year-month'])
df = df.join(at_turn_mad, on=['year-month'])
df = df.join(bm_mad, on=['year-month'])
df = df.join(CAPEI_mad, on=['year-month'])
df = df.join(capital_ratio_mad, on=['year-month'])
df = df.join(cash_conversion_mad, on=['year-month'])
df = df.join(cash_debt_mad, on=['year-month'])
df = df.join(cash_lt_mad, on=['year-month'])
df = df.join(cash_ratio_mad, on=['year-month'])
df = df.join(cfm_mad, on=['year-month'])
df = df.join(curr_debt_mad, on=['year-month'])
df = df.join(curr_ratio_mad, on=['year-month'])
df = df.join(de_ratio_mad, on=['year-month'])
df = df.join(debt_assets_mad, on=['year-month'])
df = df.join(debt_at_mad, on=['year-month'])
df = df.join(debt_capital_mad, on=['year-month'])
df = df.join(debt_ebitda_mad, on=['year-month'])
df = df.join(debt_invcap_mad, on=['year-month'])
df = df.join(DIVYIELD_mad, on=['year-month'])
df = df.join(dltt_be_mad, on=['year-month'])
df = df.join(dpr_mad, on=['year-month'])
df = df.join(efftax_mad, on=['year-month'])
df = df.join(equity_invcap_mad, on=['year-month'])
df = df.join(evm_mad, on=['year-month'])
df = df.join(fcf_ocf_mad, on=['year-month'])
df = df.join(gpm_mad, on=['year-month'])
df = df.join(GProf_mad, on=['year-month'])
df = df.join(int_debt_mad, on=['year-month'])
df = df.join(int_totdebt_mad, on=['year-month'])
df = df.join(intcov_mad, on=['year-month'])
df = df.join(intcov_ratio_mad, on=['year-month'])
df = df.join(inv_turn_mad, on=['year-month'])
df = df.join(invt_act_mad, on=['year-month'])
df = df.join(lt_debt_mad, on=['year-month'])
df = df.join(lt_ppent_mad, on=['year-month'])
df = df.join(npm_mad, on=['year-month'])
df = df.join(ocf_lct_mad, on=['year-month'])
df = df.join(opmad_mad, on=['year-month'])
df = df.join(opmbd_mad, on=['year-month'])
df = df.join(pay_turn_mad, on=['year-month'])
df = df.join(pcf_mad, on=['year-month'])
df = df.join(pe_exi_mad, on=['year-month'])
df = df.join(pe_inc_mad, on=['year-month'])
df = df.join(pe_op_basic_mad, on=['year-month'])
df = df.join(pe_op_dil_mad, on=['year-month'])
df = df.join(PEG_1yrforward_mad, on=['year-month'])
df = df.join(PEG_ltgforward_mad, on=['year-month'])
df = df.join(PEG_trailing_mad, on=['year-month'])
df = df.join(pretret_earnat_mad, on=['year-month'])
df = df.join(pretret_noa_mad, on=['year-month'])
df = df.join(profit_lct_mad, on=['year-month'])
df = df.join(ps_mad, on=['year-month'])
df = df.join(ptb_mad, on=['year-month'])
df = df.join(ptpm_mad, on=['year-month'])
df = df.join(quick_ratio_mad, on=['year-month'])
df = df.join(rd_sale_mad, on=['year-month'])
df = df.join(rect_act_mad, on=['year-month'])
df = df.join(rect_turn_mad, on=['year-month'])
df = df.join(roa_mad, on=['year-month'])
df = df.join(roce_mad, on=['year-month'])
df = df.join(roe_mad, on=['year-month'])
df = df.join(sale_equity_mad, on=['year-month'])
df = df.join(sale_invcap_mad, on=['year-month'])
df = df.join(sale_nwc_mad, on=['year-month'])
df = df.join(short_debt_mad, on=['year-month'])
df = df.join(staff_sale_mad, on=['year-month'])
df = df.join(totdebt_invcap_mad, on=['year-month'])
df = df.join(dvpspm_mad, on=['year-month'])
df = df.join(dvpsxm_mad, on=['year-month'])
df = df.join(dvrate_mad, on=['year-month'])
df = df.join(spcsrc_mad, on=['year-month'])
df = df.join(alpha_mad, on=['year-month'])
df = df.join(b_hml_mad, on=['year-month'])
df = df.join(b_mkt_mad, on=['year-month'])
df = df.join(b_smb_mad, on=['year-month'])
df = df.join(b_umd_mad, on=['year-month'])
df = df.join(exret_mad, on=['year-month'])
df = df.join(ivol_mad, on=['year-month'])
df = df.join(n_mad, on=['year-month'])
df = df.join(R2_mad, on=['year-month'])
df = df.join(tvol_mad, on=['year-month'])
df = df.join(BUYPCT_mad, on=['year-month'])
df = df.join(HOLDPCT_mad, on=['year-month'])
df = df.join(MEANREC_mad, on=['year-month'])
df = df.join(MEDREC_mad, on=['year-month'])
df = df.join(recup_mad, on=['year-month'])
df = df.join(recdown_mad, on=['year-month'])
df = df.join(SELLPCT_mad, on=['year-month'])
df = df.join(STDEV_mad, on=['year-month'])

In [66]:
# Sector MAD Variable - Enirch - All
# Z-score denominator for securities by month, use mean absolute deviation in place of median absolute deviation is MAD = 0
# Adjustments to replicate standard deviation value included in fuction (meanad and mad)
if df.groupby(['year-month', 'sector'])[['forward_one_month_return']].apply(mad).any() == 0:
    forward_one_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_one_month_return']].apply(meanad)
else:
    forward_one_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_one_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_two_month_return']].apply(mad).any() == 0:
    forward_two_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_two_month_return']].apply(meanad)
else:
    forward_two_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_two_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_three_month_return']].apply(mad).any() == 0:
    forward_three_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_three_month_return']].apply(meanad)
else:
    forward_three_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_three_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_four_month_return']].apply(mad).any() == 0:
    forward_four_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_four_month_return']].apply(meanad)
else:
    forward_four_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_four_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_five_month_return']].apply(mad).any() == 0:
    forward_five_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_five_month_return']].apply(meanad)
else:
    forward_five_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_five_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_six_month_return']].apply(mad).any() == 0:
    forward_six_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_six_month_return']].apply(meanad)
else:
    forward_six_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_six_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_seven_month_return']].apply(mad).any() == 0:
    forward_seven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_seven_month_return']].apply(meanad)
else:
    forward_seven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_seven_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_eight_month_return']].apply(mad).any() == 0:
    forward_eight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_eight_month_return']].apply(meanad)
else:
    forward_eight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_eight_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_nine_month_return']].apply(mad).any() == 0:
    forward_nine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_nine_month_return']].apply(meanad)
else:
    forward_nine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_nine_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_ten_month_return']].apply(mad).any() == 0:
    forward_ten_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_ten_month_return']].apply(meanad)
else:
    forward_ten_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_ten_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_eleven_month_return']].apply(mad).any() == 0:
    forward_eleven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_eleven_month_return']].apply(meanad)
else:
    forward_eleven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_eleven_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twelve_month_return']].apply(mad).any() == 0:
    forward_twelve_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twelve_month_return']].apply(meanad)
else:
    forward_twelve_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twelve_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirteen_month_return']].apply(mad).any() == 0:
    forward_thirteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirteen_month_return']].apply(meanad)
else:
    forward_thirteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_fourteen_month_return']].apply(mad).any() == 0:
    forward_fourteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_fourteen_month_return']].apply(meanad)
else:
    forward_fourteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_fourteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_fifteen_month_return']].apply(mad).any() == 0:
    forward_fifteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_fifteen_month_return']].apply(meanad)
else:
    forward_fifteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_fifteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_sixteen_month_return']].apply(mad).any() == 0:
    forward_sixteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_sixteen_month_return']].apply(meanad)
else:
    forward_sixteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_sixteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_seventeen_month_return']].apply(mad).any() == 0:
    forward_seventeen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_seventeen_month_return']].apply(meanad)
else:
    forward_seventeen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_seventeen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_eighteen_month_return']].apply(mad).any() == 0:
    forward_eighteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_eighteen_month_return']].apply(meanad)
else:
    forward_eighteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_eighteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_nineteen_month_return']].apply(mad).any() == 0:
    forward_nineteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_nineteen_month_return']].apply(meanad)
else:
    forward_nineteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_nineteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twenty_month_return']].apply(mad).any() == 0:
    forward_twenty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twenty_month_return']].apply(meanad)
else:
    forward_twenty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twenty_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentyone_month_return']].apply(mad).any() == 0:
    forward_twentyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyone_month_return']].apply(meanad)
else:
    forward_twentyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyone_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentytwo_month_return']].apply(mad).any() == 0:
    forward_twentytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentytwo_month_return']].apply(meanad)
else:
    forward_twentytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentytwo_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentythree_month_return']].apply(mad).any() == 0:
    forward_twentythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentythree_month_return']].apply(meanad)
else:
    forward_twentythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentythree_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentyfour_month_return']].apply(mad).any() == 0:
    forward_twentyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyfour_month_return']].apply(meanad)
else:
    forward_twentyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyfour_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentyfive_month_return']].apply(mad).any() == 0:
    forward_twentyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyfive_month_return']].apply(meanad)
else:
    forward_twentyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyfive_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentysix_month_return']].apply(mad).any() == 0:
    forward_twentysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentysix_month_return']].apply(meanad)
else:
    forward_twentysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentysix_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentyseven_month_return']].apply(mad).any() == 0:
    forward_twentyseven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyseven_month_return']].apply(meanad)
else:
    forward_twentyseven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyseven_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentyeight_month_return']].apply(mad).any() == 0:
    forward_twentyeight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyeight_month_return']].apply(meanad)
else:
    forward_twentyeight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentyeight_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_twentynine_month_return']].apply(mad).any() == 0:
    forward_twentynine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentynine_month_return']].apply(meanad)
else:
    forward_twentynine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_twentynine_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirty_month_return']].apply(mad).any() == 0:
    forward_thirty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirty_month_return']].apply(meanad)
else:
    forward_thirty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirty_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirtyone_month_return']].apply(mad).any() == 0:
    forward_thirtyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtyone_month_return']].apply(meanad)
else:
    forward_thirtyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtyone_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirtytwo_month_return']].apply(mad).any() == 0:
    forward_thirtytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtytwo_month_return']].apply(meanad)
else:
    forward_thirtytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtytwo_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirtythree_month_return']].apply(mad).any() == 0:
    forward_thirtythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtythree_month_return']].apply(meanad)
else:
    forward_thirtythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtythree_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirtyfour_month_return']].apply(mad).any() == 0:
    forward_thirtyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtyfour_month_return']].apply(meanad)
else:
    forward_thirtyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtyfour_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirtyfive_month_return']].apply(mad).any() == 0:
    forward_thirtyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtyfive_month_return']].apply(meanad)
else:
    forward_thirtyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtyfive_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['forward_thirtysix_month_return']].apply(mad).any() == 0:
    forward_thirtysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtysix_month_return']].apply(meanad)
else:
    forward_thirtysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['forward_thirtysix_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_one_month_return']].apply(mad).any() == 0:
    past_one_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_one_month_return']].apply(meanad)
else:
    past_one_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_one_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_two_month_return']].apply(mad).any() == 0:
    past_two_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_two_month_return']].apply(meanad)
else:
    past_two_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_two_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_three_month_return']].apply(mad).any() == 0:
    past_three_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_three_month_return']].apply(meanad)
else:
    past_three_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_three_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_four_month_return']].apply(mad).any() == 0:
    past_four_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_four_month_return']].apply(meanad)
else:
    past_four_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_four_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_five_month_return']].apply(mad).any() == 0:
    past_five_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_five_month_return']].apply(meanad)
else:
    past_five_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_five_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_six_month_return']].apply(mad).any() == 0:
    past_six_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_six_month_return']].apply(meanad)
else:
    past_six_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_six_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_seven_month_return']].apply(mad).any() == 0:
    past_seven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_seven_month_return']].apply(meanad)
else:
    past_seven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_seven_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_eight_month_return']].apply(mad).any() == 0:
    past_eight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_eight_month_return']].apply(meanad)
else:
    past_eight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_eight_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_nine_month_return']].apply(mad).any() == 0:
    past_nine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_nine_month_return']].apply(meanad)
else:
    past_nine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_nine_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_ten_month_return']].apply(mad).any() == 0:
    past_ten_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_ten_month_return']].apply(meanad)
else:
    past_ten_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_ten_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_eleven_month_return']].apply(mad).any() == 0:
    past_eleven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_eleven_month_return']].apply(meanad)
else:
    past_eleven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_eleven_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twelve_month_return']].apply(mad).any() == 0:
    past_twelve_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twelve_month_return']].apply(meanad)
else:
    past_twelve_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twelve_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirteen_month_return']].apply(mad).any() == 0:
    past_thirteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirteen_month_return']].apply(meanad)
else:
    past_thirteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_fourteen_month_return']].apply(mad).any() == 0:
    past_fourteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_fourteen_month_return']].apply(meanad)
else:
    past_fourteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_fourteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_fifteen_month_return']].apply(mad).any() == 0:
    past_fifteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_fifteen_month_return']].apply(meanad)
else:
    past_fifteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_fifteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_sixteen_month_return']].apply(mad).any() == 0:
    past_sixteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_sixteen_month_return']].apply(meanad)
else:
    past_sixteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_sixteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_seventeen_month_return']].apply(mad).any() == 0:
    past_seventeen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_seventeen_month_return']].apply(meanad)
else:
    past_seventeen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_seventeen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_eighteen_month_return']].apply(mad).any() == 0:
    past_eighteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_eighteen_month_return']].apply(meanad)
else:
    past_eighteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_eighteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_nineteen_month_return']].apply(mad).any() == 0:
    past_nineteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_nineteen_month_return']].apply(meanad)
else:
    past_nineteen_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_nineteen_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twenty_month_return']].apply(mad).any() == 0:
    past_twenty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twenty_month_return']].apply(meanad)
else:
    past_twenty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twenty_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentyone_month_return']].apply(mad).any() == 0:
    past_twentyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyone_month_return']].apply(meanad)
else:
    past_twentyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyone_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentytwo_month_return']].apply(mad).any() == 0:
    past_twentytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentytwo_month_return']].apply(meanad)
else:
    past_twentytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentytwo_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentythree_month_return']].apply(mad).any() == 0:
    past_twentythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentythree_month_return']].apply(meanad)
else:
    past_twentythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentythree_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentyfour_month_return']].apply(mad).any() == 0:
    past_twentyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyfour_month_return']].apply(meanad)
else:
    past_twentyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyfour_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentyfive_month_return']].apply(mad).any() == 0:
    past_twentyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyfive_month_return']].apply(meanad)
else:
    past_twentyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyfive_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentysix_month_return']].apply(mad).any() == 0:
    past_twentysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentysix_month_return']].apply(meanad)
else:
    past_twentysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentysix_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentyseven_month_return']].apply(mad).any() == 0:
    past_twentyseven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyseven_month_return']].apply(meanad)
else:
    past_twentyseven_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyseven_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentyeight_month_return']].apply(mad).any() == 0:
    past_twentyeight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyeight_month_return']].apply(meanad)
else:
    past_twentyeight_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentyeight_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_twentynine_month_return']].apply(mad).any() == 0:
    past_twentynine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentynine_month_return']].apply(meanad)
else:
    past_twentynine_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_twentynine_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirty_month_return']].apply(mad).any() == 0:
    past_thirty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirty_month_return']].apply(meanad)
else:
    past_thirty_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirty_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirtyone_month_return']].apply(mad).any() == 0:
    past_thirtyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtyone_month_return']].apply(meanad)
else:
    past_thirtyone_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtyone_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirtytwo_month_return']].apply(mad).any() == 0:
    past_thirtytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtytwo_month_return']].apply(meanad)
else:
    past_thirtytwo_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtytwo_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirtythree_month_return']].apply(mad).any() == 0:
    past_thirtythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtythree_month_return']].apply(meanad)
else:
    past_thirtythree_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtythree_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirtyfour_month_return']].apply(mad).any() == 0:
    past_thirtyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtyfour_month_return']].apply(meanad)
else:
    past_thirtyfour_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtyfour_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirtyfive_month_return']].apply(mad).any() == 0:
    past_thirtyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtyfive_month_return']].apply(meanad)
else:
    past_thirtyfive_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtyfive_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['past_thirtysix_month_return']].apply(mad).any() == 0:
    past_thirtysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtysix_month_return']].apply(meanad)
else:
    past_thirtysix_month_return_sector_mad = df.groupby(['year-month', 'sector'])[['past_thirtysix_month_return']].apply(mad)
if df.groupby(['year-month', 'sector'])[['accrual']].apply(mad).any() == 0:
    accrual_sector_mad = df.groupby(['year-month', 'sector'])[['accrual']].apply(meanad)
else:
    accrual_sector_mad = df.groupby(['year-month', 'sector'])[['accrual']].apply(mad)
if df.groupby(['year-month', 'sector'])[['adv_sale']].apply(mad).any() == 0:
    adv_sale_sector_mad = df.groupby(['year-month', 'sector'])[['adv_sale']].apply(meanad)
else:
    adv_sale_sector_mad = df.groupby(['year-month', 'sector'])[['adv_sale']].apply(mad)
if df.groupby(['year-month', 'sector'])[['aftret_eq']].apply(mad).any() == 0:
    aftret_eq_sector_mad = df.groupby(['year-month', 'sector'])[['aftret_eq']].apply(meanad)
else:
    aftret_eq_sector_mad = df.groupby(['year-month', 'sector'])[['aftret_eq']].apply(mad)
if df.groupby(['year-month', 'sector'])[['aftret_equity']].apply(mad).any() == 0:
    aftret_equity_sector_mad = df.groupby(['year-month', 'sector'])[['aftret_equity']].apply(meanad)
else:
    aftret_equity_sector_mad = df.groupby(['year-month', 'sector'])[['aftret_equity']].apply(mad)
if df.groupby(['year-month', 'sector'])[['aftret_invcapx']].apply(mad).any() == 0:
    aftret_invcapx_sector_mad = df.groupby(['year-month', 'sector'])[['aftret_invcapx']].apply(meanad)
else:
    aftret_invcapx_sector_mad = df.groupby(['year-month', 'sector'])[['aftret_invcapx']].apply(mad)
if df.groupby(['year-month', 'sector'])[['at_turn']].apply(mad).any() == 0:
    at_turn_sector_mad = df.groupby(['year-month', 'sector'])[['at_turn']].apply(meanad)
else:
    at_turn_sector_mad = df.groupby(['year-month', 'sector'])[['at_turn']].apply(mad)
if df.groupby(['year-month', 'sector'])[['bm']].apply(mad).any() == 0:
    bm_sector_mad = df.groupby(['year-month', 'sector'])[['bm']].apply(meanad)
else:
    bm_sector_mad = df.groupby(['year-month', 'sector'])[['bm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['CAPEI']].apply(mad).any() == 0:
    CAPEI_sector_mad = df.groupby(['year-month', 'sector'])[['CAPEI']].apply(meanad)
else:
    CAPEI_sector_mad = df.groupby(['year-month', 'sector'])[['CAPEI']].apply(mad)
if df.groupby(['year-month', 'sector'])[['capital_ratio']].apply(mad).any() == 0:
    capital_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['capital_ratio']].apply(meanad)
else:
    capital_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['capital_ratio']].apply(mad)
if df.groupby(['year-month', 'sector'])[['cash_conversion']].apply(mad).any() == 0:
    cash_conversion_sector_mad = df.groupby(['year-month', 'sector'])[['cash_conversion']].apply(meanad)
else:
    cash_conversion_sector_mad = df.groupby(['year-month', 'sector'])[['cash_conversion']].apply(mad)
if df.groupby(['year-month', 'sector'])[['cash_debt']].apply(mad).any() == 0:
    cash_debt_sector_mad = df.groupby(['year-month', 'sector'])[['cash_debt']].apply(meanad)
else:
    cash_debt_sector_mad = df.groupby(['year-month', 'sector'])[['cash_debt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['cash_lt']].apply(mad).any() == 0:
    cash_lt_sector_mad = df.groupby(['year-month', 'sector'])[['cash_lt']].apply(meanad)
else:
    cash_lt_sector_mad = df.groupby(['year-month', 'sector'])[['cash_lt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['cash_ratio']].apply(mad).any() == 0:
    cash_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['cash_ratio']].apply(meanad)
else:
    cash_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['cash_ratio']].apply(mad)
if df.groupby(['year-month', 'sector'])[['cfm']].apply(mad).any() == 0:
    cfm_sector_mad = df.groupby(['year-month', 'sector'])[['cfm']].apply(meanad)
else:
    cfm_sector_mad = df.groupby(['year-month', 'sector'])[['cfm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['curr_debt']].apply(mad).any() == 0:
    curr_debt_sector_mad = df.groupby(['year-month', 'sector'])[['curr_debt']].apply(meanad)
else:
    curr_debt_sector_mad = df.groupby(['year-month', 'sector'])[['curr_debt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['curr_ratio']].apply(mad).any() == 0:
    curr_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['curr_ratio']].apply(meanad)
else:
    curr_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['curr_ratio']].apply(mad)
if df.groupby(['year-month', 'sector'])[['de_ratio']].apply(mad).any() == 0:
    de_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['de_ratio']].apply(meanad)
else:
    de_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['de_ratio']].apply(mad)
if df.groupby(['year-month', 'sector'])[['debt_assets']].apply(mad).any() == 0:
    debt_assets_sector_mad = df.groupby(['year-month', 'sector'])[['debt_assets']].apply(meanad)
else:
    debt_assets_sector_mad = df.groupby(['year-month', 'sector'])[['debt_assets']].apply(mad)
if df.groupby(['year-month', 'sector'])[['debt_at']].apply(mad).any() == 0:
    debt_at_sector_mad = df.groupby(['year-month', 'sector'])[['debt_at']].apply(meanad)
else:
    debt_at_sector_mad = df.groupby(['year-month', 'sector'])[['debt_at']].apply(mad)
if df.groupby(['year-month', 'sector'])[['debt_capital']].apply(mad).any() == 0:
    debt_capital_sector_mad = df.groupby(['year-month', 'sector'])[['debt_capital']].apply(meanad)
else:
    debt_capital_sector_mad = df.groupby(['year-month', 'sector'])[['debt_capital']].apply(mad)
if df.groupby(['year-month', 'sector'])[['debt_ebitda']].apply(mad).any() == 0:
    debt_ebitda_sector_mad = df.groupby(['year-month', 'sector'])[['debt_ebitda']].apply(meanad)
else:
    debt_ebitda_sector_mad = df.groupby(['year-month', 'sector'])[['debt_ebitda']].apply(mad)
if df.groupby(['year-month', 'sector'])[['debt_invcap']].apply(mad).any() == 0:
    debt_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['debt_invcap']].apply(meanad)
else:
    debt_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['debt_invcap']].apply(mad)
if df.groupby(['year-month', 'sector'])[['DIVYIELD']].apply(mad).any() == 0:
    DIVYIELD_sector_mad = df.groupby(['year-month', 'sector'])[['DIVYIELD']].apply(meanad)
else:
    DIVYIELD_sector_mad = df.groupby(['year-month', 'sector'])[['DIVYIELD']].apply(mad)
if df.groupby(['year-month', 'sector'])[['dltt_be']].apply(mad).any() == 0:
    dltt_be_sector_mad = df.groupby(['year-month', 'sector'])[['dltt_be']].apply(meanad)
else:
    dltt_be_sector_mad = df.groupby(['year-month', 'sector'])[['dltt_be']].apply(mad)
if df.groupby(['year-month', 'sector'])[['dpr']].apply(mad).any() == 0:
    dpr_sector_mad = df.groupby(['year-month', 'sector'])[['dpr']].apply(meanad)
else:
    dpr_sector_mad = df.groupby(['year-month', 'sector'])[['dpr']].apply(mad)
if df.groupby(['year-month', 'sector'])[['efftax']].apply(mad).any() == 0:
    efftax_sector_mad = df.groupby(['year-month', 'sector'])[['efftax']].apply(meanad)
else:
    efftax_sector_mad = df.groupby(['year-month', 'sector'])[['efftax']].apply(mad)
if df.groupby(['year-month', 'sector'])[['equity_invcap']].apply(mad).any() == 0:
    equity_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['equity_invcap']].apply(meanad)
else:
    equity_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['equity_invcap']].apply(mad)
if df.groupby(['year-month', 'sector'])[['evm']].apply(mad).any() == 0:
    evm_sector_mad = df.groupby(['year-month', 'sector'])[['evm']].apply(meanad)
else:
    evm_sector_mad = df.groupby(['year-month', 'sector'])[['evm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['fcf_ocf']].apply(mad).any() == 0:
    fcf_ocf_sector_mad = df.groupby(['year-month', 'sector'])[['fcf_ocf']].apply(meanad)
else:
    fcf_ocf_sector_mad = df.groupby(['year-month', 'sector'])[['fcf_ocf']].apply(mad)
if df.groupby(['year-month', 'sector'])[['gpm']].apply(mad).any() == 0:
    gpm_sector_mad = df.groupby(['year-month', 'sector'])[['gpm']].apply(meanad)
else:
    gpm_sector_mad = df.groupby(['year-month', 'sector'])[['gpm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['GProf']].apply(mad).any() == 0:
    GProf_sector_mad = df.groupby(['year-month', 'sector'])[['GProf']].apply(meanad)
else:
    GProf_sector_mad = df.groupby(['year-month', 'sector'])[['GProf']].apply(mad)
if df.groupby(['year-month', 'sector'])[['int_debt']].apply(mad).any() == 0:
    int_debt_sector_mad = df.groupby(['year-month', 'sector'])[['int_debt']].apply(meanad)
else:
    int_debt_sector_mad = df.groupby(['year-month', 'sector'])[['int_debt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['int_totdebt']].apply(mad).any() == 0:
    int_totdebt_sector_mad = df.groupby(['year-month', 'sector'])[['int_totdebt']].apply(meanad)
else:
    int_totdebt_sector_mad = df.groupby(['year-month', 'sector'])[['int_totdebt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['intcov']].apply(mad).any() == 0:
    intcov_sector_mad = df.groupby(['year-month', 'sector'])[['intcov']].apply(meanad)
else:
    intcov_sector_mad = df.groupby(['year-month', 'sector'])[['intcov']].apply(mad)
if df.groupby(['year-month', 'sector'])[['intcov_ratio']].apply(mad).any() == 0:
    intcov_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['intcov_ratio']].apply(meanad)
else:
    intcov_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['intcov_ratio']].apply(mad)
if df.groupby(['year-month', 'sector'])[['inv_turn']].apply(mad).any() == 0:
    inv_turn_sector_mad = df.groupby(['year-month', 'sector'])[['inv_turn']].apply(meanad)
else:
    inv_turn_sector_mad = df.groupby(['year-month', 'sector'])[['inv_turn']].apply(mad)
if df.groupby(['year-month', 'sector'])[['invt_act']].apply(mad).any() == 0:
    invt_act_sector_mad = df.groupby(['year-month', 'sector'])[['invt_act']].apply(meanad)
else:
    invt_act_sector_mad = df.groupby(['year-month', 'sector'])[['invt_act']].apply(mad)
if df.groupby(['year-month', 'sector'])[['lt_debt']].apply(mad).any() == 0:
    lt_debt_sector_mad = df.groupby(['year-month', 'sector'])[['lt_debt']].apply(meanad)
else:
    lt_debt_sector_mad = df.groupby(['year-month', 'sector'])[['lt_debt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['lt_ppent']].apply(mad).any() == 0:
    lt_ppent_sector_mad = df.groupby(['year-month', 'sector'])[['lt_ppent']].apply(meanad)
else:
    lt_ppent_sector_mad = df.groupby(['year-month', 'sector'])[['lt_ppent']].apply(mad)
if df.groupby(['year-month', 'sector'])[['npm']].apply(mad).any() == 0:
    npm_sector_mad = df.groupby(['year-month', 'sector'])[['npm']].apply(meanad)
else:
    npm_sector_mad = df.groupby(['year-month', 'sector'])[['npm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['ocf_lct']].apply(mad).any() == 0:
    ocf_lct_sector_mad = df.groupby(['year-month', 'sector'])[['ocf_lct']].apply(meanad)
else:
    ocf_lct_sector_mad = df.groupby(['year-month', 'sector'])[['ocf_lct']].apply(mad)
if df.groupby(['year-month', 'sector'])[['opmad']].apply(mad).any() == 0:
    opmad_sector_mad = df.groupby(['year-month', 'sector'])[['opmad']].apply(meanad)
else:
    opmad_sector_mad = df.groupby(['year-month', 'sector'])[['opmad']].apply(mad)
if df.groupby(['year-month', 'sector'])[['opmbd']].apply(mad).any() == 0:
    opmbd_sector_mad = df.groupby(['year-month', 'sector'])[['opmbd']].apply(meanad)
else:
    opmbd_sector_mad = df.groupby(['year-month', 'sector'])[['opmbd']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pay_turn']].apply(mad).any() == 0:
    pay_turn_sector_mad = df.groupby(['year-month', 'sector'])[['pay_turn']].apply(meanad)
else:
    pay_turn_sector_mad = df.groupby(['year-month', 'sector'])[['pay_turn']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pcf']].apply(mad).any() == 0:
    pcf_sector_mad = df.groupby(['year-month', 'sector'])[['pcf']].apply(meanad)
else:
    pcf_sector_mad = df.groupby(['year-month', 'sector'])[['pcf']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pe_exi']].apply(mad).any() == 0:
    pe_exi_sector_mad = df.groupby(['year-month', 'sector'])[['pe_exi']].apply(meanad)
else:
    pe_exi_sector_mad = df.groupby(['year-month', 'sector'])[['pe_exi']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pe_inc']].apply(mad).any() == 0:
    pe_inc_sector_mad = df.groupby(['year-month', 'sector'])[['pe_inc']].apply(meanad)
else:
    pe_inc_sector_mad = df.groupby(['year-month', 'sector'])[['pe_inc']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pe_op_basic']].apply(mad).any() == 0:
    pe_op_basic_sector_mad = df.groupby(['year-month', 'sector'])[['pe_op_basic']].apply(meanad)
else:
    pe_op_basic_sector_mad = df.groupby(['year-month', 'sector'])[['pe_op_basic']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pe_op_dil']].apply(mad).any() == 0:
    pe_op_dil_sector_mad = df.groupby(['year-month', 'sector'])[['pe_op_dil']].apply(meanad)
else:
    pe_op_dil_sector_mad = df.groupby(['year-month', 'sector'])[['pe_op_dil']].apply(mad)
if df.groupby(['year-month', 'sector'])[['PEG_1yrforward']].apply(mad).any() == 0:
    PEG_1yrforward_sector_mad = df.groupby(['year-month', 'sector'])[['PEG_1yrforward']].apply(meanad)
else:
    PEG_1yrforward_sector_mad = df.groupby(['year-month', 'sector'])[['PEG_1yrforward']].apply(mad)
if df.groupby(['year-month', 'sector'])[['PEG_ltgforward']].apply(mad).any() == 0:
    PEG_ltgforward_sector_mad = df.groupby(['year-month', 'sector'])[['PEG_ltgforward']].apply(meanad)
else:
    PEG_ltgforward_sector_mad = df.groupby(['year-month', 'sector'])[['PEG_ltgforward']].apply(mad)
if df.groupby(['year-month', 'sector'])[['PEG_trailing']].apply(mad).any() == 0:
    PEG_trailing_sector_mad = df.groupby(['year-month', 'sector'])[['PEG_trailing']].apply(meanad)
else:
    PEG_trailing_sector_mad = df.groupby(['year-month', 'sector'])[['PEG_trailing']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pretret_earnat']].apply(mad).any() == 0:
    pretret_earnat_sector_mad = df.groupby(['year-month', 'sector'])[['pretret_earnat']].apply(meanad)
else:
    pretret_earnat_sector_mad = df.groupby(['year-month', 'sector'])[['pretret_earnat']].apply(mad)
if df.groupby(['year-month', 'sector'])[['pretret_noa']].apply(mad).any() == 0:
    pretret_noa_sector_mad = df.groupby(['year-month', 'sector'])[['pretret_noa']].apply(meanad)
else:
    pretret_noa_sector_mad = df.groupby(['year-month', 'sector'])[['pretret_noa']].apply(mad)
if df.groupby(['year-month', 'sector'])[['profit_lct']].apply(mad).any() == 0:
    profit_lct_sector_mad = df.groupby(['year-month', 'sector'])[['profit_lct']].apply(meanad)
else:
    profit_lct_sector_mad = df.groupby(['year-month', 'sector'])[['profit_lct']].apply(mad)
if df.groupby(['year-month', 'sector'])[['ps']].apply(mad).any() == 0:
    ps_sector_mad = df.groupby(['year-month', 'sector'])[['ps']].apply(meanad)
else:
    ps_sector_mad = df.groupby(['year-month', 'sector'])[['ps']].apply(mad)
if df.groupby(['year-month', 'sector'])[['ptb']].apply(mad).any() == 0:
    ptb_sector_mad = df.groupby(['year-month', 'sector'])[['ptb']].apply(meanad)
else:
    ptb_sector_mad = df.groupby(['year-month', 'sector'])[['ptb']].apply(mad)
if df.groupby(['year-month', 'sector'])[['ptpm']].apply(mad).any() == 0:
    ptpm_sector_mad = df.groupby(['year-month', 'sector'])[['ptpm']].apply(meanad)
else:
    ptpm_sector_mad = df.groupby(['year-month', 'sector'])[['ptpm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['quick_ratio']].apply(mad).any() == 0:
    quick_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['quick_ratio']].apply(meanad)
else:
    quick_ratio_sector_mad = df.groupby(['year-month', 'sector'])[['quick_ratio']].apply(mad)
if df.groupby(['year-month', 'sector'])[['rd_sale']].apply(mad).any() == 0:
    rd_sale_sector_mad = df.groupby(['year-month', 'sector'])[['rd_sale']].apply(meanad)
else:
    rd_sale_sector_mad = df.groupby(['year-month', 'sector'])[['rd_sale']].apply(mad)
if df.groupby(['year-month', 'sector'])[['rect_act']].apply(mad).any() == 0:
    rect_act_sector_mad = df.groupby(['year-month', 'sector'])[['rect_act']].apply(meanad)
else:
    rect_act_sector_mad = df.groupby(['year-month', 'sector'])[['rect_act']].apply(mad)
if df.groupby(['year-month', 'sector'])[['rect_turn']].apply(mad).any() == 0:
    rect_turn_sector_mad = df.groupby(['year-month', 'sector'])[['rect_turn']].apply(meanad)
else:
    rect_turn_sector_mad = df.groupby(['year-month', 'sector'])[['rect_turn']].apply(mad)
if df.groupby(['year-month', 'sector'])[['roa']].apply(mad).any() == 0:
    roa_sector_mad = df.groupby(['year-month', 'sector'])[['roa']].apply(meanad)
else:
    roa_sector_mad = df.groupby(['year-month', 'sector'])[['roa']].apply(mad)
if df.groupby(['year-month', 'sector'])[['roce']].apply(mad).any() == 0:
    roce_sector_mad = df.groupby(['year-month', 'sector'])[['roce']].apply(meanad)
else:
    roce_sector_mad = df.groupby(['year-month', 'sector'])[['roce']].apply(mad)
if df.groupby(['year-month', 'sector'])[['roe']].apply(mad).any() == 0:
    roe_sector_mad = df.groupby(['year-month', 'sector'])[['roe']].apply(meanad)
else:
    roe_sector_mad = df.groupby(['year-month', 'sector'])[['roe']].apply(mad)
if df.groupby(['year-month', 'sector'])[['sale_equity']].apply(mad).any() == 0:
    sale_equity_sector_mad = df.groupby(['year-month', 'sector'])[['sale_equity']].apply(meanad)
else:
    sale_equity_sector_mad = df.groupby(['year-month', 'sector'])[['sale_equity']].apply(mad)
if df.groupby(['year-month', 'sector'])[['sale_invcap']].apply(mad).any() == 0:
    sale_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['sale_invcap']].apply(meanad)
else:
    sale_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['sale_invcap']].apply(mad)
if df.groupby(['year-month', 'sector'])[['sale_nwc']].apply(mad).any() == 0:
    sale_nwc_sector_mad = df.groupby(['year-month', 'sector'])[['sale_nwc']].apply(meanad)
else:
    sale_nwc_sector_mad = df.groupby(['year-month', 'sector'])[['sale_nwc']].apply(mad)
if df.groupby(['year-month', 'sector'])[['short_debt']].apply(mad).any() == 0:
    short_debt_sector_mad = df.groupby(['year-month', 'sector'])[['short_debt']].apply(meanad)
else:
    short_debt_sector_mad = df.groupby(['year-month', 'sector'])[['short_debt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['staff_sale']].apply(mad).any() == 0:
    staff_sale_sector_mad = df.groupby(['year-month', 'sector'])[['staff_sale']].apply(meanad)
else:
    staff_sale_sector_mad = df.groupby(['year-month', 'sector'])[['staff_sale']].apply(mad)
if df.groupby(['year-month', 'sector'])[['totdebt_invcap']].apply(mad).any() == 0:
    totdebt_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['totdebt_invcap']].apply(meanad)
else:
    totdebt_invcap_sector_mad = df.groupby(['year-month', 'sector'])[['totdebt_invcap']].apply(mad)
if df.groupby(['year-month', 'sector'])[['dvpspm']].apply(mad).any() == 0:
    dvpspm_sector_mad = df.groupby(['year-month', 'sector'])[['dvpspm']].apply(meanad)
else:
    dvpspm_sector_mad = df.groupby(['year-month', 'sector'])[['dvpspm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['dvpsxm']].apply(mad).any() == 0:
    dvpsxm_sector_mad = df.groupby(['year-month', 'sector'])[['dvpsxm']].apply(meanad)
else:
    dvpsxm_sector_mad = df.groupby(['year-month', 'sector'])[['dvpsxm']].apply(mad)
if df.groupby(['year-month', 'sector'])[['dvrate']].apply(mad).any() == 0:
    dvrate_sector_mad = df.groupby(['year-month', 'sector'])[['dvrate']].apply(meanad)
else:
    dvrate_sector_mad = df.groupby(['year-month', 'sector'])[['dvrate']].apply(mad)
if df.groupby(['year-month', 'sector'])[['spcsrc']].apply(mad).any() == 0:
    spcsrc_sector_mad = df.groupby(['year-month', 'sector'])[['spcsrc']].apply(meanad)
else:
    spcsrc_sector_mad = df.groupby(['year-month', 'sector'])[['spcsrc']].apply(mad)
if df.groupby(['year-month', 'sector'])[['alpha']].apply(mad).any() == 0:
    alpha_sector_mad = df.groupby(['year-month', 'sector'])[['alpha']].apply(meanad)
else:
    alpha_sector_mad = df.groupby(['year-month', 'sector'])[['alpha']].apply(mad)
if df.groupby(['year-month', 'sector'])[['b_hml']].apply(mad).any() == 0:
    b_hml_sector_mad = df.groupby(['year-month', 'sector'])[['b_hml']].apply(meanad)
else:
    b_hml_sector_mad = df.groupby(['year-month', 'sector'])[['b_hml']].apply(mad)
if df.groupby(['year-month', 'sector'])[['b_mkt']].apply(mad).any() == 0:
    b_mkt_sector_mad = df.groupby(['year-month', 'sector'])[['b_mkt']].apply(meanad)
else:
    b_mkt_sector_mad = df.groupby(['year-month', 'sector'])[['b_mkt']].apply(mad)
if df.groupby(['year-month', 'sector'])[['b_smb']].apply(mad).any() == 0:
    b_smb_sector_mad = df.groupby(['year-month', 'sector'])[['b_smb']].apply(meanad)
else:
    b_smb_sector_mad = df.groupby(['year-month', 'sector'])[['b_smb']].apply(mad)
if df.groupby(['year-month', 'sector'])[['b_umd']].apply(mad).any() == 0:
    b_umd_sector_mad = df.groupby(['year-month', 'sector'])[['b_umd']].apply(meanad)
else:
    b_umd_sector_mad = df.groupby(['year-month', 'sector'])[['b_umd']].apply(mad)
if df.groupby(['year-month', 'sector'])[['exret']].apply(mad).any() == 0:
    exret_sector_mad = df.groupby(['year-month', 'sector'])[['exret']].apply(meanad)
else:
    exret_sector_mad = df.groupby(['year-month', 'sector'])[['exret']].apply(mad)
if df.groupby(['year-month', 'sector'])[['ivol']].apply(mad).any() == 0:
    ivol_sector_mad = df.groupby(['year-month', 'sector'])[['ivol']].apply(meanad)
else:
    ivol_sector_mad = df.groupby(['year-month', 'sector'])[['ivol']].apply(mad)
if df.groupby(['year-month', 'sector'])[['n']].apply(mad).any() == 0:
    n_sector_mad = df.groupby(['year-month', 'sector'])[['n']].apply(meanad)
else:
    n_sector_mad = df.groupby(['year-month', 'sector'])[['n']].apply(mad)
if df.groupby(['year-month', 'sector'])[['R2']].apply(mad).any() == 0:
    R2_sector_mad = df.groupby(['year-month', 'sector'])[['R2']].apply(meanad)
else:
    R2_sector_mad = df.groupby(['year-month', 'sector'])[['R2']].apply(mad)
if df.groupby(['year-month', 'sector'])[['tvol']].apply(mad).any() == 0:
    tvol_sector_mad = df.groupby(['year-month', 'sector'])[['tvol']].apply(meanad)
else:
    tvol_sector_mad = df.groupby(['year-month', 'sector'])[['tvol']].apply(mad)
if df.groupby(['year-month', 'sector'])[['BUYPCT']].apply(mad).any() == 0:
    BUYPCT_sector_mad = df.groupby(['year-month', 'sector'])[['BUYPCT']].apply(meanad)
else:
    BUYPCT_sector_mad = df.groupby(['year-month', 'sector'])[['BUYPCT']].apply(mad)
if df.groupby(['year-month', 'sector'])[['HOLDPCT']].apply(mad).any() == 0:
    HOLDPCT_sector_mad = df.groupby(['year-month', 'sector'])[['HOLDPCT']].apply(meanad)
else:
    HOLDPCT_sector_mad = df.groupby(['year-month', 'sector'])[['HOLDPCT']].apply(mad)
if df.groupby(['year-month', 'sector'])[['MEANREC']].apply(mad).any() == 0:
    MEANREC_sector_mad = df.groupby(['year-month', 'sector'])[['MEANREC']].apply(meanad)
else:
    MEANREC_sector_mad = df.groupby(['year-month', 'sector'])[['MEANREC']].apply(mad)
if df.groupby(['year-month', 'sector'])[['MEDREC']].apply(mad).any() == 0:
    MEDREC_sector_mad = df.groupby(['year-month', 'sector'])[['MEDREC']].apply(meanad)
else:
    MEDREC_sector_mad = df.groupby(['year-month', 'sector'])[['MEDREC']].apply(mad)
if df.groupby(['year-month', 'sector'])[['recup']].apply(mad).any() == 0:
    recup_sector_mad = df.groupby(['year-month', 'sector'])[['recup']].apply(meanad)
else:
    recup_sector_mad = df.groupby(['year-month', 'sector'])[['recup']].apply(mad)
if df.groupby(['year-month', 'sector'])[['recdown']].apply(mad).any() == 0:
    recdown_sector_mad = df.groupby(['year-month', 'sector'])[['recdown']].apply(meanad)
else:
    recdown_sector_mad = df.groupby(['year-month', 'sector'])[['recdown']].apply(mad)
if df.groupby(['year-month', 'sector'])[['SELLPCT']].apply(mad).any() == 0:
    SELLPCT_sector_mad = df.groupby(['year-month', 'sector'])[['SELLPCT']].apply(meanad)
else:
    SELLPCT_sector_mad = df.groupby(['year-month', 'sector'])[['SELLPCT']].apply(mad)
if df.groupby(['year-month', 'sector'])[['STDEV']].apply(mad).any() == 0:
    STDEV_sector_mad = df.groupby(['year-month', 'sector'])[['STDEV']].apply(meanad)
else:
    STDEV_sector_mad = df.groupby(['year-month', 'sector'])[['STDEV']].apply(mad)

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:717: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)
C:\Users\

In [67]:
# Sector MAD Datasets - Enrich - All
forward_one_month_return_sector_mad.name = 'forward_one_month_return_sector_mad'
forward_two_month_return_sector_mad.name = 'forward_two_month_return_sector_mad'
forward_three_month_return_sector_mad.name = 'forward_three_month_return_sector_mad'
forward_four_month_return_sector_mad.name = 'forward_four_month_return_sector_mad'
forward_five_month_return_sector_mad.name = 'forward_five_month_return_sector_mad'
forward_six_month_return_sector_mad.name = 'forward_six_month_return_sector_mad'
forward_seven_month_return_sector_mad.name = 'forward_seven_month_return_sector_mad'
forward_eight_month_return_sector_mad.name = 'forward_eight_month_return_sector_mad'
forward_nine_month_return_sector_mad.name = 'forward_nine_month_return_sector_mad'
forward_ten_month_return_sector_mad.name = 'forward_ten_month_return_sector_mad'
forward_eleven_month_return_sector_mad.name = 'forward_eleven_month_return_sector_mad'
forward_twelve_month_return_sector_mad.name = 'forward_twelve_month_return_sector_mad'
forward_thirteen_month_return_sector_mad.name = 'forward_thirteen_month_return_sector_mad'
forward_fourteen_month_return_sector_mad.name = 'forward_fourteen_month_return_sector_mad'
forward_fifteen_month_return_sector_mad.name = 'forward_fifteen_month_return_sector_mad'
forward_sixteen_month_return_sector_mad.name = 'forward_sixteen_month_return_sector_mad'
forward_seventeen_month_return_sector_mad.name = 'forward_seventeen_month_return_sector_mad'
forward_eighteen_month_return_sector_mad.name = 'forward_eighteen_month_return_sector_mad'
forward_nineteen_month_return_sector_mad.name = 'forward_nineteen_month_return_sector_mad'
forward_twenty_month_return_sector_mad.name = 'forward_twenty_month_return_sector_mad'
forward_twentyone_month_return_sector_mad.name = 'forward_twentyone_month_return_sector_mad'
forward_twentytwo_month_return_sector_mad.name = 'forward_twentytwo_month_return_sector_mad'
forward_twentythree_month_return_sector_mad.name = 'forward_twentythree_month_return_sector_mad'
forward_twentyfour_month_return_sector_mad.name = 'forward_twentyfour_month_return_sector_mad'
forward_twentyfive_month_return_sector_mad.name = 'forward_twentyfive_month_return_sector_mad'
forward_twentysix_month_return_sector_mad.name = 'forward_twentysix_month_return_sector_mad'
forward_twentyseven_month_return_sector_mad.name = 'forward_twentyseven_month_return_sector_mad'
forward_twentyeight_month_return_sector_mad.name = 'forward_twentyeight_month_return_sector_mad'
forward_twentynine_month_return_sector_mad.name = 'forward_twentynine_month_return_sector_mad'
forward_thirty_month_return_sector_mad.name = 'forward_thirty_month_return_sector_mad'
forward_thirtyone_month_return_sector_mad.name = 'forward_thirtyone_month_return_sector_mad'
forward_thirtytwo_month_return_sector_mad.name = 'forward_thirtytwo_month_return_sector_mad'
forward_thirtythree_month_return_sector_mad.name = 'forward_thirtythree_month_return_sector_mad'
forward_thirtyfour_month_return_sector_mad.name = 'forward_thirtyfour_month_return_sector_mad'
forward_thirtyfive_month_return_sector_mad.name = 'forward_thirtyfive_month_return_sector_mad'
forward_thirtysix_month_return_sector_mad.name = 'forward_thirtysix_month_return_sector_mad'
past_one_month_return_sector_mad.name = 'past_one_month_return_sector_mad'
past_two_month_return_sector_mad.name = 'past_two_month_return_sector_mad'
past_three_month_return_sector_mad.name = 'past_three_month_return_sector_mad'
past_four_month_return_sector_mad.name = 'past_four_month_return_sector_mad'
past_five_month_return_sector_mad.name = 'past_five_month_return_sector_mad'
past_six_month_return_sector_mad.name = 'past_six_month_return_sector_mad'
past_seven_month_return_sector_mad.name = 'past_seven_month_return_sector_mad'
past_eight_month_return_sector_mad.name = 'past_eight_month_return_sector_mad'
past_nine_month_return_sector_mad.name = 'past_nine_month_return_sector_mad'
past_ten_month_return_sector_mad.name = 'past_ten_month_return_sector_mad'
past_eleven_month_return_sector_mad.name = 'past_eleven_month_return_sector_mad'
past_twelve_month_return_sector_mad.name = 'past_twelve_month_return_sector_mad'
past_thirteen_month_return_sector_mad.name = 'past_thirteen_month_return_sector_mad'
past_fourteen_month_return_sector_mad.name = 'past_fourteen_month_return_sector_mad'
past_fifteen_month_return_sector_mad.name = 'past_fifteen_month_return_sector_mad'
past_sixteen_month_return_sector_mad.name = 'past_sixteen_month_return_sector_mad'
past_seventeen_month_return_sector_mad.name = 'past_seventeen_month_return_sector_mad'
past_eighteen_month_return_sector_mad.name = 'past_eighteen_month_return_sector_mad'
past_nineteen_month_return_sector_mad.name = 'past_nineteen_month_return_sector_mad'
past_twenty_month_return_sector_mad.name = 'past_twenty_month_return_sector_mad'
past_twentyone_month_return_sector_mad.name = 'past_twentyone_month_return_sector_mad'
past_twentytwo_month_return_sector_mad.name = 'past_twentytwo_month_return_sector_mad'
past_twentythree_month_return_sector_mad.name = 'past_twentythree_month_return_sector_mad'
past_twentyfour_month_return_sector_mad.name = 'past_twentyfour_month_return_sector_mad'
past_twentyfive_month_return_sector_mad.name = 'past_twentyfive_month_return_sector_mad'
past_twentysix_month_return_sector_mad.name = 'past_twentysix_month_return_sector_mad'
past_twentyseven_month_return_sector_mad.name = 'past_twentyseven_month_return_sector_mad'
past_twentyeight_month_return_sector_mad.name = 'past_twentyeight_month_return_sector_mad'
past_twentynine_month_return_sector_mad.name = 'past_twentynine_month_return_sector_mad'
past_thirty_month_return_sector_mad.name = 'past_thirty_month_return_sector_mad'
past_thirtyone_month_return_sector_mad.name = 'past_thirtyone_month_return_sector_mad'
past_thirtytwo_month_return_sector_mad.name = 'past_thirtytwo_month_return_sector_mad'
past_thirtythree_month_return_sector_mad.name = 'past_thirtythree_month_return_sector_mad'
past_thirtyfour_month_return_sector_mad.name = 'past_thirtyfour_month_return_sector_mad'
past_thirtyfive_month_return_sector_mad.name = 'past_thirtyfive_month_return_sector_mad'
past_thirtysix_month_return_sector_mad.name = 'past_thirtysix_month_return_sector_mad'
accrual_sector_mad.name = 'accrual_sector_mad'
adv_sale_sector_mad.name = 'adv_sale_sector_mad'
aftret_eq_sector_mad.name = 'aftret_eq_sector_mad'
aftret_equity_sector_mad.name = 'aftret_equity_sector_mad'
aftret_invcapx_sector_mad.name = 'aftret_invcapx_sector_mad'
at_turn_sector_mad.name = 'at_turn_sector_mad'
bm_sector_mad.name = 'bm_sector_mad'
CAPEI_sector_mad.name = 'CAPEI_sector_mad'
capital_ratio_sector_mad.name = 'capital_ratio_sector_mad'
cash_conversion_sector_mad.name = 'cash_conversion_sector_mad'
cash_debt_sector_mad.name = 'cash_debt_sector_mad'
cash_lt_sector_mad.name = 'cash_lt_sector_mad'
cash_ratio_sector_mad.name = 'cash_ratio_sector_mad'
cfm_sector_mad.name = 'cfm_sector_mad'
curr_debt_sector_mad.name = 'curr_debt_sector_mad'
curr_ratio_sector_mad.name = 'curr_ratio_sector_mad'
de_ratio_sector_mad.name = 'de_ratio_sector_mad'
debt_assets_sector_mad.name = 'debt_assets_sector_mad'
debt_at_sector_mad.name = 'debt_at_sector_mad'
debt_capital_sector_mad.name = 'debt_capital_sector_mad'
debt_ebitda_sector_mad.name = 'debt_ebitda_sector_mad'
debt_invcap_sector_mad.name = 'debt_invcap_sector_mad'
DIVYIELD_sector_mad.name = 'DIVYIELD_sector_mad'
dltt_be_sector_mad.name = 'dltt_be_sector_mad'
dpr_sector_mad.name = 'dpr_sector_mad'
efftax_sector_mad.name = 'efftax_sector_mad'
equity_invcap_sector_mad.name = 'equity_invcap_sector_mad'
evm_sector_mad.name = 'evm_sector_mad'
fcf_ocf_sector_mad.name = 'fcf_ocf_sector_mad'
gpm_sector_mad.name = 'gpm_sector_mad'
GProf_sector_mad.name = 'GProf_sector_mad'
int_debt_sector_mad.name = 'int_debt_sector_mad'
int_totdebt_sector_mad.name = 'int_totdebt_sector_mad'
intcov_sector_mad.name = 'intcov_sector_mad'
intcov_ratio_sector_mad.name = 'intcov_ratio_sector_mad'
inv_turn_sector_mad.name = 'inv_turn_sector_mad'
invt_act_sector_mad.name = 'invt_act_sector_mad'
lt_debt_sector_mad.name = 'lt_debt_sector_mad'
lt_ppent_sector_mad.name = 'lt_ppent_sector_mad'
npm_sector_mad.name = 'npm_sector_mad'
ocf_lct_sector_mad.name = 'ocf_lct_sector_mad'
opmad_sector_mad.name = 'opmad_sector_mad'
opmbd_sector_mad.name = 'opmbd_sector_mad'
pay_turn_sector_mad.name = 'pay_turn_sector_mad'
pcf_sector_mad.name = 'pcf_sector_mad'
pe_exi_sector_mad.name = 'pe_exi_sector_mad'
pe_inc_sector_mad.name = 'pe_inc_sector_mad'
pe_op_basic_sector_mad.name = 'pe_op_basic_sector_mad'
pe_op_dil_sector_mad.name = 'pe_op_dil_sector_mad'
PEG_1yrforward_sector_mad.name = 'PEG_1yrforward_sector_mad'
PEG_ltgforward_sector_mad.name = 'PEG_ltgforward_sector_mad'
PEG_trailing_sector_mad.name = 'PEG_trailing_sector_mad'
pretret_earnat_sector_mad.name = 'pretret_earnat_sector_mad'
pretret_noa_sector_mad.name = 'pretret_noa_sector_mad'
profit_lct_sector_mad.name = 'profit_lct_sector_mad'
ps_sector_mad.name = 'ps_sector_mad'
ptb_sector_mad.name = 'ptb_sector_mad'
ptpm_sector_mad.name = 'ptpm_sector_mad'
quick_ratio_sector_mad.name = 'quick_ratio_sector_mad'
rd_sale_sector_mad.name = 'rd_sale_sector_mad'
rect_act_sector_mad.name = 'rect_act_sector_mad'
rect_turn_sector_mad.name = 'rect_turn_sector_mad'
roa_sector_mad.name = 'roa_sector_mad'
roce_sector_mad.name = 'roce_sector_mad'
roe_sector_mad.name = 'roe_sector_mad'
sale_equity_sector_mad.name = 'sale_equity_sector_mad'
sale_invcap_sector_mad.name = 'sale_invcap_sector_mad'
sale_nwc_sector_mad.name = 'sale_nwc_sector_mad'
short_debt_sector_mad.name = 'short_debt_sector_mad'
staff_sale_sector_mad.name = 'staff_sale_sector_mad'
totdebt_invcap_sector_mad.name = 'totdebt_invcap_sector_mad'
dvpspm_sector_mad.name = 'dvpspm_sector_mad'
dvpsxm_sector_mad.name = 'dvpsxm_sector_mad'
dvrate_sector_mad.name = 'dvrate_sector_mad'
spcsrc_sector_mad.name = 'spcsrc_sector_mad'
alpha_sector_mad.name = 'alpha_sector_mad'
b_hml_sector_mad.name = 'b_hml_sector_mad'
b_mkt_sector_mad.name = 'b_mkt_sector_mad'
b_smb_sector_mad.name = 'b_smb_sector_mad'
b_umd_sector_mad.name = 'b_umd_sector_mad'
exret_sector_mad.name = 'exret_sector_mad'
ivol_sector_mad.name = 'ivol_sector_mad'
n_sector_mad.name = 'n_sector_mad'
R2_sector_mad.name = 'R2_sector_mad'
tvol_sector_mad.name = 'tvol_sector_mad'
BUYPCT_sector_mad.name = 'BUYPCT_sector_mad'
HOLDPCT_sector_mad.name = 'HOLDPCT_sector_mad'
MEANREC_sector_mad.name = 'MEANREC_sector_mad'
MEDREC_sector_mad.name = 'MEDREC_sector_mad'
recup_sector_mad.name = 'recup_sector_mad'
recdown_sector_mad.name = 'recdown_sector_mad'
SELLPCT_sector_mad.name = 'SELLPCT_sector_mad'
STDEV_sector_mad.name = 'STDEV_sector_mad'

In [68]:
# Join - df & Sector Mad Datasets
df = df.join(forward_one_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_two_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_three_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_four_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_five_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_six_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_seven_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_eight_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_nine_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_ten_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_eleven_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twelve_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_fourteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_fifteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_sixteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_seventeen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_eighteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_nineteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twenty_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentyone_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentytwo_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentythree_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentyfour_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentyfive_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentysix_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentyseven_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentyeight_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_twentynine_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirty_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirtyone_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirtytwo_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirtythree_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirtyfour_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirtyfive_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(forward_thirtysix_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_one_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_two_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_three_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_four_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_five_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_six_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_seven_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_eight_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_nine_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_ten_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_eleven_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twelve_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_fourteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_fifteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_sixteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_seventeen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_eighteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_nineteen_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twenty_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentyone_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentytwo_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentythree_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentyfour_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentyfive_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentysix_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentyseven_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentyeight_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_twentynine_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirty_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirtyone_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirtytwo_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirtythree_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirtyfour_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirtyfive_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(past_thirtysix_month_return_sector_mad, on=['year-month', 'sector'])
df = df.join(accrual_sector_mad, on=['year-month', 'sector'])
df = df.join(adv_sale_sector_mad, on=['year-month', 'sector'])
df = df.join(aftret_eq_sector_mad, on=['year-month', 'sector'])
df = df.join(aftret_equity_sector_mad, on=['year-month', 'sector'])
df = df.join(aftret_invcapx_sector_mad, on=['year-month', 'sector'])
df = df.join(at_turn_sector_mad, on=['year-month', 'sector'])
df = df.join(bm_sector_mad, on=['year-month', 'sector'])
df = df.join(CAPEI_sector_mad, on=['year-month', 'sector'])
df = df.join(capital_ratio_sector_mad, on=['year-month', 'sector'])
df = df.join(cash_conversion_sector_mad, on=['year-month', 'sector'])
df = df.join(cash_debt_sector_mad, on=['year-month', 'sector'])
df = df.join(cash_lt_sector_mad, on=['year-month', 'sector'])
df = df.join(cash_ratio_sector_mad, on=['year-month', 'sector'])
df = df.join(cfm_sector_mad, on=['year-month', 'sector'])
df = df.join(curr_debt_sector_mad, on=['year-month', 'sector'])
df = df.join(curr_ratio_sector_mad, on=['year-month', 'sector'])
df = df.join(de_ratio_sector_mad, on=['year-month', 'sector'])
df = df.join(debt_assets_sector_mad, on=['year-month', 'sector'])
df = df.join(debt_at_sector_mad, on=['year-month', 'sector'])
df = df.join(debt_capital_sector_mad, on=['year-month', 'sector'])
df = df.join(debt_ebitda_sector_mad, on=['year-month', 'sector'])
df = df.join(debt_invcap_sector_mad, on=['year-month', 'sector'])
df = df.join(DIVYIELD_sector_mad, on=['year-month', 'sector'])
df = df.join(dltt_be_sector_mad, on=['year-month', 'sector'])
df = df.join(dpr_sector_mad, on=['year-month', 'sector'])
df = df.join(efftax_sector_mad, on=['year-month', 'sector'])
df = df.join(equity_invcap_sector_mad, on=['year-month', 'sector'])
df = df.join(evm_sector_mad, on=['year-month', 'sector'])
df = df.join(fcf_ocf_sector_mad, on=['year-month', 'sector'])
df = df.join(gpm_sector_mad, on=['year-month', 'sector'])
df = df.join(GProf_sector_mad, on=['year-month', 'sector'])
df = df.join(int_debt_sector_mad, on=['year-month', 'sector'])
df = df.join(int_totdebt_sector_mad, on=['year-month', 'sector'])
df = df.join(intcov_sector_mad, on=['year-month', 'sector'])
df = df.join(intcov_ratio_sector_mad, on=['year-month', 'sector'])
df = df.join(inv_turn_sector_mad, on=['year-month', 'sector'])
df = df.join(invt_act_sector_mad, on=['year-month', 'sector'])
df = df.join(lt_debt_sector_mad, on=['year-month', 'sector'])
df = df.join(lt_ppent_sector_mad, on=['year-month', 'sector'])
df = df.join(npm_sector_mad, on=['year-month', 'sector'])
df = df.join(ocf_lct_sector_mad, on=['year-month', 'sector'])
df = df.join(opmad_sector_mad, on=['year-month', 'sector'])
df = df.join(opmbd_sector_mad, on=['year-month', 'sector'])
df = df.join(pay_turn_sector_mad, on=['year-month', 'sector'])
df = df.join(pcf_sector_mad, on=['year-month', 'sector'])
df = df.join(pe_exi_sector_mad, on=['year-month', 'sector'])
df = df.join(pe_inc_sector_mad, on=['year-month', 'sector'])
df = df.join(pe_op_basic_sector_mad, on=['year-month', 'sector'])
df = df.join(pe_op_dil_sector_mad, on=['year-month', 'sector'])
df = df.join(PEG_1yrforward_sector_mad, on=['year-month', 'sector'])
df = df.join(PEG_ltgforward_sector_mad, on=['year-month', 'sector'])
df = df.join(PEG_trailing_sector_mad, on=['year-month', 'sector'])
df = df.join(pretret_earnat_sector_mad, on=['year-month', 'sector'])
df = df.join(pretret_noa_sector_mad, on=['year-month', 'sector'])
df = df.join(profit_lct_sector_mad, on=['year-month', 'sector'])
df = df.join(ps_sector_mad, on=['year-month', 'sector'])
df = df.join(ptb_sector_mad, on=['year-month', 'sector'])
df = df.join(ptpm_sector_mad, on=['year-month', 'sector'])
df = df.join(quick_ratio_sector_mad, on=['year-month', 'sector'])
df = df.join(rd_sale_sector_mad, on=['year-month', 'sector'])
df = df.join(rect_act_sector_mad, on=['year-month', 'sector'])
df = df.join(rect_turn_sector_mad, on=['year-month', 'sector'])
df = df.join(roa_sector_mad, on=['year-month', 'sector'])
df = df.join(roce_sector_mad, on=['year-month', 'sector'])
df = df.join(roe_sector_mad, on=['year-month', 'sector'])
df = df.join(sale_equity_sector_mad, on=['year-month', 'sector'])
df = df.join(sale_invcap_sector_mad, on=['year-month', 'sector'])
df = df.join(sale_nwc_sector_mad, on=['year-month', 'sector'])
df = df.join(short_debt_sector_mad, on=['year-month', 'sector'])
df = df.join(staff_sale_sector_mad, on=['year-month', 'sector'])
df = df.join(totdebt_invcap_sector_mad, on=['year-month', 'sector'])
df = df.join(dvpspm_sector_mad, on=['year-month', 'sector'])
df = df.join(dvpsxm_sector_mad, on=['year-month', 'sector'])
df = df.join(dvrate_sector_mad, on=['year-month', 'sector'])
df = df.join(spcsrc_sector_mad, on=['year-month', 'sector'])
df = df.join(alpha_sector_mad, on=['year-month', 'sector'])
df = df.join(b_hml_sector_mad, on=['year-month', 'sector'])
df = df.join(b_mkt_sector_mad, on=['year-month', 'sector'])
df = df.join(b_smb_sector_mad, on=['year-month', 'sector'])
df = df.join(b_umd_sector_mad, on=['year-month', 'sector'])
df = df.join(exret_sector_mad, on=['year-month', 'sector'])
df = df.join(ivol_sector_mad, on=['year-month', 'sector'])
df = df.join(n_sector_mad, on=['year-month', 'sector'])
df = df.join(R2_sector_mad, on=['year-month', 'sector'])
df = df.join(tvol_sector_mad, on=['year-month', 'sector'])
df = df.join(BUYPCT_sector_mad, on=['year-month', 'sector'])
df = df.join(HOLDPCT_sector_mad, on=['year-month', 'sector'])
df = df.join(MEANREC_sector_mad, on=['year-month', 'sector'])
df = df.join(MEDREC_sector_mad, on=['year-month', 'sector'])
df = df.join(recup_sector_mad, on=['year-month', 'sector'])
df = df.join(recdown_sector_mad, on=['year-month', 'sector'])
df = df.join(SELLPCT_sector_mad, on=['year-month', 'sector'])
df = df.join(STDEV_sector_mad, on=['year-month', 'sector'])

In [69]:
# df - Enirch - Market Zscore - All
df['forward_one_month_return_zscore'] = (df['forward_one_month_return'] - df['forward_one_month_return_median']) / df['forward_one_month_return_mad']
df['forward_two_month_return_zscore'] = (df['forward_two_month_return'] - df['forward_two_month_return_median']) / df['forward_two_month_return_mad']
df['forward_three_month_return_zscore'] = (df['forward_three_month_return'] - df['forward_three_month_return_median']) / df['forward_three_month_return_mad']
df['forward_four_month_return_zscore'] = (df['forward_four_month_return'] - df['forward_four_month_return_median']) / df['forward_four_month_return_mad']
df['forward_five_month_return_zscore'] = (df['forward_five_month_return'] - df['forward_five_month_return_median']) / df['forward_five_month_return_mad']
df['forward_six_month_return_zscore'] = (df['forward_six_month_return'] - df['forward_six_month_return_median']) / df['forward_six_month_return_mad']
df['forward_seven_month_return_zscore'] = (df['forward_seven_month_return'] - df['forward_seven_month_return_median']) / df['forward_seven_month_return_mad']
df['forward_eight_month_return_zscore'] = (df['forward_eight_month_return'] - df['forward_eight_month_return_median']) / df['forward_eight_month_return_mad']
df['forward_nine_month_return_zscore'] = (df['forward_nine_month_return'] - df['forward_nine_month_return_median']) / df['forward_nine_month_return_mad']
df['forward_ten_month_return_zscore'] = (df['forward_ten_month_return'] - df['forward_ten_month_return_median']) / df['forward_ten_month_return_mad']
df['forward_eleven_month_return_zscore'] = (df['forward_eleven_month_return'] - df['forward_eleven_month_return_median']) / df['forward_eleven_month_return_mad']
df['forward_twelve_month_return_zscore'] = (df['forward_twelve_month_return'] - df['forward_twelve_month_return_median']) / df['forward_twelve_month_return_mad']
df['forward_thirteen_month_return_zscore'] = (df['forward_thirteen_month_return'] - df['forward_thirteen_month_return_median']) / df['forward_thirteen_month_return_mad']
df['forward_fourteen_month_return_zscore'] = (df['forward_fourteen_month_return'] - df['forward_fourteen_month_return_median']) / df['forward_fourteen_month_return_mad']
df['forward_fifteen_month_return_zscore'] = (df['forward_fifteen_month_return'] - df['forward_fifteen_month_return_median']) / df['forward_fifteen_month_return_mad']
df['forward_sixteen_month_return_zscore'] = (df['forward_sixteen_month_return'] - df['forward_sixteen_month_return_median']) / df['forward_sixteen_month_return_mad']
df['forward_seventeen_month_return_zscore'] = (df['forward_seventeen_month_return'] - df['forward_seventeen_month_return_median']) / df['forward_seventeen_month_return_mad']
df['forward_eighteen_month_return_zscore'] = (df['forward_eighteen_month_return'] - df['forward_eighteen_month_return_median']) / df['forward_eighteen_month_return_mad']
df['forward_nineteen_month_return_zscore'] = (df['forward_nineteen_month_return'] - df['forward_nineteen_month_return_median']) / df['forward_nineteen_month_return_mad']
df['forward_twenty_month_return_zscore'] = (df['forward_twenty_month_return'] - df['forward_twenty_month_return_median']) / df['forward_twenty_month_return_mad']
df['forward_twentyone_month_return_zscore'] = (df['forward_twentyone_month_return'] - df['forward_twentyone_month_return_median']) / df['forward_twentyone_month_return_mad']
df['forward_twentytwo_month_return_zscore'] = (df['forward_twentytwo_month_return'] - df['forward_twentytwo_month_return_median']) / df['forward_twentytwo_month_return_mad']
df['forward_twentythree_month_return_zscore'] = (df['forward_twentythree_month_return'] - df['forward_twentythree_month_return_median']) / df['forward_twentythree_month_return_mad']
df['forward_twentyfour_month_return_zscore'] = (df['forward_twentyfour_month_return'] - df['forward_twentyfour_month_return_median']) / df['forward_twentyfour_month_return_mad']
df['forward_twentyfive_month_return_zscore'] = (df['forward_twentyfive_month_return'] - df['forward_twentyfive_month_return_median']) / df['forward_twentyfive_month_return_mad']
df['forward_twentysix_month_return_zscore'] = (df['forward_twentysix_month_return'] - df['forward_twentysix_month_return_median']) / df['forward_twentysix_month_return_mad']
df['forward_twentyseven_month_return_zscore'] = (df['forward_twentyseven_month_return'] - df['forward_twentyseven_month_return_median']) / df['forward_twentyseven_month_return_mad']
df['forward_twentyeight_month_return_zscore'] = (df['forward_twentyeight_month_return'] - df['forward_twentyeight_month_return_median']) / df['forward_twentyeight_month_return_mad']
df['forward_twentynine_month_return_zscore'] = (df['forward_twentynine_month_return'] - df['forward_twentynine_month_return_median']) / df['forward_twentynine_month_return_mad']
df['forward_thirty_month_return_zscore'] = (df['forward_thirty_month_return'] - df['forward_thirty_month_return_median']) / df['forward_thirty_month_return_mad']
df['forward_thirtyone_month_return_zscore'] = (df['forward_thirtyone_month_return'] - df['forward_thirtyone_month_return_median']) / df['forward_thirtyone_month_return_mad']
df['forward_thirtytwo_month_return_zscore'] = (df['forward_thirtytwo_month_return'] - df['forward_thirtytwo_month_return_median']) / df['forward_thirtytwo_month_return_mad']
df['forward_thirtythree_month_return_zscore'] = (df['forward_thirtythree_month_return'] - df['forward_thirtythree_month_return_median']) / df['forward_thirtythree_month_return_mad']
df['forward_thirtyfour_month_return_zscore'] = (df['forward_thirtyfour_month_return'] - df['forward_thirtyfour_month_return_median']) / df['forward_thirtyfour_month_return_mad']
df['forward_thirtyfive_month_return_zscore'] = (df['forward_thirtyfive_month_return'] - df['forward_thirtyfive_month_return_median']) / df['forward_thirtyfive_month_return_mad']
df['forward_thirtysix_month_return_zscore'] = (df['forward_thirtysix_month_return'] - df['forward_thirtysix_month_return_median']) / df['forward_thirtysix_month_return_mad']
df['past_one_month_return_zscore'] = (df['past_one_month_return'] - df['past_one_month_return_median']) / df['past_one_month_return_mad']
df['past_two_month_return_zscore'] = (df['past_two_month_return'] - df['past_two_month_return_median']) / df['past_two_month_return_mad']
df['past_three_month_return_zscore'] = (df['past_three_month_return'] - df['past_three_month_return_median']) / df['past_three_month_return_mad']
df['past_four_month_return_zscore'] = (df['past_four_month_return'] - df['past_four_month_return_median']) / df['past_four_month_return_mad']
df['past_five_month_return_zscore'] = (df['past_five_month_return'] - df['past_five_month_return_median']) / df['past_five_month_return_mad']
df['past_six_month_return_zscore'] = (df['past_six_month_return'] - df['past_six_month_return_median']) / df['past_six_month_return_mad']
df['past_seven_month_return_zscore'] = (df['past_seven_month_return'] - df['past_seven_month_return_median']) / df['past_seven_month_return_mad']
df['past_eight_month_return_zscore'] = (df['past_eight_month_return'] - df['past_eight_month_return_median']) / df['past_eight_month_return_mad']
df['past_nine_month_return_zscore'] = (df['past_nine_month_return'] - df['past_nine_month_return_median']) / df['past_nine_month_return_mad']
df['past_ten_month_return_zscore'] = (df['past_ten_month_return'] - df['past_ten_month_return_median']) / df['past_ten_month_return_mad']
df['past_eleven_month_return_zscore'] = (df['past_eleven_month_return'] - df['past_eleven_month_return_median']) / df['past_eleven_month_return_mad']
df['past_twelve_month_return_zscore'] = (df['past_twelve_month_return'] - df['past_twelve_month_return_median']) / df['past_twelve_month_return_mad']
df['past_thirteen_month_return_zscore'] = (df['past_thirteen_month_return'] - df['past_thirteen_month_return_median']) / df['past_thirteen_month_return_mad']
df['past_fourteen_month_return_zscore'] = (df['past_fourteen_month_return'] - df['past_fourteen_month_return_median']) / df['past_fourteen_month_return_mad']
df['past_fifteen_month_return_zscore'] = (df['past_fifteen_month_return'] - df['past_fifteen_month_return_median']) / df['past_fifteen_month_return_mad']
df['past_sixteen_month_return_zscore'] = (df['past_sixteen_month_return'] - df['past_sixteen_month_return_median']) / df['past_sixteen_month_return_mad']
df['past_seventeen_month_return_zscore'] = (df['past_seventeen_month_return'] - df['past_seventeen_month_return_median']) / df['past_seventeen_month_return_mad']
df['past_eighteen_month_return_zscore'] = (df['past_eighteen_month_return'] - df['past_eighteen_month_return_median']) / df['past_eighteen_month_return_mad']
df['past_nineteen_month_return_zscore'] = (df['past_nineteen_month_return'] - df['past_nineteen_month_return_median']) / df['past_nineteen_month_return_mad']
df['past_twenty_month_return_zscore'] = (df['past_twenty_month_return'] - df['past_twenty_month_return_median']) / df['past_twenty_month_return_mad']
df['past_twentyone_month_return_zscore'] = (df['past_twentyone_month_return'] - df['past_twentyone_month_return_median']) / df['past_twentyone_month_return_mad']
df['past_twentytwo_month_return_zscore'] = (df['past_twentytwo_month_return'] - df['past_twentytwo_month_return_median']) / df['past_twentytwo_month_return_mad']
df['past_twentythree_month_return_zscore'] = (df['past_twentythree_month_return'] - df['past_twentythree_month_return_median']) / df['past_twentythree_month_return_mad']
df['past_twentyfour_month_return_zscore'] = (df['past_twentyfour_month_return'] - df['past_twentyfour_month_return_median']) / df['past_twentyfour_month_return_mad']
df['past_twentyfive_month_return_zscore'] = (df['past_twentyfive_month_return'] - df['past_twentyfive_month_return_median']) / df['past_twentyfive_month_return_mad']
df['past_twentysix_month_return_zscore'] = (df['past_twentysix_month_return'] - df['past_twentysix_month_return_median']) / df['past_twentysix_month_return_mad']
df['past_twentyseven_month_return_zscore'] = (df['past_twentyseven_month_return'] - df['past_twentyseven_month_return_median']) / df['past_twentyseven_month_return_mad']
df['past_twentyeight_month_return_zscore'] = (df['past_twentyeight_month_return'] - df['past_twentyeight_month_return_median']) / df['past_twentyeight_month_return_mad']
df['past_twentynine_month_return_zscore'] = (df['past_twentynine_month_return'] - df['past_twentynine_month_return_median']) / df['past_twentynine_month_return_mad']
df['past_thirty_month_return_zscore'] = (df['past_thirty_month_return'] - df['past_thirty_month_return_median']) / df['past_thirty_month_return_mad']
df['past_thirtyone_month_return_zscore'] = (df['past_thirtyone_month_return'] - df['past_thirtyone_month_return_median']) / df['past_thirtyone_month_return_mad']
df['past_thirtytwo_month_return_zscore'] = (df['past_thirtytwo_month_return'] - df['past_thirtytwo_month_return_median']) / df['past_thirtytwo_month_return_mad']
df['past_thirtythree_month_return_zscore'] = (df['past_thirtythree_month_return'] - df['past_thirtythree_month_return_median']) / df['past_thirtythree_month_return_mad']
df['past_thirtyfour_month_return_zscore'] = (df['past_thirtyfour_month_return'] - df['past_thirtyfour_month_return_median']) / df['past_thirtyfour_month_return_mad']
df['past_thirtyfive_month_return_zscore'] = (df['past_thirtyfive_month_return'] - df['past_thirtyfive_month_return_median']) / df['past_thirtyfive_month_return_mad']
df['past_thirtysix_month_return_zscore'] = (df['past_thirtysix_month_return'] - df['past_thirtysix_month_return_median']) / df['past_thirtysix_month_return_mad']
df['accrual_zscore'] = (df['accrual'] - df['accrual_median']) / df['accrual_mad']
df['adv_sale_zscore'] = (df['adv_sale'] - df['adv_sale_median']) / df['adv_sale_mad']
df['aftret_eq_zscore'] = (df['aftret_eq'] - df['aftret_eq_median']) / df['aftret_eq_mad']
df['aftret_equity_zscore'] = (df['aftret_equity'] - df['aftret_equity_median']) / df['aftret_equity_mad']
df['aftret_invcapx_zscore'] = (df['aftret_invcapx'] - df['aftret_invcapx_median']) / df['aftret_invcapx_mad']
df['at_turn_zscore'] = (df['at_turn'] - df['at_turn_median']) / df['at_turn_mad']
df['bm_zscore'] = (df['bm'] - df['bm_median']) / df['bm_mad']
df['CAPEI_zscore'] = (df['CAPEI'] - df['CAPEI_median']) / df['CAPEI_mad']
df['capital_ratio_zscore'] = (df['capital_ratio'] - df['capital_ratio_median']) / df['capital_ratio_mad']
df['cash_conversion_zscore'] = (df['cash_conversion'] - df['cash_conversion_median']) / df['cash_conversion_mad']
df['cash_debt_zscore'] = (df['cash_debt'] - df['cash_debt_median']) / df['cash_debt_mad']
df['cash_lt_zscore'] = (df['cash_lt'] - df['cash_lt_median']) / df['cash_lt_mad']
df['cash_ratio_zscore'] = (df['cash_ratio'] - df['cash_ratio_median']) / df['cash_ratio_mad']
df['cfm_zscore'] = (df['cfm'] - df['cfm_median']) / df['cfm_mad']
df['curr_debt_zscore'] = (df['curr_debt'] - df['curr_debt_median']) / df['curr_debt_mad']
df['curr_ratio_zscore'] = (df['curr_ratio'] - df['curr_ratio_median']) / df['curr_ratio_mad']
df['de_ratio_zscore'] = (df['de_ratio'] - df['de_ratio_median']) / df['de_ratio_mad']
df['debt_assets_zscore'] = (df['debt_assets'] - df['debt_assets_median']) / df['debt_assets_mad']
df['debt_at_zscore'] = (df['debt_at'] - df['debt_at_median']) / df['debt_at_mad']
df['debt_capital_zscore'] = (df['debt_capital'] - df['debt_capital_median']) / df['debt_capital_mad']
df['debt_ebitda_zscore'] = (df['debt_ebitda'] - df['debt_ebitda_median']) / df['debt_ebitda_mad']
df['debt_invcap_zscore'] = (df['debt_invcap'] - df['debt_invcap_median']) / df['debt_invcap_mad']
df['DIVYIELD_zscore'] = (df['DIVYIELD'] - df['DIVYIELD_median']) / df['DIVYIELD_mad']
df['dltt_be_zscore'] = (df['dltt_be'] - df['dltt_be_median']) / df['dltt_be_mad']
df['dpr_zscore'] = (df['dpr'] - df['dpr_median']) / df['dpr_mad']
df['efftax_zscore'] = (df['efftax'] - df['efftax_median']) / df['efftax_mad']
df['equity_invcap_zscore'] = (df['equity_invcap'] - df['equity_invcap_median']) / df['equity_invcap_mad']
df['evm_zscore'] = (df['evm'] - df['evm_median']) / df['evm_mad']
df['fcf_ocf_zscore'] = (df['fcf_ocf'] - df['fcf_ocf_median']) / df['fcf_ocf_mad']
df['gpm_zscore'] = (df['gpm'] - df['gpm_median']) / df['gpm_mad']
df['GProf_zscore'] = (df['GProf'] - df['GProf_median']) / df['GProf_mad']
df['int_debt_zscore'] = (df['int_debt'] - df['int_debt_median']) / df['int_debt_mad']
df['int_totdebt_zscore'] = (df['int_totdebt'] - df['int_totdebt_median']) / df['int_totdebt_mad']
df['intcov_zscore'] = (df['intcov'] - df['intcov_median']) / df['intcov_mad']
df['intcov_ratio_zscore'] = (df['intcov_ratio'] - df['intcov_ratio_median']) / df['intcov_ratio_mad']
df['inv_turn_zscore'] = (df['inv_turn'] - df['inv_turn_median']) / df['inv_turn_mad']
df['invt_act_zscore'] = (df['invt_act'] - df['invt_act_median']) / df['invt_act_mad']
df['lt_debt_zscore'] = (df['lt_debt'] - df['lt_debt_median']) / df['lt_debt_mad']
df['lt_ppent_zscore'] = (df['lt_ppent'] - df['lt_ppent_median']) / df['lt_ppent_mad']
df['npm_zscore'] = (df['npm'] - df['npm_median']) / df['npm_mad']
df['ocf_lct_zscore'] = (df['ocf_lct'] - df['ocf_lct_median']) / df['ocf_lct_mad']
df['opmad_zscore'] = (df['opmad'] - df['opmad_median']) / df['opmad_mad']
df['opmbd_zscore'] = (df['opmbd'] - df['opmbd_median']) / df['opmbd_mad']
df['pay_turn_zscore'] = (df['pay_turn'] - df['pay_turn_median']) / df['pay_turn_mad']
df['pcf_zscore'] = (df['pcf'] - df['pcf_median']) / df['pcf_mad']
df['pe_exi_zscore'] = (df['pe_exi'] - df['pe_exi_median']) / df['pe_exi_mad']
df['pe_inc_zscore'] = (df['pe_inc'] - df['pe_inc_median']) / df['pe_inc_mad']
df['pe_op_basic_zscore'] = (df['pe_op_basic'] - df['pe_op_basic_median']) / df['pe_op_basic_mad']
df['pe_op_dil_zscore'] = (df['pe_op_dil'] - df['pe_op_dil_median']) / df['pe_op_dil_mad']
df['PEG_1yrforward_zscore'] = (df['PEG_1yrforward'] - df['PEG_1yrforward_median']) / df['PEG_1yrforward_mad']
df['PEG_ltgforward_zscore'] = (df['PEG_ltgforward'] - df['PEG_ltgforward_median']) / df['PEG_ltgforward_mad']
df['PEG_trailing_zscore'] = (df['PEG_trailing'] - df['PEG_trailing_median']) / df['PEG_trailing_mad']
df['pretret_earnat_zscore'] = (df['pretret_earnat'] - df['pretret_earnat_median']) / df['pretret_earnat_mad']
df['pretret_noa_zscore'] = (df['pretret_noa'] - df['pretret_noa_median']) / df['pretret_noa_mad']
df['profit_lct_zscore'] = (df['profit_lct'] - df['profit_lct_median']) / df['profit_lct_mad']
df['ps_zscore'] = (df['ps'] - df['ps_median']) / df['ps_mad']
df['ptb_zscore'] = (df['ptb'] - df['ptb_median']) / df['ptb_mad']
df['ptpm_zscore'] = (df['ptpm'] - df['ptpm_median']) / df['ptpm_mad']
df['quick_ratio_zscore'] = (df['quick_ratio'] - df['quick_ratio_median']) / df['quick_ratio_mad']
df['rd_sale_zscore'] = (df['rd_sale'] - df['rd_sale_median']) / df['rd_sale_mad']
df['rect_act_zscore'] = (df['rect_act'] - df['rect_act_median']) / df['rect_act_mad']
df['rect_turn_zscore'] = (df['rect_turn'] - df['rect_turn_median']) / df['rect_turn_mad']
df['roa_zscore'] = (df['roa'] - df['roa_median']) / df['roa_mad']
df['roce_zscore'] = (df['roce'] - df['roce_median']) / df['roce_mad']
df['roe_zscore'] = (df['roe'] - df['roe_median']) / df['roe_mad']
df['sale_equity_zscore'] = (df['sale_equity'] - df['sale_equity_median']) / df['sale_equity_mad']
df['sale_invcap_zscore'] = (df['sale_invcap'] - df['sale_invcap_median']) / df['sale_invcap_mad']
df['sale_nwc_zscore'] = (df['sale_nwc'] - df['sale_nwc_median']) / df['sale_nwc_mad']
df['short_debt_zscore'] = (df['short_debt'] - df['short_debt_median']) / df['short_debt_mad']
df['staff_sale_zscore'] = (df['staff_sale'] - df['staff_sale_median']) / df['staff_sale_mad']
df['totdebt_invcap_zscore'] = (df['totdebt_invcap'] - df['totdebt_invcap_median']) / df['totdebt_invcap_mad']
df['dvpspm_zscore'] = (df['dvpspm'] - df['dvpspm_median']) / df['dvpspm_mad']
df['dvpsxm_zscore'] = (df['dvpsxm'] - df['dvpsxm_median']) / df['dvpsxm_mad']
df['dvrate_zscore'] = (df['dvrate'] - df['dvrate_median']) / df['dvrate_mad']
df['spcsrc_zscore'] = (df['spcsrc'] - df['spcsrc_median']) / df['spcsrc_mad']
df['alpha_zscore'] = (df['alpha'] - df['alpha_median']) / df['alpha_mad']
df['b_hml_zscore'] = (df['b_hml'] - df['b_hml_median']) / df['b_hml_mad']
df['b_mkt_zscore'] = (df['b_mkt'] - df['b_mkt_median']) / df['b_mkt_mad']
df['b_smb_zscore'] = (df['b_smb'] - df['b_smb_median']) / df['b_smb_mad']
df['b_umd_zscore'] = (df['b_umd'] - df['b_umd_median']) / df['b_umd_mad']
df['exret_zscore'] = (df['exret'] - df['exret_median']) / df['exret_mad']
df['ivol_zscore'] = (df['ivol'] - df['ivol_median']) / df['ivol_mad']
df['n_zscore'] = (df['n'] - df['n_median']) / df['n_mad']
df['R2_zscore'] = (df['R2'] - df['R2_median']) / df['R2_mad']
df['tvol_zscore'] = (df['tvol'] - df['tvol_median']) / df['tvol_mad']
df['BUYPCT_zscore'] = (df['BUYPCT'] - df['BUYPCT_median']) / df['BUYPCT_mad']
df['HOLDPCT_zscore'] = (df['HOLDPCT'] - df['HOLDPCT_median']) / df['HOLDPCT_mad']
df['MEANREC_zscore'] = (df['MEANREC'] - df['MEANREC_median']) / df['MEANREC_mad']
df['MEDREC_zscore'] = (df['MEDREC'] - df['MEDREC_median']) / df['MEDREC_mad']
df['recup_zscore'] = (df['recup'] - df['recup_median']) / df['recup_mad']
df['recdown_zscore'] = (df['recdown'] - df['recdown_median']) / df['recdown_mad']
df['SELLPCT_zscore'] = (df['SELLPCT'] - df['SELLPCT_median']) / df['SELLPCT_mad']
df['STDEV_zscore'] = (df['STDEV'] - df['STDEV_median']) / df['STDEV_mad']

In [70]:
# Dataframe - Enirch - Sector Zscore - All
df['forward_one_month_return_sector_zscore'] = (df['forward_one_month_return'] - df['forward_one_month_return_sector_median']) / df['forward_one_month_return_sector_mad']
df['forward_two_month_return_sector_zscore'] = (df['forward_two_month_return'] - df['forward_two_month_return_sector_median']) / df['forward_two_month_return_sector_mad']
df['forward_three_month_return_sector_zscore'] = (df['forward_three_month_return'] - df['forward_three_month_return_sector_median']) / df['forward_three_month_return_sector_mad']
df['forward_four_month_return_sector_zscore'] = (df['forward_four_month_return'] - df['forward_four_month_return_sector_median']) / df['forward_four_month_return_sector_mad']
df['forward_five_month_return_sector_zscore'] = (df['forward_five_month_return'] - df['forward_five_month_return_sector_median']) / df['forward_five_month_return_sector_mad']
df['forward_six_month_return_sector_zscore'] = (df['forward_six_month_return'] - df['forward_six_month_return_sector_median']) / df['forward_six_month_return_sector_mad']
df['forward_seven_month_return_sector_zscore'] = (df['forward_seven_month_return'] - df['forward_seven_month_return_sector_median']) / df['forward_seven_month_return_sector_mad']
df['forward_eight_month_return_sector_zscore'] = (df['forward_eight_month_return'] - df['forward_eight_month_return_sector_median']) / df['forward_eight_month_return_sector_mad']
df['forward_nine_month_return_sector_zscore'] = (df['forward_nine_month_return'] - df['forward_nine_month_return_sector_median']) / df['forward_nine_month_return_sector_mad']
df['forward_ten_month_return_sector_zscore'] = (df['forward_ten_month_return'] - df['forward_ten_month_return_sector_median']) / df['forward_ten_month_return_sector_mad']
df['forward_eleven_month_return_sector_zscore'] = (df['forward_eleven_month_return'] - df['forward_eleven_month_return_sector_median']) / df['forward_eleven_month_return_sector_mad']
df['forward_twelve_month_return_sector_zscore'] = (df['forward_twelve_month_return'] - df['forward_twelve_month_return_sector_median']) / df['forward_twelve_month_return_sector_mad']
df['forward_thirteen_month_return_sector_zscore'] = (df['forward_thirteen_month_return'] - df['forward_thirteen_month_return_sector_median']) / df['forward_thirteen_month_return_sector_mad']
df['forward_fourteen_month_return_sector_zscore'] = (df['forward_fourteen_month_return'] - df['forward_fourteen_month_return_sector_median']) / df['forward_fourteen_month_return_sector_mad']
df['forward_fifteen_month_return_sector_zscore'] = (df['forward_fifteen_month_return'] - df['forward_fifteen_month_return_sector_median']) / df['forward_fifteen_month_return_sector_mad']
df['forward_sixteen_month_return_sector_zscore'] = (df['forward_sixteen_month_return'] - df['forward_sixteen_month_return_sector_median']) / df['forward_sixteen_month_return_sector_mad']
df['forward_seventeen_month_return_sector_zscore'] = (df['forward_seventeen_month_return'] - df['forward_seventeen_month_return_sector_median']) / df['forward_seventeen_month_return_sector_mad']
df['forward_eighteen_month_return_sector_zscore'] = (df['forward_eighteen_month_return'] - df['forward_eighteen_month_return_sector_median']) / df['forward_eighteen_month_return_sector_mad']
df['forward_nineteen_month_return_sector_zscore'] = (df['forward_nineteen_month_return'] - df['forward_nineteen_month_return_sector_median']) / df['forward_nineteen_month_return_sector_mad']
df['forward_twenty_month_return_sector_zscore'] = (df['forward_twenty_month_return'] - df['forward_twenty_month_return_sector_median']) / df['forward_twenty_month_return_sector_mad']
df['forward_twentyone_month_return_sector_zscore'] = (df['forward_twentyone_month_return'] - df['forward_twentyone_month_return_sector_median']) / df['forward_twentyone_month_return_sector_mad']
df['forward_twentytwo_month_return_sector_zscore'] = (df['forward_twentytwo_month_return'] - df['forward_twentytwo_month_return_sector_median']) / df['forward_twentytwo_month_return_sector_mad']
df['forward_twentythree_month_return_sector_zscore'] = (df['forward_twentythree_month_return'] - df['forward_twentythree_month_return_sector_median']) / df['forward_twentythree_month_return_sector_mad']
df['forward_twentyfour_month_return_sector_zscore'] = (df['forward_twentyfour_month_return'] - df['forward_twentyfour_month_return_sector_median']) / df['forward_twentyfour_month_return_sector_mad']
df['forward_twentyfive_month_return_sector_zscore'] = (df['forward_twentyfive_month_return'] - df['forward_twentyfive_month_return_sector_median']) / df['forward_twentyfive_month_return_sector_mad']
df['forward_twentysix_month_return_sector_zscore'] = (df['forward_twentysix_month_return'] - df['forward_twentysix_month_return_sector_median']) / df['forward_twentysix_month_return_sector_mad']
df['forward_twentyseven_month_return_sector_zscore'] = (df['forward_twentyseven_month_return'] - df['forward_twentyseven_month_return_sector_median']) / df['forward_twentyseven_month_return_sector_mad']
df['forward_twentyeight_month_return_sector_zscore'] = (df['forward_twentyeight_month_return'] - df['forward_twentyeight_month_return_sector_median']) / df['forward_twentyeight_month_return_sector_mad']
df['forward_twentynine_month_return_sector_zscore'] = (df['forward_twentynine_month_return'] - df['forward_twentynine_month_return_sector_median']) / df['forward_twentynine_month_return_sector_mad']
df['forward_thirty_month_return_sector_zscore'] = (df['forward_thirty_month_return'] - df['forward_thirty_month_return_sector_median']) / df['forward_thirty_month_return_sector_mad']
df['forward_thirtyone_month_return_sector_zscore'] = (df['forward_thirtyone_month_return'] - df['forward_thirtyone_month_return_sector_median']) / df['forward_thirtyone_month_return_sector_mad']
df['forward_thirtytwo_month_return_sector_zscore'] = (df['forward_thirtytwo_month_return'] - df['forward_thirtytwo_month_return_sector_median']) / df['forward_thirtytwo_month_return_sector_mad']
df['forward_thirtythree_month_return_sector_zscore'] = (df['forward_thirtythree_month_return'] - df['forward_thirtythree_month_return_sector_median']) / df['forward_thirtythree_month_return_sector_mad']
df['forward_thirtyfour_month_return_sector_zscore'] = (df['forward_thirtyfour_month_return'] - df['forward_thirtyfour_month_return_sector_median']) / df['forward_thirtyfour_month_return_sector_mad']
df['forward_thirtyfive_month_return_sector_zscore'] = (df['forward_thirtyfive_month_return'] - df['forward_thirtyfive_month_return_sector_median']) / df['forward_thirtyfive_month_return_sector_mad']
df['forward_thirtysix_month_return_sector_zscore'] = (df['forward_thirtysix_month_return'] - df['forward_thirtysix_month_return_sector_median']) / df['forward_thirtysix_month_return_sector_mad']
df['past_one_month_return_sector_zscore'] = (df['past_one_month_return'] - df['past_one_month_return_sector_median']) / df['past_one_month_return_sector_mad']
df['past_two_month_return_sector_zscore'] = (df['past_two_month_return'] - df['past_two_month_return_sector_median']) / df['past_two_month_return_sector_mad']
df['past_three_month_return_sector_zscore'] = (df['past_three_month_return'] - df['past_three_month_return_sector_median']) / df['past_three_month_return_sector_mad']
df['past_four_month_return_sector_zscore'] = (df['past_four_month_return'] - df['past_four_month_return_sector_median']) / df['past_four_month_return_sector_mad']
df['past_five_month_return_sector_zscore'] = (df['past_five_month_return'] - df['past_five_month_return_sector_median']) / df['past_five_month_return_sector_mad']
df['past_six_month_return_sector_zscore'] = (df['past_six_month_return'] - df['past_six_month_return_sector_median']) / df['past_six_month_return_sector_mad']
df['past_seven_month_return_sector_zscore'] = (df['past_seven_month_return'] - df['past_seven_month_return_sector_median']) / df['past_seven_month_return_sector_mad']
df['past_eight_month_return_sector_zscore'] = (df['past_eight_month_return'] - df['past_eight_month_return_sector_median']) / df['past_eight_month_return_sector_mad']
df['past_nine_month_return_sector_zscore'] = (df['past_nine_month_return'] - df['past_nine_month_return_sector_median']) / df['past_nine_month_return_sector_mad']
df['past_ten_month_return_sector_zscore'] = (df['past_ten_month_return'] - df['past_ten_month_return_sector_median']) / df['past_ten_month_return_sector_mad']
df['past_eleven_month_return_sector_zscore'] = (df['past_eleven_month_return'] - df['past_eleven_month_return_sector_median']) / df['past_eleven_month_return_sector_mad']
df['past_twelve_month_return_sector_zscore'] = (df['past_twelve_month_return'] - df['past_twelve_month_return_sector_median']) / df['past_twelve_month_return_sector_mad']
df['past_thirteen_month_return_sector_zscore'] = (df['past_thirteen_month_return'] - df['past_thirteen_month_return_sector_median']) / df['past_thirteen_month_return_sector_mad']
df['past_fourteen_month_return_sector_zscore'] = (df['past_fourteen_month_return'] - df['past_fourteen_month_return_sector_median']) / df['past_fourteen_month_return_sector_mad']
df['past_fifteen_month_return_sector_zscore'] = (df['past_fifteen_month_return'] - df['past_fifteen_month_return_sector_median']) / df['past_fifteen_month_return_sector_mad']
df['past_sixteen_month_return_sector_zscore'] = (df['past_sixteen_month_return'] - df['past_sixteen_month_return_sector_median']) / df['past_sixteen_month_return_sector_mad']
df['past_seventeen_month_return_sector_zscore'] = (df['past_seventeen_month_return'] - df['past_seventeen_month_return_sector_median']) / df['past_seventeen_month_return_sector_mad']
df['past_eighteen_month_return_sector_zscore'] = (df['past_eighteen_month_return'] - df['past_eighteen_month_return_sector_median']) / df['past_eighteen_month_return_sector_mad']
df['past_nineteen_month_return_sector_zscore'] = (df['past_nineteen_month_return'] - df['past_nineteen_month_return_sector_median']) / df['past_nineteen_month_return_sector_mad']
df['past_twenty_month_return_sector_zscore'] = (df['past_twenty_month_return'] - df['past_twenty_month_return_sector_median']) / df['past_twenty_month_return_sector_mad']
df['past_twentyone_month_return_sector_zscore'] = (df['past_twentyone_month_return'] - df['past_twentyone_month_return_sector_median']) / df['past_twentyone_month_return_sector_mad']
df['past_twentytwo_month_return_sector_zscore'] = (df['past_twentytwo_month_return'] - df['past_twentytwo_month_return_sector_median']) / df['past_twentytwo_month_return_sector_mad']
df['past_twentythree_month_return_sector_zscore'] = (df['past_twentythree_month_return'] - df['past_twentythree_month_return_sector_median']) / df['past_twentythree_month_return_sector_mad']
df['past_twentyfour_month_return_sector_zscore'] = (df['past_twentyfour_month_return'] - df['past_twentyfour_month_return_sector_median']) / df['past_twentyfour_month_return_sector_mad']
df['past_twentyfive_month_return_sector_zscore'] = (df['past_twentyfive_month_return'] - df['past_twentyfive_month_return_sector_median']) / df['past_twentyfive_month_return_sector_mad']
df['past_twentysix_month_return_sector_zscore'] = (df['past_twentysix_month_return'] - df['past_twentysix_month_return_sector_median']) / df['past_twentysix_month_return_sector_mad']
df['past_twentyseven_month_return_sector_zscore'] = (df['past_twentyseven_month_return'] - df['past_twentyseven_month_return_sector_median']) / df['past_twentyseven_month_return_sector_mad']
df['past_twentyeight_month_return_sector_zscore'] = (df['past_twentyeight_month_return'] - df['past_twentyeight_month_return_sector_median']) / df['past_twentyeight_month_return_sector_mad']
df['past_twentynine_month_return_sector_zscore'] = (df['past_twentynine_month_return'] - df['past_twentynine_month_return_sector_median']) / df['past_twentynine_month_return_sector_mad']
df['past_thirty_month_return_sector_zscore'] = (df['past_thirty_month_return'] - df['past_thirty_month_return_sector_median']) / df['past_thirty_month_return_sector_mad']
df['past_thirtyone_month_return_sector_zscore'] = (df['past_thirtyone_month_return'] - df['past_thirtyone_month_return_sector_median']) / df['past_thirtyone_month_return_sector_mad']
df['past_thirtytwo_month_return_sector_zscore'] = (df['past_thirtytwo_month_return'] - df['past_thirtytwo_month_return_sector_median']) / df['past_thirtytwo_month_return_sector_mad']
df['past_thirtythree_month_return_sector_zscore'] = (df['past_thirtythree_month_return'] - df['past_thirtythree_month_return_sector_median']) / df['past_thirtythree_month_return_sector_mad']
df['past_thirtyfour_month_return_sector_zscore'] = (df['past_thirtyfour_month_return'] - df['past_thirtyfour_month_return_sector_median']) / df['past_thirtyfour_month_return_sector_mad']
df['past_thirtyfive_month_return_sector_zscore'] = (df['past_thirtyfive_month_return'] - df['past_thirtyfive_month_return_sector_median']) / df['past_thirtyfive_month_return_sector_mad']
df['past_thirtysix_month_return_sector_zscore'] = (df['past_thirtysix_month_return'] - df['past_thirtysix_month_return_sector_median']) / df['past_thirtysix_month_return_sector_mad']
df['accrual_sector_zscore'] = (df['accrual'] - df['accrual_sector_median']) / df['accrual_sector_mad']
df['adv_sale_sector_zscore'] = (df['adv_sale'] - df['adv_sale_sector_median']) / df['adv_sale_sector_mad']
df['aftret_eq_sector_zscore'] = (df['aftret_eq'] - df['aftret_eq_sector_median']) / df['aftret_eq_sector_mad']
df['aftret_equity_sector_zscore'] = (df['aftret_equity'] - df['aftret_equity_sector_median']) / df['aftret_equity_sector_mad']
df['aftret_invcapx_sector_zscore'] = (df['aftret_invcapx'] - df['aftret_invcapx_sector_median']) / df['aftret_invcapx_sector_mad']
df['at_turn_sector_zscore'] = (df['at_turn'] - df['at_turn_sector_median']) / df['at_turn_sector_mad']
df['bm_sector_zscore'] = (df['bm'] - df['bm_sector_median']) / df['bm_sector_mad']
df['CAPEI_sector_zscore'] = (df['CAPEI'] - df['CAPEI_sector_median']) / df['CAPEI_sector_mad']
df['capital_ratio_sector_zscore'] = (df['capital_ratio'] - df['capital_ratio_sector_median']) / df['capital_ratio_sector_mad']
df['cash_conversion_sector_zscore'] = (df['cash_conversion'] - df['cash_conversion_sector_median']) / df['cash_conversion_sector_mad']
df['cash_debt_sector_zscore'] = (df['cash_debt'] - df['cash_debt_sector_median']) / df['cash_debt_sector_mad']
df['cash_lt_sector_zscore'] = (df['cash_lt'] - df['cash_lt_sector_median']) / df['cash_lt_sector_mad']
df['cash_ratio_sector_zscore'] = (df['cash_ratio'] - df['cash_ratio_sector_median']) / df['cash_ratio_sector_mad']
df['cfm_sector_zscore'] = (df['cfm'] - df['cfm_sector_median']) / df['cfm_sector_mad']
df['curr_debt_sector_zscore'] = (df['curr_debt'] - df['curr_debt_sector_median']) / df['curr_debt_sector_mad']
df['curr_ratio_sector_zscore'] = (df['curr_ratio'] - df['curr_ratio_sector_median']) / df['curr_ratio_sector_mad']
df['de_ratio_sector_zscore'] = (df['de_ratio'] - df['de_ratio_sector_median']) / df['de_ratio_sector_mad']
df['debt_assets_sector_zscore'] = (df['debt_assets'] - df['debt_assets_sector_median']) / df['debt_assets_sector_mad']
df['debt_at_sector_zscore'] = (df['debt_at'] - df['debt_at_sector_median']) / df['debt_at_sector_mad']
df['debt_capital_sector_zscore'] = (df['debt_capital'] - df['debt_capital_sector_median']) / df['debt_capital_sector_mad']
df['debt_ebitda_sector_zscore'] = (df['debt_ebitda'] - df['debt_ebitda_sector_median']) / df['debt_ebitda_sector_mad']
df['debt_invcap_sector_zscore'] = (df['debt_invcap'] - df['debt_invcap_sector_median']) / df['debt_invcap_sector_mad']
df['DIVYIELD_sector_zscore'] = (df['DIVYIELD'] - df['DIVYIELD_sector_median']) / df['DIVYIELD_sector_mad']
df['dltt_be_sector_zscore'] = (df['dltt_be'] - df['dltt_be_sector_median']) / df['dltt_be_sector_mad']
df['dpr_sector_zscore'] = (df['dpr'] - df['dpr_sector_median']) / df['dpr_sector_mad']
df['efftax_sector_zscore'] = (df['efftax'] - df['efftax_sector_median']) / df['efftax_sector_mad']
df['equity_invcap_sector_zscore'] = (df['equity_invcap'] - df['equity_invcap_sector_median']) / df['equity_invcap_sector_mad']
df['evm_sector_zscore'] = (df['evm'] - df['evm_sector_median']) / df['evm_sector_mad']
df['fcf_ocf_sector_zscore'] = (df['fcf_ocf'] - df['fcf_ocf_sector_median']) / df['fcf_ocf_sector_mad']
df['gpm_sector_zscore'] = (df['gpm'] - df['gpm_sector_median']) / df['gpm_sector_mad']
df['GProf_sector_zscore'] = (df['GProf'] - df['GProf_sector_median']) / df['GProf_sector_mad']
df['int_debt_sector_zscore'] = (df['int_debt'] - df['int_debt_sector_median']) / df['int_debt_sector_mad']
df['int_totdebt_sector_zscore'] = (df['int_totdebt'] - df['int_totdebt_sector_median']) / df['int_totdebt_sector_mad']
df['intcov_sector_zscore'] = (df['intcov'] - df['intcov_sector_median']) / df['intcov_sector_mad']
df['intcov_ratio_sector_zscore'] = (df['intcov_ratio'] - df['intcov_ratio_sector_median']) / df['intcov_ratio_sector_mad']
df['inv_turn_sector_zscore'] = (df['inv_turn'] - df['inv_turn_sector_median']) / df['inv_turn_sector_mad']
df['invt_act_sector_zscore'] = (df['invt_act'] - df['invt_act_sector_median']) / df['invt_act_sector_mad']
df['lt_debt_sector_zscore'] = (df['lt_debt'] - df['lt_debt_sector_median']) / df['lt_debt_sector_mad']
df['lt_ppent_sector_zscore'] = (df['lt_ppent'] - df['lt_ppent_sector_median']) / df['lt_ppent_sector_mad']
df['npm_sector_zscore'] = (df['npm'] - df['npm_sector_median']) / df['npm_sector_mad']
df['ocf_lct_sector_zscore'] = (df['ocf_lct'] - df['ocf_lct_sector_median']) / df['ocf_lct_sector_mad']
df['opmad_sector_zscore'] = (df['opmad'] - df['opmad_sector_median']) / df['opmad_sector_mad']
df['opmbd_sector_zscore'] = (df['opmbd'] - df['opmbd_sector_median']) / df['opmbd_sector_mad']
df['pay_turn_sector_zscore'] = (df['pay_turn'] - df['pay_turn_sector_median']) / df['pay_turn_sector_mad']
df['pcf_sector_zscore'] = (df['pcf'] - df['pcf_sector_median']) / df['pcf_sector_mad']
df['pe_exi_sector_zscore'] = (df['pe_exi'] - df['pe_exi_sector_median']) / df['pe_exi_sector_mad']
df['pe_inc_sector_zscore'] = (df['pe_inc'] - df['pe_inc_sector_median']) / df['pe_inc_sector_mad']
df['pe_op_basic_sector_zscore'] = (df['pe_op_basic'] - df['pe_op_basic_sector_median']) / df['pe_op_basic_sector_mad']
df['pe_op_dil_sector_zscore'] = (df['pe_op_dil'] - df['pe_op_dil_sector_median']) / df['pe_op_dil_sector_mad']
df['PEG_1yrforward_sector_zscore'] = (df['PEG_1yrforward'] - df['PEG_1yrforward_sector_median']) / df['PEG_1yrforward_sector_mad']
df['PEG_ltgforward_sector_zscore'] = (df['PEG_ltgforward'] - df['PEG_ltgforward_sector_median']) / df['PEG_ltgforward_sector_mad']
df['PEG_trailing_sector_zscore'] = (df['PEG_trailing'] - df['PEG_trailing_sector_median']) / df['PEG_trailing_sector_mad']
df['pretret_earnat_sector_zscore'] = (df['pretret_earnat'] - df['pretret_earnat_sector_median']) / df['pretret_earnat_sector_mad']
df['pretret_noa_sector_zscore'] = (df['pretret_noa'] - df['pretret_noa_sector_median']) / df['pretret_noa_sector_mad']
df['profit_lct_sector_zscore'] = (df['profit_lct'] - df['profit_lct_sector_median']) / df['profit_lct_sector_mad']
df['ps_sector_zscore'] = (df['ps'] - df['ps_sector_median']) / df['ps_sector_mad']
df['ptb_sector_zscore'] = (df['ptb'] - df['ptb_sector_median']) / df['ptb_sector_mad']
df['ptpm_sector_zscore'] = (df['ptpm'] - df['ptpm_sector_median']) / df['ptpm_sector_mad']
df['quick_ratio_sector_zscore'] = (df['quick_ratio'] - df['quick_ratio_sector_median']) / df['quick_ratio_sector_mad']
df['rd_sale_sector_zscore'] = (df['rd_sale'] - df['rd_sale_sector_median']) / df['rd_sale_sector_mad']
df['rect_act_sector_zscore'] = (df['rect_act'] - df['rect_act_sector_median']) / df['rect_act_sector_mad']
df['rect_turn_sector_zscore'] = (df['rect_turn'] - df['rect_turn_sector_median']) / df['rect_turn_sector_mad']
df['roa_sector_zscore'] = (df['roa'] - df['roa_sector_median']) / df['roa_sector_mad']
df['roce_sector_zscore'] = (df['roce'] - df['roce_sector_median']) / df['roce_sector_mad']
df['roe_sector_zscore'] = (df['roe'] - df['roe_sector_median']) / df['roe_sector_mad']
df['sale_equity_sector_zscore'] = (df['sale_equity'] - df['sale_equity_sector_median']) / df['sale_equity_sector_mad']
df['sale_invcap_sector_zscore'] = (df['sale_invcap'] - df['sale_invcap_sector_median']) / df['sale_invcap_sector_mad']
df['sale_nwc_sector_zscore'] = (df['sale_nwc'] - df['sale_nwc_sector_median']) / df['sale_nwc_sector_mad']
df['short_debt_sector_zscore'] = (df['short_debt'] - df['short_debt_sector_median']) / df['short_debt_sector_mad']
df['staff_sale_sector_zscore'] = (df['staff_sale'] - df['staff_sale_sector_median']) / df['staff_sale_sector_mad']
df['totdebt_invcap_sector_zscore'] = (df['totdebt_invcap'] - df['totdebt_invcap_sector_median']) / df['totdebt_invcap_sector_mad']
df['dvpspm_sector_zscore'] = (df['dvpspm'] - df['dvpspm_sector_median']) / df['dvpspm_sector_mad']
df['dvpsxm_sector_zscore'] = (df['dvpsxm'] - df['dvpsxm_sector_median']) / df['dvpsxm_sector_mad']
df['dvrate_sector_zscore'] = (df['dvrate'] - df['dvrate_sector_median']) / df['dvrate_sector_mad']
df['spcsrc_sector_zscore'] = (df['spcsrc'] - df['spcsrc_sector_median']) / df['spcsrc_sector_mad']
df['alpha_sector_zscore'] = (df['alpha'] - df['alpha_sector_median']) / df['alpha_sector_mad']
df['b_hml_sector_zscore'] = (df['b_hml'] - df['b_hml_sector_median']) / df['b_hml_sector_mad']
df['b_mkt_sector_zscore'] = (df['b_mkt'] - df['b_mkt_sector_median']) / df['b_mkt_sector_mad']
df['b_smb_sector_zscore'] = (df['b_smb'] - df['b_smb_sector_median']) / df['b_smb_sector_mad']
df['b_umd_sector_zscore'] = (df['b_umd'] - df['b_umd_sector_median']) / df['b_umd_sector_mad']
df['exret_sector_zscore'] = (df['exret'] - df['exret_sector_median']) / df['exret_sector_mad']
df['ivol_sector_zscore'] = (df['ivol'] - df['ivol_sector_median']) / df['ivol_sector_mad']
df['n_sector_zscore'] = (df['n'] - df['n_sector_median']) / df['n_sector_mad']
df['R2_sector_zscore'] = (df['R2'] - df['R2_sector_median']) / df['R2_sector_mad']
df['tvol_sector_zscore'] = (df['tvol'] - df['tvol_sector_median']) / df['tvol_sector_mad']
df['BUYPCT_sector_zscore'] = (df['BUYPCT'] - df['BUYPCT_sector_median']) / df['BUYPCT_sector_mad']
df['HOLDPCT_sector_zscore'] = (df['HOLDPCT'] - df['HOLDPCT_sector_median']) / df['HOLDPCT_sector_mad']
df['MEANREC_sector_zscore'] = (df['MEANREC'] - df['MEANREC_sector_median']) / df['MEANREC_sector_mad']
df['MEDREC_sector_zscore'] = (df['MEDREC'] - df['MEDREC_sector_median']) / df['MEDREC_sector_mad']
df['recup_sector_zscore'] = (df['recup'] - df['recup_sector_median']) / df['recup_sector_mad']
df['recdown_sector_zscore'] = (df['recdown'] - df['recdown_sector_median']) / df['recdown_sector_mad']
df['SELLPCT_sector_zscore'] = (df['SELLPCT'] - df['SELLPCT_sector_median']) / df['SELLPCT_sector_mad']
df['STDEV_sector_zscore'] = (df['STDEV'] - df['STDEV_sector_median']) / df['STDEV_sector_mad']

In [71]:
temp = df.describe().transpose()

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3817: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


In [72]:
temp.to_csv('describe.csv', sep=',')

In [73]:
return_all_rows(df.describe().transpose())

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3817: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,count,mean,std,min,25%,50%,75%,max
GVKEY,47539.0,7.069949e+04,7.112535e+04,1004.000000,1.263500e+04,2.606900e+04,1.473490e+05,3.153180e+05
forward_one_month_return,47539.0,-4.497048e-01,1.184465e+01,-29.797553,-5.865016e+00,-3.221996e-01,4.800478e+00,3.080945e+01
forward_two_month_return,47539.0,-1.184556e+00,1.452095e+01,-37.693105,-7.424227e+00,-5.799503e-01,5.431461e+00,3.445295e+01
forward_three_month_return,47539.0,-1.941632e+00,1.678223e+01,-45.064231,-7.954899e+00,-9.524738e-01,4.969398e+00,3.971608e+01
forward_four_month_return,47539.0,-3.535769e+00,1.693312e+01,-48.790612,-8.168741e+00,-2.309859e+00,2.787829e+00,3.660044e+01
forward_five_month_return,47539.0,-4.848162e+00,1.784743e+01,-52.906516,-7.525928e+00,-3.758622e+00,-4.299268e-01,3.924682e+01
forward_six_month_return,47539.0,-5.817913e+00,1.676854e+01,-53.567884,-4.649209e+00,-4.649209e+00,-4.649209e+00,3.428912e+01
forward_seven_month_return,47539.0,-6.240171e+00,1.645254e+01,-54.694885,-5.235602e+00,-5.235602e+00,-5.235602e+00,3.507209e+01
forward_eight_month_return,47539.0,-5.988397e+00,1.437782e+01,-51.702970,-5.071970e+00,-5.071970e+00,-5.071970e+00,2.964462e+01
forward_nine_month_return,47539.0,-4.708819e+00,1.284053e+01,-47.037707,-4.166667e+00,-4.166667e+00,-4.166667e+00,3.067325e+01


In [74]:
df.dtypes

GVKEY                                               int64
sector                                             object
year-month                                         object
forward_one_month_return                          float64
forward_two_month_return                          float64
forward_three_month_return                        float64
forward_four_month_return                         float64
forward_five_month_return                         float64
forward_six_month_return                          float64
forward_seven_month_return                        float64
forward_eight_month_return                        float64
forward_nine_month_return                         float64
forward_ten_month_return                          float64
forward_eleven_month_return                       float64
forward_twelve_month_return                       float64
forward_thirteen_month_return                     float64
forward_fourteen_month_return                     float64
forward_fiftee

In [75]:
# df - Clean - Clip Top/Bottom 3% and fill NaN's with median
for column in df:
    if (df[column].dtype == float):
        df[column] = clip_outliers(df[column])

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3817: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


In [76]:
return_all_rows(df.describe().transpose())

C:\Users\sconner\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3817: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,count,mean,std,min,25%,50%,75%,max
GVKEY,47539.0,7.069949e+04,7.112535e+04,1004.000000,1.263500e+04,2.606900e+04,1.473490e+05,3.153180e+05
forward_one_month_return,47539.0,-4.498101e-01,1.184432e+01,-29.797187,-5.865016e+00,-3.221996e-01,4.800478e+00,3.080557e+01
forward_two_month_return,47539.0,-1.184536e+00,1.452080e+01,-37.691730,-7.424227e+00,-5.799503e-01,5.431461e+00,3.445223e+01
forward_three_month_return,47539.0,-1.941732e+00,1.678160e+01,-45.061727,-7.954899e+00,-9.524738e-01,4.969398e+00,3.971025e+01
forward_four_month_return,47539.0,-3.535790e+00,1.693306e+01,-48.790571,-8.168741e+00,-2.309859e+00,2.787829e+00,3.659970e+01
forward_five_month_return,47539.0,-4.848292e+00,1.784705e+01,-52.906119,-7.525928e+00,-3.758622e+00,-4.299268e-01,3.924208e+01
forward_six_month_return,47539.0,-5.818021e+00,1.676818e+01,-53.567192,-4.649209e+00,-4.649209e+00,-4.649209e+00,3.428486e+01
forward_seven_month_return,47539.0,-6.240035e+00,1.645134e+01,-54.685427,-5.235602e+00,-5.235602e+00,-5.235602e+00,3.506716e+01
forward_eight_month_return,47539.0,-5.988419e+00,1.437754e+01,-51.701615,-5.071970e+00,-5.071970e+00,-5.071970e+00,2.964252e+01
forward_nine_month_return,47539.0,-4.708697e+00,1.284007e+01,-47.033360,-4.166667e+00,-4.166667e+00,-4.166667e+00,3.067297e+01
